###### Imports and Settings

In [1]:
import pandas as pd
#import geopandas as gpd
import numpy as np
import requests
import io
import pickle
import matplotlib.pyplot as plt
from collections import deque
from functools import reduce
%matplotlib inline
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
def percent(x, y):
    return (x/y)*100

## Calculations

# ALL:

In [3]:
data = pd.read_csv('../../Data/ACS20205YR.csv')
coldropper = data.set_index(['NAME', 'GEO_ID'])

In [4]:
data['Population'] = data['agebysex_total_series']

#variables for calculations, but datapoint chosen from another dataset
totalpopulation = data['agebysex_total_series']
whitealone = data['raceeth_white_alone']
blackafricanamericanalone = data['raceeth_blackafricanamerican_alone']
americanindianalaskanativealone = data['raceeth_americanindianalaskanative_alone']
asianalone = data['raceeth_asian_alone']
nativehawaiianotherpacificislanderalone = data['raceeth_nativehawaiianotherpacificislander_alone']
someotherracealone = data['raceeth_someotherrace_alone']
twoormoreraces = data['raceeth_twoormoreraces_alone']
hispanicorlatino = data['raceeth_hispanicorlatino']
nonminority = data['raceeth_whitealone_nothispanicorlatino']
minority = totalpopulation - nonminority
#health insurance
data['HealthCoverage:Total Series'] = data['healthcoverage_total_healthcare_series']
#total no health insurance
nohealthcare = [data['healthcoverage_mu6_wo'],data['healthcoverage_m6to18_wo'],data['healthcoverage_m19to25_wo'],data['healthcoverage_m26to34_wo'],
                data['healthcoverage_m35to44_wo'],data['healthcoverage_m45to54_wo'],data['healthcoverage_m55to64_wo'],data['healthcoverage_m65to74_wo'],
                data['healthcoverage_m75+_wo'],data['healthcoverage_fu6_wo'],data['healthcoverage_f6to18_wo'],data['healthcoverage_f19to25_wo'],
                data['healthcoverage_f26to34_wo'],data['healthcoverage_f35to44_wo'],data['healthcoverage_f45to54_wo'],data['healthcoverage_f55to64_wo'],
                data['healthcoverage_f65to74_wo'],data['healthcoverage_f75+_wo']]
data['HealthCoverage:None'] = sum(nohealthcare)
#% of total with No Health Insurance Coverage
data['HealthCoverage%: None'] = percent(data['HealthCoverage:None'],data['HealthCoverage:Total Series'])
#total with health insurance
healthcare = [data['healthcoverage_mu6_w'],data['healthcoverage_m6to18_w'],data['healthcoverage_m19to25_w'],data['healthcoverage_m26to34_w'],
                data['healthcoverage_m35to44_w'],data['healthcoverage_m45to54_w'],data['healthcoverage_m55to64_w'],data['healthcoverage_m65to74_w'],
                data['healthcoverage_m75+_w'],data['healthcoverage_fu6_w'],data['healthcoverage_f6to18_w'],data['healthcoverage_f19to25_w'],
                data['healthcoverage_f26to34_w'],data['healthcoverage_f35to44_w'],data['healthcoverage_f45to54_w'],data['healthcoverage_f55to64_w'],
                data['healthcoverage_f65to74_w'],data['healthcoverage_f75+_w']]
data['HealthCoverage: With Healthcare Coverage'] = sum(healthcare)
#% of total with Health Insurance Coverage
data['HealthCoverage%: With Healthcare Coverage'] = percent(data['HealthCoverage: With Healthcare Coverage'], data['HealthCoverage:Total Series'])
# Total with Health Insurance: Public Coverage
public = [data['healthcoverage_mu6_wpublic'],data['healthcoverage_m6to18_wpublic'],data['healthcoverage_m19to25_wpublic'],data['healthcoverage_m26to34_wpublic'],
          data['healthcoverage_m35to44_wpublic'],data['healthcoverage_m45to54_wpublic'],data['healthcoverage_m55to64_wpublic'],
          data['healthcoverage_m65to74_wpublic'],data['healthcoverage_m75+_wpublic'],data['healthcoverage_fu6_wpublic'],
          data['healthcoverage_f6to18_wpublic'],data['healthcoverage_f19to25_wpublic'],data['healthcoverage_f26to34_wpublic'],
          data['healthcoverage_f35to44_wpublic'],data['healthcoverage_f45to54_wpublic'],data['healthcoverage_f55to64_wpublic'],
          data['healthcoverage_f65to74_wpublic'],data['healthcoverage_f75+_wpublic']]
data['HealthCoverage: With Public Healthcare Coverage'] = sum(public)
# % of covered with public
data['HealthCoverage%: With Public Healthcare Coverage'] = percent(data['HealthCoverage: With Public Healthcare Coverage'], 
                                                                   data['HealthCoverage: With Healthcare Coverage'])
# Total with Health Insurance: Private Coverage
private = [data['healthcoverage_mu6_wprivate'],data['healthcoverage_m6to18_wprivate'],data['healthcoverage_m19to25_wprivate'],data['healthcoverage_m26to34_wprivate'],
          data['healthcoverage_m35to44_wprivate'],data['healthcoverage_m45to54_wprivate'],data['healthcoverage_m55to64_wprivate'],
          data['healthcoverage_m65to74_wprivate'],data['healthcoverage_m75+_wprivate'],data['healthcoverage_fu6_wprivate'],
          data['healthcoverage_f6to18_wprivate'],data['healthcoverage_f19to25_wprivate'],data['healthcoverage_f26to34_wprivate'],
          data['healthcoverage_f35to44_wprivate'],data['healthcoverage_f45to54_wprivate'],data['healthcoverage_f55to64_wprivate'],
          data['healthcoverage_f65to74_wprivate'],data['healthcoverage_f75+_wprivate']]
data['HealthCoverage: With Private Healthcare Coverage'] = sum(private)
# % of covered with private
data['HealthCoverage%: With Private Healthcare Coverage'] = percent(data['HealthCoverage: With Private Healthcare Coverage'], 
                                                                    data['HealthCoverage: With Healthcare Coverage'])
#small groups m and f
data['Male Under 5'] = data['age_m_u5']
data['Female Under 5'] = data['age_f_u5']
data['Male 5 to 9'] = data['age_m_5to9']
data['Female 5 to 9'] = data['age_f_5to9']
data['Male 5 to 14'] = data['age_m_5to9'] + data['age_m_10to14']
data['Female 5 to 14'] = data['age_f_5to9'] + data['age_f_10to14']
data['Male 15 to 24'] = data['age_m_15to17'] + data['age_m_18to19']+data['age_m_20']+data['age_m_21']+data['age_m_22to24']
data['Female 15 to 24'] = data['age_f_15to17'] + data['age_f_18to19']+data['age_f_20']+data['age_f_21']+data['age_f_22to24']
data['Male 10 to 14'] = data['age_m_10to14']
data['Female 10 to 14'] = data['age_f_10to14']
data['Male 15 to 17'] = data['age_m_15to17']
data['Female 15 to 17'] = data['age_f_15to17']
data['Male 18 to 24'] = data['age_m_18to19']+data['age_m_20']+data['age_m_21']+data['age_m_22to24']
data['Female 18 to 24'] = data['age_f_18to19']+data['age_f_20']+data['age_f_21']+data['age_f_22to24']
data['Male 25 to 34'] = data['age_m_25to29']+data['age_m_30to34']
data['Female 25 to 34'] = data['age_f_25to29']+data['age_f_30to34']
data['Male 35 to 44'] = data['age_m_35to39']+data['age_m_40to44']
data['Female 35 to 44'] = data['age_f_35to39']+data['age_f_40to44']
data['Male 45 to 54'] = data['age_m_45to49']+data['age_m_50to54']
data['Female 45 to 54'] = data['age_f_45to49']+data['age_f_50to54']
data['Male 55 to 64'] = data['age_m_55to59']+data['age_m_60to61']+data['age_m_62to64']
data['Female 55 to 64'] = data['age_f_55to59']+data['age_f_60to61']+data['age_f_62to64']
data['Male 65 to 74'] = data['age_m_65to66']+data['age_m_67to69']+data['age_m_70to74']
data['Female 65 to 74'] = data['age_f_65to66']+data['age_f_67to69']+data['age_f_70to74']
data['Male 75 to 84'] = data['age_m_75to79']+data['age_m_80to84']
data['Female 75 to 84'] = data['age_f_75to79']+data['age_f_80to84']
data['Male 85 and Older'] = data['age_m_85+']
data['Female 85 and Older'] = data['age_f_85+']
#age group breakouts
data['Age:Under 5'] = data['Male Under 5']+data['Female Under 5']
data['Age %:Under 5'] = percent(data['Age:Under 5'], data['Population'])
data['Age:5 to 9'] = data['Male 5 to 9']+data['Female 5 to 9']
data['Age%:5 to 9'] = percent(data['Age:5 to 9'], data['Population'])
data['Age:10 to 14'] = data['Male 10 to 14']+data['Female 10 to 14']
data['Age%:10 to 14'] = percent(data['Age:10 to 14'], data['Population'])
data['Age:15 to 17'] = data['Male 15 to 17']+data['Female 15 to 17']
data['Age%:15 to 17'] = percent(data['Age:15 to 17'], data['Population'])
data['Age:18 to 24'] = data['Male 18 to 24']+data['Female 18 to 24']
data['Age%:18 to 24'] = percent(data['Age:18 to 24'], data['Population'])
data['Age:25 to 34'] = data['Male 25 to 34']+data['Female 25 to 34']
data['Age%:25 to 34'] = percent(data['Age:25 to 34'], data['Population'])
data['Age:35 to 44'] = data['Male 35 to 44']+data['Female 35 to 44']
data['Age%:35 to 44'] = percent(data['Age:35 to 44'], data['Population'])
data['Age:45 to 54'] = data['Male 35 to 44']+data['Female 45 to 54']
data['Age%:45 to 54'] = percent(data['Age:45 to 54'], data['Population'])
data['Age:55 to 64'] = data['Male 55 to 64']+data['Female 55 to 64']
data['Age%:55 to 64'] = percent(data['Age:55 to 64'], data['Population'])
data['Age:65 to 74'] = data['Male 65 to 74']+data['Female 65 to 74']
data['Age%:65 to 74'] = percent(data['Age:65 to 74'], data['Population'])
data['Age:75 to 84'] = data['Male 75 to 84']+data['Female 75 to 84']
data['Age%:75 to 84'] = percent(data['Age:75 to 84'], data['Population'])
data['Age:85 and Older'] = data['Male 85 and Older']+data['Female 85 and Older']
data['Age%:85 and Older'] = percent(data['Age:85 and Older'], data['Population'])
data['Age:School Age 5 to 18'] = data['Age:5 to 9']+data['Age:10 to 14']+data['Age:15 to 17']
data['Age%:School Age 5 to 18'] = percent(data['Age:School Age 5 to 18'], data['Population'])
#age brackets
u18list = [data['Male Under 5'],data['Female Under 5'],data['Male 5 to 9'],data['Female 5 to 9'],data['Male 10 to 14'],data['Female 10 to 14'],data['Male 15 to 17'],
           data['Female 15 to 17']]
data['Age:Under 18'] = sum(u18list)
data['Age%:Under 18'] = percent(data['Age:Under 18'], data['Population'])
eighteento54list = [data['Male 18 to 24'],data['Female 18 to 24'],data['Male 25 to 34'],data['Female 25 to 34'],data['Male 35 to 44'],data['Female 35 to 44'],
              data['Male 45 to 54'],data['Female 45 to 54']]
data['Age:18 to 54'] = sum(eighteento54list)
data['Age%:18 to 54'] = percent(data['Age:18 to 54'], data['Population'])
#18to64
eighteento64list = [data['Male 18 to 24'],data['Female 18 to 24'],data['Male 25 to 34'],data['Female 25 to 34'],data['Male 35 to 44'],data['Female 35 to 44'],
                    data['Male 45 to 54'],data['Female 45 to 54'], data['Male 55 to 64'],data['Female 55 to 64']]
data['Age:18 to 64'] = sum(eighteento64list)
data['Age%:18 to 64'] = percent(data['Age:18 to 64'], data['Population'])
fifty5pluslist = [data['Male 55 to 64'],data['Female 55 to 64'],data['Male 65 to 74'],data['Female 65 to 74'],data['Male 75 to 84'],data['Female 75 to 84'],
                  data['Male 85 and Older'],data['Female 85 and Older']]
data['Age:55 and Older'] = sum(fifty5pluslist)
data['Age%:55 and Older'] = percent(data['Age:55 and Older'], data['Population'])
#65+
sixty5pluslist = [data['Male 65 to 74'],data['Female 65 to 74'],data['Male 75 to 84'],data['Female 75 to 84'],data['Male 85 and Older'],data['Female 85 and Older']]
data['Age:65 and Older'] = sum(sixty5pluslist)
data['Age%:65 and Older'] = percent(data['Age:65 and Older'], data['Population'])
#household type
data['Total Households'] = data['hhtype_total_series']
data['Family Households'] = data['hhtype_familyhh']
data['Family Households: Married Couple Family'] = data['hhtype_familyhh_marriedcouplefam']
data['Household Type%: Family, Married Couple Households'] = percent(data['Family Households: Married Couple Family'], data['hhtype_total_series'])
data['Family Households: Not Married Couple Family'] = data['hhtype_familyhh_otherfam']
data['Household Type%: Family, Not Married Couple Households'] = percent(data['Family Households: Not Married Couple Family'], data['hhtype_total_series'])
data['Family Households: Not Married Couple: Male no Spouse'] = data['hhtype_familyhh_malenospouse']
data['Family Households: Not Married Couple: Female no Spouse'] = data['hhtype_familyhh_femalenospouse']
data['Nonfamily Households'] = data['hhtype_nonfamhh']
data['Nonfamily Households: Householder Alone'] = data['hhtype_nonfamhh_householderalone']
data['Household Type%: Nonfamily, Householder Alone'] = percent(data['Nonfamily Households: Householder Alone'], data['hhtype_total_series'])
data['Nonfamily Households: Householder not Alone'] = data['hhtype_nonfamhh_householdernotalone']
data['Household Type%: Nonfamily, Householder Not Alone'] = percent(data['Nonfamily Households: Householder not Alone'], data['hhtype_total_series'])
#avg hh size
data['Average Household Size'] = data['hhsize_avg']
#median hh income
data['Median Household Income'] = data['hhincome_median']
#hh income brackets
data['HHIncome:Total Households'] = data['hhincome_total_series']
data['HHIncome:Less than 10,000'] = data['hhincome_lessthan10000']
data['HHIncome%:Less than 10,000'] = percent(data['HHIncome:Less than 10,000'], data['HHIncome:Total Households'])
data['HHIncome:10 to 14,999'] = data['hhincome_10to14999']
data['HHIncome%:10 to 14,999'] = percent(data['HHIncome:10 to 14,999'], data['HHIncome:Total Households'])
data['HHIncome:15 to 19,999'] = data['hhincome_15to19999']
data['HHIncome%:15 to 19,999'] = percent(data['HHIncome:15 to 19,999'], data['HHIncome:Total Households'])
data['HHIncome:20 to 24,999'] = data['hhincome_20to24999']
data['HHIncome%:20 to 24,999'] = percent(data['HHIncome:20 to 24,999'], data['HHIncome:Total Households'])
data['HHIncome:25 to 29,999'] = data['hhincome_25to29999']
data['HHIncome%:25 to 29,999'] = percent(data['HHIncome:25 to 29,999'], data['HHIncome:Total Households'])
data['HHIncome:30 to 34,999'] = data['hhincome_30to34999']
data['HHIncome:%30 to 34,999'] = percent(data['HHIncome:30 to 34,999'], data['HHIncome:Total Households'])
data['HHIncome:35 to 39,999'] = data['hhincome_35to39999']
data['HHIncome%:35 to 39,999'] = percent(data['HHIncome:35 to 39,999'], data['HHIncome:Total Households'])
data['HHIncome:40 to 44,999'] = data['hhincome_40to44999']
data['HHIncome%:40 to 44,999'] = percent(data['HHIncome:40 to 44,999'], data['HHIncome:Total Households'])
data['HHIncome:45 to 49,999'] = data['hhincome_45to49999']
data['HHIncome%:45 to 49,999'] = percent(data['HHIncome:45 to 49,999'], data['HHIncome:Total Households'])
data['HHIncome:50 to 59,999'] = data['hhincome_50to59999']
data['HHIncome%:50 to 59,999'] = percent(data['HHIncome:50 to 59,999'], data['HHIncome:Total Households'])
data['HHIncome:60 to 74,999'] = data['hhincome_60to74999']
data['HHIncome%:60 to 74,999'] = percent(data['HHIncome:60 to 74,999'], data['HHIncome:Total Households'])
data['HHIncome:75 to 99,999'] = data['hhincome_75to99999']
data['HHIncome%:75 to 99,999'] = percent(data['HHIncome:75 to 99,999'], data['HHIncome:Total Households'])
data['HHIncome:100 to 124,999'] = data['hhincome_100to124999']
data['HHIncome%:100 to 124,999'] = percent(data['HHIncome:100 to 124,999'], data['HHIncome:Total Households'])
data['HHIncome:125 to 149,999'] = data['hhincome125to149999']
data['HHIncome%:125 to 149,999'] = percent(data['HHIncome:125 to 149,999'], data['HHIncome:Total Households'])
data['HHIncome:150 to 199,999'] = data['hhincome150to199999']
data['HHIncome%:150 to 199,999'] = percent(data['HHIncome:150 to 199,999'], data['HHIncome:Total Households'])
data['HHIncome:200K or More'] = data['hhincome200ormore']
data['HHIncome%:200K or More'] = percent(data['HHIncome:200K or More'], data['HHIncome:Total Households'])
#educational attainment
data['Ed:Population 25+ Educational Attainment'] = data['attainment_total_over25_series']
lesshighschoollist = [data['attainment_noschooling'],data['attainment_nurseryschool'],data['attainment_kindergarten'],data['attainment_1stgrade'],
                      data['attainment_2ndgrade'],data['attainment_3rdgrade'],data['attainment_4thgrade'],data['attainment_5thgrade'],data['attainment_6thgrade'],
                      data['attainment_7thgrade'],data['attainment_8thgrade'],data['attainment_9thgrade'],data['attainment_10thgrade'],data['attainment_11thgrade'],
                      data['attainment_12thgradenodiploma']]
data['Ed:Less than High School'] = sum(lesshighschoollist)
data['Ed%:Less than High School'] = percent(data['Ed:Less than High School'], data['Ed:Population 25+ Educational Attainment'])
data['Ed:High School Graduate or Equivalency'] = data['attainment_regularhighschooldiploma']+data['attainment_gedoralternativecredential']
data['Ed%:High School Graduate or Equivalency'] = percent(data['Ed:High School Graduate or Equivalency'], data['Ed:Population 25+ Educational Attainment'])
highormore = [data['attainment_regularhighschooldiploma'],data['attainment_gedoralternativecredential'],data['attainment_somecollegelessthan1year'],
              data['attainment_somecollege1ormoreyearsnodegree'],data['attainment_associatesdegree'],data['attainment_bachelorsdegree'],
              data['attainment_mastersdegree'],data['attainment_professionalschooldegree'],data['attainment_doctoratedegree']]
data['Ed%:High School Graduate or More'] = percent(sum(highormore),data['Ed:Population 25+ Educational Attainment'])
data['Ed:Some College'] = data['attainment_somecollegelessthan1year']+data['attainment_somecollege1ormoreyearsnodegree']
data['Ed%:Some College'] = percent(data['Ed:Some College'], data['Ed:Population 25+ Educational Attainment'])
somecollegeormore = [data['attainment_somecollege1ormoreyearsnodegree'],data['attainment_associatesdegree'],data['attainment_bachelorsdegree'],
                     data['attainment_mastersdegree'],data['attainment_professionalschooldegree'],data['attainment_doctoratedegree']]
data['Ed%:Some College or More'] = percent(sum(somecollegeormore),data['Ed:Population 25+ Educational Attainment'])
data['Ed:Associates'] = data['attainment_associatesdegree']
data['Ed%:Associates'] = percent(data['Ed:Associates'], data['Ed:Population 25+ Educational Attainment'])
data['Ed:Bachelors'] = data['attainment_bachelorsdegree']
data['Ed%:Bachelors'] = percent(data['Ed:Bachelors'], data['Ed:Population 25+ Educational Attainment'])
bachormore = [data['attainment_bachelorsdegree'],data['attainment_mastersdegree'],data['attainment_professionalschooldegree'],data['attainment_doctoratedegree']]
data['Ed%:Bachelors or More'] = percent(sum(bachormore),data['Ed:Population 25+ Educational Attainment'])
data['Ed:Masters'] = data['attainment_mastersdegree']
mastersormore = [data['attainment_mastersdegree'],data['attainment_professionalschooldegree'],data['attainment_doctoratedegree']]
data['Ed%:Masters or More'] = percent(sum(mastersormore),data['Ed:Population 25+ Educational Attainment'])
data['Ed%:Masters'] = percent(data['Ed:Masters'], data['Ed:Population 25+ Educational Attainment'])
data['Ed:Professional School Degree'] = data['attainment_professionalschooldegree']
data['Ed%:Professional School Degree'] = percent(data['Ed:Professional School Degree'], data['Ed:Population 25+ Educational Attainment'])
proformore = [data['attainment_mastersdegree'],data['attainment_professionalschooldegree'],data['attainment_doctoratedegree']]
data['Ed%:Professional School Degree or More'] = percent(sum(proformore),data['Ed:Population 25+ Educational Attainment'])
data['Ed:Doctorate Degree'] = data['attainment_doctoratedegree']
data['Ed%:Doctorate Degree'] = percent(data['Ed:Doctorate Degree'], data['Ed:Population 25+ Educational Attainment'])
#industry employment
data['Ind:Employed Civilians 16+'] = data['industry_sexbyindustrycivilianpop_total_series']
aglist = [data['industry_m_agriculture_forestry_fishing_hunting_mining'],data['industry_f_agriculture_forestry_fishing_hunting_mining']]
data['Ind:Agriculture, Forestry, Fishing and Hunting, and Mining'] = sum(aglist)
data['Ind%:Agriculture, Forestry, Fishing and Hunting, and Mining']=percent(data['Ind:Agriculture, Forestry, Fishing and Hunting, and Mining'],
                                                                            data['Ind:Employed Civilians 16+'])
data['Ind:Agriculture, Forestry, Fishing and Hunting'] = data['industry_m_agmining_agriculture_forestry_fishing_hunting']+data['industry_f_agmining_agriculture_forestry_fishing_hunting']
data['Ind%:Agriculture, Forestry, Fishing and Hunting'] = percent(data['Ind:Agriculture, Forestry, Fishing and Hunting'],data['Ind:Employed Civilians 16+'])
data['Ind:Mining, Quarrying, Oil and Gas'] = data['industry_m_agmining_mining_quarrying_oilandgas']+data['industry_f_agmining_mining_quarrying_oilandgas']
data['Ind%:Mining, Quarrying, Oil and Gas'] = percent(data['Ind:Mining, Quarrying, Oil and Gas'],data['Ind:Employed Civilians 16+'])
data['Ind:Construction'] = data['industry_m_construction']+data['industry_f_construction']
data['Ind%:Construction']=percent(data['Ind:Construction'],data['Ind:Employed Civilians 16+'])
data['Ind:Manufacturing'] = data['industry_m_manufacturing'] + data['industry_f_manufacturing']
data['Ind%:Manufacturing']=percent(data['Ind:Manufacturing'],data['Ind:Employed Civilians 16+'])
data['Ind:Wholesale Trade'] = data['industry_m_wholesaletrade']+data['industry_f_wholesaletrade']
data['Ind%:Wholesale Trade']=percent(data['Ind:Wholesale Trade'],data['Ind:Employed Civilians 16+'])
data['Ind:Retail Trade'] = data['industry_m_retailtrade']+data['industry_f_retailtrade']
data['Ind%:Retail Trade']=percent(data['Ind:Retail Trade'],data['Ind:Employed Civilians 16+'])
transware = [data['industry_m_transportation_warehousing_utilities'],data['industry_f_transportation_warehousing_utilities']]
data['Ind:Transportation and Warehousing, Utilities'] = sum(transware)
data['Ind%:Transportation and Warehousing, Utilities']=percent(data['Ind:Transportation and Warehousing, Utilities'],data['Ind:Employed Civilians 16+'])
data['Ind:Information'] = data['industry_m_information']+data['industry_f_information']
data['Ind%:Information']=percent(data['Ind:Information'],data['Ind:Employed Civilians 16+'])
finins = [data['industry_m_finance_insurance_realestate_rental'],data['industry_f_finance_insurance_realestate_rental']]
data['Ind:Finance, Insurance, Real Estate and Rental and Leasing'] = sum(finins)
data['Ind%:Finance, Insurance, Real Estate and Rental and Leasing']=percent(data['Ind:Finance, Insurance, Real Estate and Rental and Leasing'],
                                                                            data['Ind:Employed Civilians 16+'])
profsci = [data['industry_m_professional_scientific_managementadmin_waste'],data['industry_f_professional_scientific_managementadmin_waste']]
data['Ind:Professional, Scientific, Management, Administrative, and Waste Management Services']= sum(profsci)
data['Ind%:Professional, Scientific, Management, Administrative, and Waste Management Services']=percent(data['Ind:Professional, Scientific, Management, Administrative, and Waste Management Services'],
                                                                                                         data['Ind:Employed Civilians 16+'])
proftech = [data['industry_m_profwaste_professional_scientific_technicalservices'],data['industry_f_profwaste_professional_scientific_technicalservices']]
data['Ind:Professional, Scientific, and Technical Services']= sum(proftech)
data['Ind%:Professional, Scientific, and Technical Services']=percent(data['Ind:Professional, Scientific, and Technical Services'],data['Ind:Employed Civilians 16+'])
managementcoent = [data['industry_m_profwaste_managementcompaniesenterprises'],data['industry_f_profwaste_managementcompaniesenterprises']]
data['Ind:Management of Companies and Enterprises']= sum(managementcoent)
data['Ind%:Management of Companies and Enterprises']=percent(data['Ind:Management of Companies and Enterprises'],data['Ind:Employed Civilians 16+'])
edhealth = [data['industry_m_educational_healthcare_socialassistance'],data['industry_f_educational_healthcare_socialassistance']]
data['Ind:Educational, Health and Social Services'] = sum(edhealth)
data['Ind%:Educational, Health and Social Services']=percent(data['Ind:Educational, Health and Social Services'],data['Ind:Employed Civilians 16+'])
artsrec = [data['industry_m_arts_entertainmentrec_accommodation_food'],data['industry_f_arts_entertainmentrec_accommodation_food']]
data['Ind:Arts, Entertainment, Recreation, Accommodation and Food Services'] = sum(artsrec)
data['Ind%:Arts, Entertainment, Recreation, Accommodation and Food Services']=percent(data['Ind:Arts, Entertainment, Recreation, Accommodation and Food Services'],
                                                                                      data['Ind:Employed Civilians 16+'])
data['Ind:Other Services (Except Public Administration)']=data['industry_m_otherservices_notpublicadmin']+data['industry_f_otherservices_notpublicadmin']
data['Ind%:Other Services (Except Public Administration)']=percent(data['Ind:Other Services (Except Public Administration)'],data['Ind:Employed Civilians 16+'])
data['Ind:Public Administration'] = data['industry_m_publicadmin']+data['industry_f_publicadmin']
data['Ind%:Public Administration']=percent(data['Ind:Public Administration'],data['Ind:Employed Civilians 16+'])
blueco = [data['industry_m_agmining_mining_quarrying_oilandgas'], data['industry_f_agmining_mining_quarrying_oilandgas'], 
          data['Ind:Construction'], data['Ind:Manufacturing'],
          data['Ind:Wholesale Trade'], data['Ind:Transportation and Warehousing, Utilities']]
data['Ind:Blue Collar'] = sum(blueco)
data['Ind%:Blue Collar'] = percent(data['Ind:Blue Collar'], data['Ind:Employed Civilians 16+'])
whiteco = [data['Ind:Information'], data['Ind%:Finance, Insurance, Real Estate and Rental and Leasing'], data['Ind:Professional, Scientific, and Technical Services'],
           data['Ind:Management of Companies and Enterprises'], data['Ind:Educational, Health and Social Services'], data['Ind:Public Administration']]
data['Ind:White Collar'] = sum(whiteco)
data['Ind%:White Collar'] = percent(data['Ind:White Collar'], data['Ind:Employed Civilians 16+'])
#occupational employment
data['Employed Civilians 16+ Occupational'] = data['occupation_sexbyoccupationcivilianpop_total_series']
manbusfin = [data['occupation_m_management'],data['occupation_f_management'],data['occupation_m_businessandfinancial'],
             data['occupation_f_businessandfinancial']]
data['Occ:Management, Business, and Financial Operations'] = sum(manbusfin)
data['Occ%:Management, Business, and Financial Operations'] = percent(data['Occ:Management, Business, and Financial Operations'],
                                                                      data['Employed Civilians 16+ Occupational'])
profandrelated = [data['occupation_m_businessandfinancial'], data['occupation_m_computerandmathematical'], data['occupation_m_architectureandengineering'],
                  data['occupation_m_lifephysicalsocialscience'], data['occupation_m_communityandsocialservices'], data['occupation_m_legal'],
                  data['occupation_m_educationtrainingandlibrary'], data['occupation_m_artsdesignentertainmentsportsandmedia'], 
                  data['occupation_m_healthcarepractitionersandtechnical'], data['occupation_f_businessandfinancial'], 
                  data['occupation_f_computerandmathematical'], data['occupation_f_architectureandengineering'], data['occupation_f_architectureandengineering'],
                  data['occupation_f_lifephysicalsocialscience'], data['occupation_f_communityandsocialservices'], data['occupation_f_legal'],
                  data['occupation_f_educationtrainingandlibrary'], data['occupation_f_artsdesignentertainmentsportsandmedia'],
                  data['occupation_f_healthcarepractitionersandtechnical']]
data['Occ:Professional and Related'] = sum(profandrelated)
data['Occ%:Professional and Related'] = percent(data['Occ:Professional and Related'],data['Employed Civilians 16+ Occupational'])
data['Occ:Healthcare Support'] = data['occupation_m_healthcaresupport']+data['occupation_f_healthcaresupport']
data['Occ%:Healthcare Support'] = percent(data['Occ:Healthcare Support'],data['Employed Civilians 16+ Occupational'])
data['Occ:Protective Services'] = data['occupation_m_protectiveservice']+data['occupation_f_protectiveservice']
data['Occ%:Protective Services'] = percent(data['Occ:Protective Services'],data['Employed Civilians 16+ Occupational'])
data['Occ:Food Preparation and Serving Related'] = data['occupation_m_foodprepandserving']+data['occupation_f_foodprepandserving']
data['Occ%:Food Preparation and Serving Related'] = percent(data['Occ:Food Preparation and Serving Related'],data['Employed Civilians 16+ Occupational'])
buildinggrounds = [data['occupation_m_buildingandgroundscleaningandmaintenance'],data['occupation_f_buildingandgroundscleaningandmaintenance']]
data['Occ:Building and Grounds Cleaning and Maintenance'] = sum(buildinggrounds)
data['Occ%:Building and Grounds Cleaning and Maintenance'] = percent(data['Occ:Building and Grounds Cleaning and Maintenance'],
                                                                     data['Employed Civilians 16+ Occupational'])
data['Occ:Personal Care and Service'] = data['occupation_m_personalcareandservice'] + data['occupation_f_personalcareandservice']
data['Occ%:Personal Care and Service'] = percent(data['Occ:Personal Care and Service'],data['Employed Civilians 16+ Occupational'])
data['Occ:Sales and Related'] = data['occupation_m_salesandrelated']+data['occupation_f_salesandrelated']
data['Occ%:Sales and Related'] = percent(data['Occ:Sales and Related'],data['Employed Civilians 16+ Occupational'])
data['Occ:Office and Administrative Support'] = data['occupation_m_officeandadmin'] + data['occupation_f_officeandadmin']
data['Occ%:Office and Administrative Support'] = percent(data['Occ:Office and Administrative Support'],data['Employed Civilians 16+ Occupational'])
data['Occ:Farming, Fishing, and Forestry'] = data['occupation_m_farmingfishingandforestry']+data['occupation_f_farmingfishingandforestry']
data['Occ%:Farming, Fishing, and Forestry'] = percent(data['Occ:Farming, Fishing, and Forestry'],data['Employed Civilians 16+ Occupational'])
constexmaint = [data['occupation_m_constructionandextraction'],data['occupation_m_installationmaintenanceandrepair'],data['occupation_f_constructionandextraction'],
               data['occupation_f_installationmaintenanceandrepair']]
data['Occ:Construction, Extraction, and Maintenance'] = sum(constexmaint)
data['Occ%:Construction, Extraction, and Maintenance'] = percent(data['Occ:Construction, Extraction, and Maintenance'],data['Employed Civilians 16+ Occupational'])
data['Occ:Production'] = data['occupation_m_production']+data['occupation_f_production']
data['Occ%:Production'] = percent(data['Occ:Production'],data['Employed Civilians 16+ Occupational'])
transpomat = [data['occupation_m_transportation'],data['occupation_f_transportation'],data['occupation_m_materialmoving'],data['occupation_f_materialmoving']]
data['Occ:Transportation and Material Moving'] = sum(transpomat)
data['Occ%:Transportation and Material Moving'] = percent(data['Occ:Transportation and Material Moving'],data['Employed Civilians 16+ Occupational'])
#sector
data['Sector: Total Workers'] = data['classworker_total_series']
data['Sector:Private'] = data['classworker_privateforprofit']
data['Sector%:Private'] = percent(data['Sector:Private'], data['Sector: Total Workers'])
data['Sector:Public'] = data['classworker_localgovt']+data['classworker_stategovt']+data['classworker_federalgovt']
data['Sector%:Public'] = percent(data['Sector:Public'], data['Sector: Total Workers'])
data['Sector:Self-Employed'] = data['classworker_privateforprofit_selfemployedincorporatedbusiness']+data['classworker_selfemployednonincorporatedbusiness']
data['Sector%:Self-Employed'] = percent(data['Sector:Self-Employed'], data['Sector: Total Workers'])
data['Sector:Non-Profit'] = data['classworker_privatenotforprofit']
data['Sector%:Non-Profit'] = percent(data['Sector:Non-Profit'], data['Sector: Total Workers'])
data['Sector:Unpaid Family Workers'] = data['classworker_unpaidfamilyworker']
data['Sector%:Unpaid Family Workers'] = percent(data['Sector:Unpaid Family Workers'], data['Sector: Total Workers'])
#employment status
armedforces = [data['lfstatus_m_16to19_inlf_armedforces'],data['lfstatus_m_20to21_inlf_armedforces'],data['lfstatus_m_22to24_inlf_armedforces'],
               data['lfstatus_m_25to29_inlf_armedforces'],data['lfstatus_m_30to34_inlf_armedforces'],data['lfstatus_m_35to44_inlf_armedforces'],
               data['lfstatus_m_45to54_inlf_armedforces'],data['lfstatus_m_55to59_inlf_armedforces'],data['lfstatus_m_60to61_inlf_armedforces'],
               data['lfstatus_m_62to64_inlf_armedforces'],
               data['lfstatus_f_16to19_inlf_armedforces'],data['lfstatus_f_20to21_inlf_armedforces'],
               data['lfstatus_f_22to24_inlf_armedforces'],data['lfstatus_f_25to29_inlf_armedforces'],data['lfstatus_f_30to34_inlf_armedforces'],
               data['lfstatus_f_35to44_inlf_armedforces'],data['lfstatus_f_45to54_inlf_armedforces'],data['lfstatus_f_55to59_inlf_armedforces'],
               data['lfstatus_f_60to61_inlf_armedforces'],data['lfstatus_f_62to64_inlf_armedforces']]
notinlf = [data['lfstatus_m_16to19_notinlf'],data['lfstatus_m_20to21_notinlf'],data['lfstatus_m_22to24_notinlf'],
               data['lfstatus_m_25to29_notinlf'],data['lfstatus_m_30to34_notinlf'],data['lfstatus_m_35to44_notinlf'],
               data['lfstatus_m_45to54_notinlf'],data['lfstatus_m_55to59_notinlf'],data['lfstatus_m_60to61_notinlf'],
               data['lfstatus_m_62to64_notinlf'],data['lfstatus_m_65to69_notinlf'],data['lfstatus_m_70to74_notinlf'],
               data['lfstatus_m_75andolder_notinlf'],
               data['lfstatus_f_16to19_notinlf'],data['lfstatus_f_20to21_notinlf'],
               data['lfstatus_f_22to24_notinlf'],data['lfstatus_f_25to29_notinlf'],data['lfstatus_f_30to34_notinlf'],
               data['lfstatus_f_35to44_notinlf'],data['lfstatus_f_45to54_notinlf'],data['lfstatus_f_55to59_notinlf'],
               data['lfstatus_f_60to61_notinlf'],data['lfstatus_f_62to64_notinlf'],data['lfstatus_f_65to69_notinlf'],
               data['lfstatus_f_70to74_notinlf'],data['lfstatus_f_75andolder_notinlf']]
civemployed = [data['lfstatus_m_16to19_inlf_civilian_employed'],data['lfstatus_m_20to21_inlf_civilian_employed'],data['lfstatus_m_22to24_inlf_civilian_employed'],
               data['lfstatus_m_25to29_inlf_civilian_employed'],data['lfstatus_m_30to34_inlf_civilian_employed'],data['lfstatus_m_35to44_inlf_civilian_employed'],
               data['lfstatus_m_45to54_inlf_civilian_employed'],data['lfstatus_m_55to59_inlf_civilian_employed'],data['lfstatus_m_60to61_inlf_civilian_employed'],
               data['lfstatus_m_62to64_inlf_civilian_employed'],data['lfstatus_m_65to69_inlf_employed'],data['lfstatus_m_70to74_inlf_employed'],
               data['lfstatus_m_75andolder_inlf_employed'],
               data['lfstatus_f_16to19_inlf_civilian_employed'],data['lfstatus_f_20to21_inlf_civilian_employed'],
               data['lfstatus_f_22to24_inlf_civilian_employed'],data['lfstatus_f_25to29_inlf_civilian_employed'],data['lfstatus_f_30to34_inlf_civilian_employed'],
               data['lfstatus_f_35to44_inlf_civilian_employed'],data['lfstatus_f_45to54_inlf_civilian_employed'],data['lfstatus_f_55to59_inlf_civilian_employed'],
               data['lfstatus_f_60to61_inlf_civilian_employed'],data['lfstatus_f_62to64_inlf_civilian_employed'],data['lfstatus_f_65to69_inlf_employed'],
               data['lfstatus_f_70to74_inlf_employed'],data['lfstatus_f_75andolder_inlf_employed']]
data['Employment:Employed Civilian'] = sum(civemployed)
civunemployed = [data['lfstatus_m_16to19_inlf_civilian_unemployed'],data['lfstatus_m_20to21_inlf_civilian_unemployed'],data['lfstatus_m_22to24_inlf_civilian_unemployed'],
               data['lfstatus_m_25to29_inlf_civilian_unemployed'],data['lfstatus_m_30to34_inlf_civilian_unemployed'],data['lfstatus_m_35to44_inlf_civilian_unemployed'],
               data['lfstatus_m_45to54_inlf_civilian_unemployed'],data['lfstatus_m_55to59_inlf_civilian_unemployed'],data['lfstatus_m_60to61_inlf_civilian_unemployed'],
               data['lfstatus_m_62to64_inlf_civilian_unemployed'],data['lfstatus_m_65to69_inlf_unemployed'],data['lfstatus_m_70to74_inlf_unemployed'],
               data['lfstatus_m_75andolder_inlf_unemployed'],
               data['lfstatus_f_16to19_inlf_civilian_unemployed'],data['lfstatus_f_20to21_inlf_civilian_unemployed'],
               data['lfstatus_f_22to24_inlf_civilian_unemployed'],data['lfstatus_f_25to29_inlf_civilian_unemployed'],data['lfstatus_f_30to34_inlf_civilian_unemployed'],
               data['lfstatus_f_35to44_inlf_civilian_unemployed'],data['lfstatus_f_45to54_inlf_civilian_unemployed'],data['lfstatus_f_55to59_inlf_civilian_unemployed'],
               data['lfstatus_f_60to61_inlf_civilian_unemployed'],data['lfstatus_f_62to64_inlf_civilian_unemployed'],data['lfstatus_f_65to69_inlf_unemployed'],
               data['lfstatus_f_70to74_inlf_unemployed'],data['lfstatus_f_75andolder_inlf_unemployed']]
data['Employment:Unemployed Civilian'] = sum(civunemployed)
totalemployment = data['Employment:Unemployed Civilian']+data['Employment:Employed Civilian']
data['Employment:Civilian Unemployment Rate%'] = percent(data['Employment:Unemployed Civilian'], totalemployment)
data['Employment:Armed Forces'] = sum(armedforces)
data['Employment:% of Labor Force Armed Forces'] = percent(data['Employment:Armed Forces'], totalemployment)
data['Employment:Not in Labor Force'] = sum(notinlf)
data['Employment:% of Population Not in Labor Force'] = percent(data['Employment:Not in Labor Force'], data['lfstatus_total_sexbyagebyemploymentstatus16+_series'])
data['Employment:In Labor Force'] = totalemployment + data['Employment:Armed Forces']
data['Employment:%In Labor Force'] = percent(data['Employment:In Labor Force'], data['lfstatus_total_sexbyagebyemploymentstatus16+_series'])
#per capita income
data['Per Capita Income'] = data['percapita_income']
#poverty
data['Poverty:Population for Whom Poverty Status is Determined'] = data['poverty_total_bysexbyage_series']
data['Poverty: Population Below Poverty Level'] = data['poverty_belowlevel']
data['Poverty%: Below Poverty Level'] = percent(data['Poverty: Population Below Poverty Level'], data['Poverty:Population for Whom Poverty Status is Determined'])
#cost burden
data['CB:Total Housing Units'] = data['housingcost_total_selectedownercosts%hhincome_series']+data['housingcost_total_rent%hhincome_series']
allcostburden = [data['housingcost_%ownercost30to34.9_wmortgage'],data['housingcost_%ownercost35to39.9_wmortgage'],data['housingcost_%ownercost40to49.9_wmortgage'],
                 data['housingcost_%ownercost50+_wmortgage'],data['housingcost_%ownercost30to34.9_womortgage'],data['housingcost_%ownercost35to39.9_womortgage'],
                 data['housingcost_%ownercost40to49.9_womortgage'],data['housingcost_%ownercost50+_womortgage'],data['housingcost_%rentercost30to34.9'],
                 data['housingcost_%rentercost35to39.9'],data['housingcost_%rentercost40to49.9'],data['housingcost_%rentercost50+']]
data['CB:Cost Burdened Households'] = sum(allcostburden)
data['CB:Renter Occupied Units'] = data['housingcost_total_rent%hhincome_series']
rentercostburden = [data['housingcost_%rentercost30to34.9'],data['housingcost_%rentercost35to39.9'],
                    data['housingcost_%rentercost40to49.9'],data['housingcost_%rentercost50+']]
data['CB:Cost Burdened Renters'] = sum(rentercostburden)
data['CB%:Cost Burdened Renters'] = percent(data['CB:Cost Burdened Renters'], data['CB:Renter Occupied Units'])
data['CB:Owner Occupied Units'] = data['housingcost_total_selectedownercosts%hhincome_series']
ownercostburden = [data['housingcost_%ownercost30to34.9_wmortgage'],data['housingcost_%ownercost35to39.9_wmortgage'],data['housingcost_%ownercost40to49.9_wmortgage'],
                   data['housingcost_%ownercost50+_wmortgage'],data['housingcost_%ownercost30to34.9_womortgage'],data['housingcost_%ownercost35to39.9_womortgage'],
                   data['housingcost_%ownercost40to49.9_womortgage'],data['housingcost_%ownercost50+_womortgage']]
data['CB:Cost Burdened Homeowners'] = sum(ownercostburden)
data['CB%: Cost Burdened Homeowners'] = percent(data['CB:Cost Burdened Homeowners'], data['CB:Owner Occupied Units'])
severecostburden = [data['housingcost_%ownercost50+_womortgage'],data['housingcost_%ownercost50+_wmortgage'],data['housingcost_%rentercost50+']]
data['CB:Severe Cost Burdened Households'] = sum(severecostburden)
data['CB%:Severe Cost Burdened Households'] = percent(data['CB:Severe Cost Burdened Households'],data['CB:Total Housing Units'])
ownerseverecostburden = [data['housingcost_%ownercost50+_womortgage'],data['housingcost_%ownercost50+_wmortgage']]
data['CB:Severe Cost Burdened Homeowners'] = sum(ownerseverecostburden)
data['CB%:Severe Cost Burdened Homeowners'] = percent(data['CB:Severe Cost Burdened Homeowners'],data['CB:Owner Occupied Units'])
renterseverecostburden = [data['housingcost_%rentercost50+']]
data['CB:Severe Cost Burdened Renters'] = sum(renterseverecostburden)
data['CB%:Severe Cost Burdened Renters'] = percent(data['CB:Severe Cost Burdened Renters'],data['CB:Renter Occupied Units'])
#add-ons
data['CB%:Cost Burdened Households'] = percent(data['CB:Cost Burdened Households'], data['CB:Total Housing Units'])
data['CB%:Cost Burdened Households Homeowners'] = percent(data['CB:Cost Burdened Homeowners'], data['CB:Cost Burdened Households'])
data['CB%:Cost Burdened Households Renters'] = percent(data['CB:Cost Burdened Renters'], data['CB:Cost Burdened Households'])
data['CB%:Severe Cost Burdened Households Homeowners'] = percent(data['CB:Severe Cost Burdened Homeowners'], data['CB:Severe Cost Burdened Households'])
data['CB%:Severe Cost Burdened Households Renters'] = percent(data['CB:Severe Cost Burdened Renters'], data['CB:Severe Cost Burdened Households'])
#total households
data['Total Households'] = data['units_allhousing']
#median home value
data['Median Home Value Owner Occupied with Mortgage'] = data['housingcost_medvalue_ownerocc_wmortgage']
data['Median Home Value Owner Occupied without Mortgage'] = data['housingcost_medvalue_ownerocc_womortgage']
#median gross rent
data['Median Gross Rent'] = data['housingcost_mediangrossrent_renteroccupied']
#median monthly owner costs
data['Median Monthly Owner Costs'] = data['housingcost_medcosts_ownerocc']
data['Median Monthly Owner Costs Home with Mortgage'] = data['housingcost_medcosts_ownerocc_mortgage']
data['Median Monthly Owner Costs Home without Mortgage'] = data['housingcost_medcosts_ownerocc_womortgage']
#tenure
data['Tenure:Total Households'] = data['tenure_total_series']
data['Tenure:Owners'] = data['tenure_owneroccunits']
data['Tenure%:Owners'] = percent(data['Tenure:Owners'], data['Tenure:Total Households'])
data['Tenure:Renters'] = data['tenure_renteroccunits']
data['Tenure%:Renters'] = percent(data['Tenure:Renters'], data['Tenure:Total Households'])
#year structure built
data['StructureAge:Total Structures'] = data['structures_total_yearbuilt_series']
data['StructureAge:Built 2014 or Later'] = data['structures_built2014orlater']
data['StructureAge%:Built 2014 or Later'] = percent(data['StructureAge:Built 2014 or Later'], data['StructureAge:Total Structures'])
data['StructureAge:Built 2010 to 2013'] = data['structures_built2010to2013']
data['StructureAge%:Built 2010 to 2013'] = percent(data['StructureAge:Built 2010 to 2013'], data['StructureAge:Total Structures'])
data['StructureAge:Built 2000 to 2009'] = data['structures_built2000to2009']
data['StructureAge%:Built 2000 to 2009'] = percent(data['StructureAge:Built 2000 to 2009'], data['StructureAge:Total Structures'])
data['StructureAge:Built 1990 to 1999'] = data['structures_built1990to1999']
data['StructureAge%:Built 1990 to 1999'] = percent(data['StructureAge:Built 1990 to 1999'], data['StructureAge:Total Structures'])
data['StructureAge:Built 1980 to 1989'] = data['structures_built1980to1989']
data['StructureAge%:Built 1980 to 1989'] = percent(data['StructureAge:Built 1980 to 1989'], data['StructureAge:Total Structures'])
data['StructureAge:Built 1970 to 1979'] = data['structures_built1970to1979']
data['StructureAge%: Built 1970 to 1979'] = percent(data['StructureAge:Built 1970 to 1979'], data['StructureAge:Total Structures'])
data['StructureAge:Built 1960 to 1969'] = data['structures_built1960to1969']
data['StructureAge%:Built 1960 to 1969'] = percent(data['StructureAge:Built 1960 to 1969'], data['StructureAge:Total Structures'])
data['StructureAge:Built 1950 to 1959'] = data['structures_built1950to1959']
data['StructureAge%:Built 1950 to 1959'] = percent(data['StructureAge:Built 1950 to 1959'], data['StructureAge:Total Structures'])
data['StructureAge:Built 1940 to 1949'] = data['structures_built1940to1949']
data['StructureAge%:Built 1940 to 1949'] = percent(data['StructureAge:Built 1940 to 1949'], data['StructureAge:Total Structures'])
data['StructureAge:Built 1939 or Earlier'] = data['structures_built1939orearlier']
data['StructureAge%:Built 1939 or Earlier'] = percent(data['StructureAge:Built 1939 or Earlier'], data['StructureAge:Total Structures'])
#median age of structure
data['StructureAge:Median Year Structure Built'] = data['structures_medianyearbuilt']  
data['StructureAge:Median Age of Structure'] = 2020 - data['StructureAge:Median Year Structure Built']
#units
data['Units:Total Series'] = data['units_total_series']
data['Units:1 Unit, Detached'] = data['units_one_detached']
data['Units%:1 Unit, Detached'] = percent(data['Units:1 Unit, Detached'], data['Units:Total Series'])
data['Units:1 Unit, Attached'] = data['units_one_attached']
data['Units%:1 Unit, Attached'] = percent(data['Units:1 Unit, Attached'], data['Units:Total Series'])
data['Units:2'] = data['units_two']
data['Units%:2'] = percent(data['Units:2'], data['Units:Total Series'])
data['Units:3 to 4'] = data['units_threetofour']
data['Units%:3 to 4'] = percent(data['Units:3 to 4'], data['Units:Total Series'])
data['Units:5 to 9'] = data['units_fivetonine']
data['Units%:5 to 9'] = percent(data['Units:5 to 9'], data['Units:Total Series'])
data['Units:10 to 19'] = data['units_tentonineteen']
data['Units%:10 to 19'] = percent(data['Units:10 to 19'], data['Units:Total Series'])
data['Units:20 to 49'] = data['units_twentytofortynine']
data['Units%:20 to 49'] = percent(data['Units:20 to 49'], data['Units:Total Series'])
data['Units:50 or More'] = data['units_fiftyormore']
data['Units%:50 or More'] = percent(data['Units:50 or More'], data['Units:Total Series'])
data['Units:Mobile Home'] = data['units_mobilehome']
data['Units%:Mobile Home'] = percent(data['Units:Mobile Home'], data['Units:Total Series'])
data['Units:Boat, RV, Van etc'] = data['units_boatrvvanetc']
data['Units%:Boat, RV, Van etc'] = percent(data['Units:Boat, RV, Van etc'], data['Units:Total Series'])
#means of transportation to work
data['Transpo:Workers Commuting'] = data['commute_total_meansoftransportationtowork_series']
data['Transpo:Car, Truck, or Van'] = data['commute_cartruckvan']
data['Transpo%:Car, Truck, or Van'] = percent(data['Transpo:Car, Truck, or Van'],data['Transpo:Workers Commuting'])
data['Transpo:Public Transportation'] = data['commute_publictransportation']
data['Transpo%:Public Transportation'] = percent(data['Transpo:Public Transportation'],data['Transpo:Workers Commuting'])
data['Transpo:Bicycle'] = data['commute_bicycle']
data['Transpo%:Bicycle'] = percent(data['Transpo:Bicycle'],data['Transpo:Workers Commuting'])
data['Transpo:Walk'] = data['commute_walk']
data['Transpo%:Walk'] = percent(data['Transpo:Walk'],data['Transpo:Workers Commuting'])
data['Transpo:Worked From Home'] = data['commute_workedfromhome']
data['Transpo%:Worked From Home'] = percent(data['Transpo:Worked From Home'],data['Transpo:Workers Commuting'])
data['Transpo:Taxi, Motorcycle, Other'] = data['commute_taxicabmotorcycleother']
data['Transpo%:Taxi, Motorcycle, Other'] = percent(data['Transpo:Taxi, Motorcycle, Other'],data['Transpo:Workers Commuting'])
#vehicle ownership
data['Vehicles:Households'] = data['vehicles_tenurebyvehicles_total_series']
data['Vehicles:None'] = data['vehicles_ownerocc_novehicle']+data['vehicles_renterocc_novehicle']
data['Vehicles%:None'] = percent(data['Vehicles:None'],data['Vehicles:Households'])
data['Vehicles:One'] = data['vehicles_ownerocc_1vehicle']+data['vehicles_renterocc_1vehicle']
data['Vehicles%:One'] = percent(data['Vehicles:One'],data['Vehicles:Households'])
data['Vehicles:Two'] = data['vehicles_ownerocc_2vehicles']+data['vehicles_renterocc_2vehicles']
data['Vehicles%:Two'] = percent(data['Vehicles:Two'],data['Vehicles:Households'])
data['Vehicles:Three'] = data['vehicles_ownerocc_3vehicles']+data['vehicles_renterocc_3vehicles']
data['Vehicles%:Three'] = percent(data['Vehicles:Three'],data['Vehicles:Households'])
data['Vehicles:Four'] = data['vehicles_ownerocc_4vehicles']+data['vehicles_renterocc_4vehicles']
data['Vehicles%:Four'] = percent(data['Vehicles:Four'],data['Vehicles:Households'])
data['Vehicles:Five or More'] = data['vehicles_ownerocc_5+vehicles']+data['vehicles_renterocc_5+vehicles']
data['Vehicles%:Five or More'] = percent(data['Vehicles:Five or More'],data['Vehicles:Households'])
#travel time by means of transportation to work
data['Commute:Workers 16+ Not Working at Home'] = data['traveltimemode_series_total']  
data['Commute:Less than 10 Minutes'] = data['traveltimemode_lessthan10']
data['Commute:10 to 14 Minutes'] = data['traveltimemode_10to14']
data['Commute:15 to 19 Minutes'] = data['traveltimemode_15to19']
data['Commute:20 to 24 Minutes'] = data['traveltimemode_20to24']
data['Commute:25 to 29 Minutes'] = data['traveltimemode_25to29']
data['Commute:30 to 34 Minutes'] = data['traveltimemode_30to34']
data['Commute:35 to 44 Minutes'] = data['traveltimemode_35to44']
data['Commute:45 to 59 Minutes'] = data['traveltimemode_45to59']
data['Commute:60 or More Minutes'] = data['traveltimemode_60ormore']
data['Commute:Car, Truck, Van Drove Alone:Less than 10 Minutes'] = data['traveltimemode_cartruckvan_drovealone_lessthan10']
data['Commute:Car, Truck, Van Drove Alone:10 to 14 Minutes'] = data['traveltimemode_cartruckvan_drovealone_10to14']
data['Commute:Car, Truck, Van Drove Alone:15 to 19 Minutes'] = data['traveltimemode_cartruckvan_drovealone_15to19']
data['Commute:Car, Truck, Van Drove Alone:20 to 24 Minutes'] = data['traveltimemode_cartruckvan_drovealone_20to24']
data['Commute:Car, Truck, Van Drove Alone:25 to 29 Minutes'] = data['traveltimemode_cartruckvan_drovealone_25to29']
data['Commute:Car, Truck, Van Drove Alone:30 to 34 Minutes'] = data['traveltimemode_cartruckvan_drovealone_30to34']
data['Commute:Car, Truck, Van Drove Alone:35 to 44 Minutes'] = data['traveltimemode_cartruckvan_drovealone_35to44']
data['Commute:Car, Truck, Van Drove Alone:45 to 59 Minutes'] = data['traveltimemode_cartruckvan_drovealone_45to59']
data['Commute:Car, Truck, Van Drove Alone:60 or More Minutes'] = data['traveltimemode_cartruckvan_drovealone_60ormore']
data['Commute:Car, Truck, Van Carpooled:Less than 10 Minutes'] = data['traveltimemode_cartruckvan_carpooled_lessthan10']
data['Commute:Car, Truck, Van Carpooled:10 to 14 Minutes'] = data['traveltimemode_cartruckvan_carpooled_10to14']
data['Commute:Car, Truck, Van Carpooled:15 to 19 Minutes'] = data['traveltimemode_cartruckvan_carpooled_15to19']
data['Commute:Car, Truck, Van Carpooled:20 to 24 Minutes'] = data['traveltimemode_cartruckvan_carpooled_20to24']
data['Commute:Car, Truck, Van Carpooled:25 to 29 Minutes'] = data['traveltimemode_cartruckvan_carpooled_25to29']
data['Commute:Car, Truck, Van Carpooled:30 to 34 Minutes'] = data['traveltimemode_cartruckvan_carpooled_30to34']
data['Commute:Car, Truck, Van Carpooled:35 to 44 Minutes'] = data['traveltimemode_cartruckvan_carpooled_35to44']
data['Commute:Car, Truck, Van Carpooled:45 to 59 Minutes'] = data['traveltimemode_cartruckvan_carpooled_45to59']
data['Commute:Car, Truck, Van Carpooled:60 or More Minutes'] = data['traveltimemode_cartruckvan_carpooled_60ormore']
data['Commute:Walked:Less than 10 Minutes'] = data['traveltimemode_walked_lessthan10']
data['Commute:Walked:10 to 14 Minutes'] = data['traveltimemode_walked_10to14']
data['Commute:Walked:15 to 19 Minutes'] = data['traveltimemode_walked_15to19']
data['Commute:Walked:20 to 24 Minutes'] = data['traveltimemode_walked_20to24']
data['Commute:Walked:25 to 29 Minutes'] = data['traveltimemode_walked_25to29']
data['Commute:Walked:30 to 34 Minutes'] = data['traveltimemode_walked_30to34']
data['Commute:Walked:35 to 44 Minutes'] = data['traveltimemode_walked_35to44']
data['Commute:Walked:45 to 59 Minutes'] = data['traveltimemode_walked_45to59']
data['Commute:Walked:60 or More Minutes'] = data['traveltimemode_walked_60ormore']
data['Commute:Walked:Less than 10 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_lessthan10']
data['Commute:Taxi, Motorcycle, Bicycle, Other:10 to 14 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_10to14']
data['Commute:Taxi, Motorcycle, Bicycle, Other:15 to 19 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_15to19']
data['Commute:Taxi, Motorcycle, Bicycle, Other:20 to 24 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_20to24']
data['Commute:Taxi, Motorcycle, Bicycle, Other:25 to 29 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_25to29']
data['Commute:Taxi, Motorcycle, Bicycle, Other:30 to 34 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_30to34']
data['Commute:Taxi, Motorcycle, Bicycle, Other:35 to 44 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_35to44']
data['Commute:Taxi, Motorcycle, Bicycle, Other:45 to 59 Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_45to59']
data['Commute:Taxi, Motorcycle, Bicycle, Other:60 or More Minutes'] = data['traveltimemode_taximotorcyclebicycleorother_60ormore']
#vet status
data['Vet:Population 18+'] = data['veteran_total_series']
data['Vet:Veteran'] = data['veteran_total_veteran']
data['Vet%:Veteran'] = percent(data['Vet:Veteran'],data['Vet:Population 18+'])
data['Vet:Nonveteran'] = data['veteran_total_nonveteran']
data['Vet%:Nonveteran'] = percent(data['Vet:Nonveteran'],data['Vet:Population 18+'])
total1864 = [data['veteran_m18to34'],data['veteran_m35to54'],data['veteran_m55to64'],data['veteran_f18to34'],data['veteran_f35to54'],data['veteran_f55to64']]
vet1864 = [data['veteran_m18to34_veteran'],data['veteran_m35to54_veteran'],data['veteran_m55to64_veteran'],data['veteran_f18to34_veteran'],
           data['veteran_f35to54_veteran'],data['veteran_f55to64_veteran']]
data['Vet:Veteran 18 to 64'] = sum(vet1864)
data['Vet%:Veterans who are 18 to 64'] = percent(data['Vet:Veteran 18 to 64'], sum(total1864))
total65up = [data['veteran_m65to74'],data['veteran_m75+'],data['veteran_f65to74'],data['veteran_f75+']]
vet65up = [data['veteran_m65to74_veteran'],data['veteran_m75+_veteran'],data['veteran_f65to74_veteran'],data['veteran_f75+_veteran']]
data['Vet:Veteran 65 and Older'] = sum(vet65up)
data['Vet%:Veterans who are 65 and Older'] = percent(data['Vet:Veteran 65 and Older'], sum(total65up))
nonvet1864 = [data['veteran_m18to34_nonveteran'],data['veteran_m35to54_nonveteran'],data['veteran_m55to64_nonveteran'],data['veteran_f18to34_nonveteran'],
           data['veteran_f35to54_nonveteran'],data['veteran_f55to64_nonveteran']]
data['Vet:Nonveteran 18 to 64'] = sum(nonvet1864)
nonvet65up = [data['veteran_m65to74_nonveteran'],data['veteran_m75+_nonveteran'],data['veteran_f65to74_nonveteran'],data['veteran_f75+_nonveteran']]
data['Vet:Nonveteran 65 and Older'] = sum(vet65up)
#disability
data['Disability:Total Population'] = data['disability_sexbyage_total_series']
disability = [data['disability_mu5_disability'],data['disability_m5to17_disability'],data['disability_m18to34_disability'],data['disability_m35to64_disability'],
              data['disability_m65to74_disability'],data['disability_m75+_disability'],data['disability_fu5_disability'],data['disability_f5to17_disability'],
              data['disability_f18to34_disability'],data['disability_f35to64_disability'],data['disability_f65to74_disability'],data['disability_f75+_disability']]
data['Disability:With Disability'] = sum(disability)
data['Disability%:With Disability'] = percent(data['Disability:With Disability'],data['Disability:Total Population'])
hearing = [data['disability_mu5_hearingdifficulty'],data['disability_m5to17_hearingdifficulty'],data['disability_m18to34_hearingdifficulty'],
           data['disability_m35to64_hearingdifficulty'],data['disability_m65to74_hearingdifficulty'],data['disability_m75+_hearingdifficulty'],
           data['disability_fu5_hearingdifficulty'],data['disability_f5to17_hearingdifficulty'],data['disability_f18to34_hearingdifficulty'],
           data['disability_f35to64_hearingdifficulty'],data['disability_f65to74_hearingdifficulty'],data['disability_f75+_hearingdifficulty']]
data['Disability:Hearing Difficulty'] = sum(hearing)
data['Disability%:Hearing Difficulty'] = percent(data['Disability:Hearing Difficulty'], data['Disability:Total Population'])
vision = [data['disability_mu5_visiondifficulty'],data['disability_m5to17_visiondifficulty'],data['disability_m18to34_visiondifficulty'],
          data['disability_m35to64_visiondifficulty'],data['disability_m65to74_visiondifficulty'],data['disability_m75+_visiondifficulty'],
          data['disability_fu5_visiondifficulty'],data['disability_f5to17_visiondifficulty'],data['disability_f18to34_visiondifficulty'],
          data['disability_f35to64_visiondifficulty'],data['disability_f65to74_visiondifficulty'],data['disability_f75+_visiondifficulty']]
data['Disability:Vision Difficulty'] = sum(vision)
data['Disability%:Vision Difficulty'] = percent(data['Disability:Vision Difficulty'], data['Disability:Total Population'])
cognitive = [data['disability_m5to17_cognitivedifficulty'],data['disability_m18to34_cognitivedifficulty'],data['disability_m35to64_cognitivedifficulty'],
             data['disability_m65to74_cognitivedifficulty'],data['disability_m75+_cognitivedifficulty'],data['disability_f5to17_cognitivedifficulty'],
             data['disability_f18to34_cognitivedifficulty'],data['disability_f35to64_cognitivedifficulty'],data['disability_f65to74_cognitivedifficulty'],
             data['disability_f75+_cognitivedifficulty']]
data['Disability:Cognitive Difficulty'] = sum(cognitive)
data['Disability%:Cognitive Difficulty'] = percent(data['Disability:Cognitive Difficulty'], data['Disability:Total Population'])
ambulatory = [data['disability_m5to17_ambulatorydifficulty'],data['disability_m18to34_ambulatorydifficulty'],
             data['disability_m35to64_ambulatorydifficulty'],data['disability_m65to74_ambulatorydifficulty'],data['disability_m75+_ambulatorydifficulty'],
             data['disability_f5to17_ambulatorydifficulty'],data['disability_f18to34_ambulatorydifficulty'],
             data['disability_f35to64_ambulatorydifficulty'],data['disability_f65to74_ambulatorydifficulty'],data['disability_f75+_ambulatorydifficulty']]
data['Disability:Ambulatory Difficulty'] = sum(ambulatory)
data['Disability%:Ambulatory Difficulty'] = percent(data['Disability:Ambulatory Difficulty'], data['Disability:Total Population'])
selfcare = [data['disability_m5to17_selfcaredifficulty'],data['disability_m18to34_selfcaredifficulty'],data['disability_m35to64_selfcaredifficulty'],
            data['disability_m65to74_selfcaredifficulty'],data['disability_m75+_selfcaredifficulty'],data['disability_f5to17_selfcaredifficulty'],
            data['disability_f18to34_selfcaredifficulty'],data['disability_f35to64_selfcaredifficulty'],data['disability_f65to74_selfcaredifficulty'],
            data['disability_f75+_selfcaredifficulty']]
data['Disability:Self-Care Difficulty'] = sum(selfcare)
data['Disability%:Self-Care Difficulty'] = percent(data['Disability:Self-Care Difficulty'], data['Disability:Total Population'])
indliving = [data['disability_m18to34_independentlivingdifficulty'],
             data['disability_m35to64_independentlivingdifficulty'],data['disability_m65to74_independentlivingdifficulty'],
             data['disability_m75+_independentlivingdifficulty'],
             data['disability_f18to34_independentlivingdifficulty'],data['disability_f35to64_independentlivingdifficulty'],
             data['disability_f65to74_independentlivingdifficulty'],data['disability_f75+_independentlivingdifficulty']]
data['Disability:Independent Living Difficulty'] = sum(indliving)
data['Disability%:Independent Living Difficulty'] = percent(data['Disability:Independent Living Difficulty'], data['Disability:Total Population'])
data['Disability:Total Disabilities'] = sum(hearing)+sum(vision)+sum(cognitive)+sum(ambulatory)+sum(selfcare)+sum(indliving)

onedisability = [data['disability_u18_1disability'],data['disability_18to64_1disability'],data['disability_65+_1disability']]
data['Disability:Population with One Disability'] = sum(onedisability)
data['Disability%:Population with One Disability'] = percent(data['Disability:Population with One Disability'], data['disability_agebynumber_series_total'])
twomoredisability = [data['disability_u18_2ormoredisabilities'],data['disability_18to64_2ormoredisabilities'],data['disability_65+_2ormoredisabilities']]
data['Disability:Population with Two or More Disabilities'] = sum(twomoredisability)
data['Disability%:Population with Two or More Disabilities'] = percent(data['Disability:Population with Two or More Disabilities'], 
                                                                       data['disability_agebynumber_series_total'])
#geographic mobility
data['GeoMobility:Total'] = data['geo_total_series']
data['GeoMobility:Same House'] = data['geo_samehouse_total']
data['GeoMobility%:Same House'] = percent(data['GeoMobility:Same House'],data['GeoMobility:Total'])
data['GeoMobility:Moved Same County'] = data['geo_movedsamecounty_total']
data['GeoMobility%:Moved Same County'] = percent(data['GeoMobility:Moved Same County'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different County Same State'] = data['geo_moveddifferentcountysamestate_total']
data['GeoMobility%:Moved Different County Same State'] = percent(data['GeoMobility:Moved Different County Same State'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different State'] = data['geo_moveddifferentstate_total']
data['GeoMobility%:Moved Different State'] = percent(data['GeoMobility:Moved Different State'],data['GeoMobility:Total'])
data['GeoMobility:Moved From Abroad'] = data['geo_movedabroad_total']
data['GeoMobility%:Moved From Abroad'] = percent(data['GeoMobility:Moved From Abroad'],data['GeoMobility:Total'])
#geographic mobility seniors
thelist = [data['geo_65to69_total'], data['geo_70to74_total'], data['geo_75+_total']]
sixtyfiveovertotal = sum(thelist)
thelist = [data['geo_65to69_total'], data['geo_70to74_total'], data['geo_75+_total'], data['geo_55to59_total'], data['geo_60to64_total']]
fiftyfiveovertotal = sum(thelist)
thelist = [data['geo_55to59_samehouse'], data['geo_60to64_samehouse'], data['geo_65to69_samehouse'], data['geo_70to74_samehouse'], data['geo_75+_samehouse']]
data['GeoMobility55Over:Same House'] = sum(thelist)
data['GeoMobility55Over%:Same House'] = percent(data['GeoMobility55Over:Same House'], fiftyfiveovertotal)
thelist = [data['geo_65to69_samehouse'], data['geo_70to74_samehouse'], data['geo_75+_samehouse']]
data['GeoMobility65Over:Same House'] = sum(thelist)
data['GeoMobility65Over%:Same House'] = percent(data['GeoMobility65Over:Same House'], sixtyfiveovertotal)
thelist = [data['geo_55to59_movedsamecounty'], data['geo_60to64_movedsamecounty'], data['geo_65to69_movedsamecounty'], 
           data['geo_70to74_movedsamecounty'], data['geo_75+_movedsamecounty']]
data['GeoMobility55Over:Moved Same County'] = sum(thelist)
data['GeoMobility55Over%:Moved Same County'] = percent(data['GeoMobility55Over:Moved Same County'], fiftyfiveovertotal)
thelist = [data['geo_65to69_movedsamecounty'], data['geo_70to74_movedsamecounty'], data['geo_75+_movedsamecounty']]
data['GeoMobility65Over:Moved Same County'] = sum(thelist)
data['GeoMobility65Over%:Moved Same County'] = percent(data['GeoMobility65Over:Moved Same County'], sixtyfiveovertotal)
thelist = [data['geo_55to59_moveddifferentcountysamestate'], data['geo_60to64_moveddifferentcountysamestate'], data['geo_65to69_moveddifferentcountysamestate'], 
           data['geo_70to74_moveddifferentcountysamestate'], data['geo_75+_moveddifferentcountysamestate']]
data['GeoMobility55Over:Moved Different County Same State'] = sum(thelist)
data['GeoMobility55Over%:Moved Different County Same State'] = percent(data['GeoMobility55Over:Moved Different County Same State'], fiftyfiveovertotal)
thelist = [data['geo_65to69_moveddifferentcountysamestate'], data['geo_70to74_moveddifferentcountysamestate'], data['geo_75+_moveddifferentcountysamestate']]
data['GeoMobility65Over:Moved Different County Same State'] = sum(thelist)
data['GeoMobility65Over%:Moved Different County Same State'] = percent(data['GeoMobility65Over:Moved Different County Same State'], sixtyfiveovertotal)
thelist = [data['geo_55to59_moveddifferentstate'], data['geo_60to64_moveddifferentstate'], data['geo_65to69_moveddifferentstate'], 
           data['geo_70to74_moveddifferentstate'], data['geo_75+_moveddifferentstate']]
data['GeoMobility55Over:Moved Different State'] = sum(thelist)
data['GeoMobility55Over%:Moved Different State'] = percent(data['GeoMobility55Over:Moved Different State'], fiftyfiveovertotal)
thelist = [data['geo_65to69_moveddifferentstate'], data['geo_70to74_moveddifferentstate'], data['geo_75+_moveddifferentstate']]
data['GeoMobility65Over:Moved Different State'] = sum(thelist)
data['GeoMobility65Over%:Moved Different State'] = percent(data['GeoMobility65Over:Moved Different State'], sixtyfiveovertotal)
thelist = [data['geo_55to59_movedabroad'], data['geo_60to64_movedabroad'], data['geo_65to69_movedabroad'], 
           data['geo_70to74_movedabroad'], data['geo_75+_movedabroad']]
data['GeoMobility55Over:Moved From Abroad'] = sum(thelist)
data['GeoMobility55Over%:Moved From Abroad'] = percent(data['GeoMobility55Over:Moved From Abroad'], fiftyfiveovertotal)
thelist = [data['geo_65to69_movedabroad'], data['geo_70to74_movedabroad'], data['geo_75+_movedabroad']]
data['GeoMobility65Over:Moved From Abroad'] = sum(thelist)
data['GeoMobility65Over%:Moved From Abroad'] = percent(data['GeoMobility65Over:Moved From Abroad'], sixtyfiveovertotal)
#foreign born citizenship
ForeignBornTotal = data['fb_total_series']
data['ForeignBorn:Native Born'] = data['fb_nativeborn']
data['ForeignBorn%:Native Born'] = percent(data['ForeignBorn:Native Born'], ForeignBornTotal)
data['ForeignBorn:Foreign Born'] = data['fb_foreignborn']
data['ForeignBorn%:Foreign Born'] = percent(data['ForeignBorn:Foreign Born'], ForeignBornTotal)
data['ForeignBorn:Foreign Born Naturalized Citizen'] = data['fb_foreignborn_naturalizeduscitizen']
data['ForeignBorn%:Foreign Born Population that are Naturalized Citizen'] = percent(data['ForeignBorn:Foreign Born Naturalized Citizen'],
                                                                                    data['ForeignBorn:Foreign Born'])
data['ForeignBorn:Foreign Born Not a Citizen'] = data['fb_foreignborn_notauscitizen']
data['ForeignBorn%:Foreign Born Population that are Not Naturalized Citizen'] = percent(data['ForeignBorn:Foreign Born Not a Citizen'],
                                                                                        data['ForeignBorn:Foreign Born'])
#foreign born origin
data['Foreign Born Population'] = data['foreignborn_total']
data['ForeignBorn:Europe'] = data['fb_europe']
data['ForeignBorn:Asia'] = data['fb_asia']
data['ForeignBorn:Africa'] = data['fb_africa']
data['ForeignBorn:Oceania'] = data['fb_oceania']
data['ForeignBorn:Central & South America, Caribbean'] = data['fb_am_latinamerica']
data['ForeignBorn:Canada & Other Northern American'] = data['fb_am_northern']
data['ForeignBorn%:Europe'] = percent(data['ForeignBorn:Europe'], data['Foreign Born Population'])
data['ForeignBorn%:Asia'] = percent(data['ForeignBorn:Asia'], data['Foreign Born Population'])
data['ForeignBorn%:Africa'] = percent(data['ForeignBorn:Africa'], data['Foreign Born Population'])
data['ForeignBorn%:Oceania'] = percent(data['ForeignBorn:Oceania'], data['Foreign Born Population'])
data['ForeignBorn%:Central & South America, Caribbean'] = percent(data['ForeignBorn:Central & South America, Caribbean'], data['Foreign Born Population'])
data['ForeignBorn%:Canada & Other Northern American'] = percent(data['ForeignBorn:Canada & Other Northern American'], data['Foreign Born Population'])
#language spoken at home by ability to speak english
data['Language:Population 5 and Older'] = data['language_languagebyabilitytospeakenglish_total5+_series']
onlyenglish = [data['language_speakonlyenglish_5to17'],data['language_speakonlyenglish_18to64'],data['language_speakonlyenglish_65+']]
data['Language:Speak Only English'] = sum(onlyenglish)
data['Language%:Speak Only English'] = percent(data['Language:Speak Only English'],data['Language:Population 5 and Older'])
besidesenglish = data['Language:Population 5 and Older'] - data['Language:Speak Only English']
#spanish
speakspanish = [data['language_speakspanish_5to17'],data['language_speakspanish_18to64'],data['language_speakspanish_65+']]
data['Language:Speak Spanish'] = sum(speakspanish)
data['Language%:Speak Spanish'] = percent(data['Language:Speak Spanish'], data['Language:Population 5 and Older'])
data['Language%:Non-English Speak Spanish'] = percent(data['Language:Speak Spanish'], besidesenglish)
spanishverywell = [data['language_speakspanish_speakenglishverywell_5to17'],data['language_speakspanish_speakenglishverywell_18to64'],
                   data['language_speakspanish_speakenglishverywell_65+']]
data['Language:Spanish:Speak English Very Well'] = sum(spanishverywell)
data['Language:Spanish%:Speak English Very Well'] = percent(data['Language:Spanish:Speak English Very Well'], data['Language:Speak Spanish'])
spanishwell = [data['language_speakspanish_speakenglishwell_5to17'],data['language_speakspanish_speakenglishwell_18to64'],
                   data['language_speakspanish_speakenglishwell_65+']]
data['Language:Spanish:Speak English Well'] = sum(spanishwell)
data['Language:Spanish%:Speak English Well'] = percent(data['Language:Spanish:Speak English Well'], data['Language:Speak Spanish'])
data['Language:Spanish:Speak English Well or Very Well'] = data['Language:Spanish:Speak English Very Well'] + data['Language:Spanish:Speak English Well']
data['Language:Spanish%:Speak English Well or Very Well'] = percent(data['Language:Spanish:Speak English Well or Very Well'], data['Language:Speak Spanish'])
spanishnotwell = [data['language_speakspanish_speakenglishnotwell_5to17'],data['language_speakspanish_speakenglishnotwell_18to64'],
                   data['language_speakspanish_speakenglishnotwell_65+']]
data['Language:Spanish:Speak English Not Well'] = sum(spanishnotwell)
data['Language:Spanish%:Speak English Not Well'] = percent(data['Language:Spanish:Speak English Not Well'], data['Language:Speak Spanish'])
spanishnotatall = [data['language_speakspanish_speakenglishnotatall_5to17'],data['language_speakspanish_speakenglishnotatall_18to64'],
                   data['language_speakspanish_speakenglishnotatall_65+']]
data['Language:Spanish:Speak English Not At All'] = sum(spanishnotatall)
data['Language:Spanish%:Speak English Not At All'] = percent(data['Language:Spanish:Speak English Not At All'], data['Language:Speak Spanish'])
data['Language:Spanish:Speak English Not Well or At All'] = data['Language:Spanish:Speak English Not Well'] + data['Language:Spanish:Speak English Not At All']
data['Language:Spanish%:Speak English Not Well or At All'] = percent(data['Language:Spanish:Speak English Not Well or At All'], data['Language:Speak Spanish'])
#otherindoeuropeanlanguage
speakotherindoeuro = [data['language_speakotherindoeuro_5to17'],data['language_speakotherindoeuro_18to64'],data['language_speakotherindoeuro_65+']]
data['Language:Speak Other Indo-European Language'] = sum(speakotherindoeuro)
data['Language%:Speak Other Indo-European Language'] = percent(data['Language:Speak Other Indo-European Language'], data['Language:Population 5 and Older'])
data['Language%:Non-English Speak Other Indo-European Language'] = percent(data['Language:Speak Other Indo-European Language'], besidesenglish)
otherindoeuroverywell = [data['language_speakotherindoeuro_speakenglishverywell_5to17'],data['language_speakotherindoeuro_speakenglishverywell_18to64'],
                   data['language_speakotherindoeuro_speakenglishverywell_65+']]
data['Language:Other Indo-European:Speak English Very Well'] = sum(otherindoeuroverywell)
data['Language:Other Indo-European%:Speak English Very Well'] = percent(data['Language:Other Indo-European:Speak English Very Well'], 
                                                                        data['Language:Speak Other Indo-European Language'])
otherindoeurowell = [data['language_speakotherindoeuro_speakenglishwell_5to17'],data['language_speakotherindoeuro_speakenglishwell_18to64'],
                   data['language_speakotherindoeuro_speakenglishwell_65+']]
data['Language:Other Indo-European:Speak English Well'] = sum(otherindoeurowell)
data['Language:Other Indo-European%:Speak English Well'] = percent(data['Language:Other Indo-European:Speak English Well'], 
                                                                   data['Language:Speak Other Indo-European Language'])
otherindoeurowellorverywell = [data['Language:Other Indo-European:Speak English Very Well'], data['Language:Other Indo-European:Speak English Well']]
data['Language:Other Indo-European:Speak English Well or Very Well'] = sum(otherindoeurowellorverywell)
data['Language:Other Indo-European%:Speak English Well or Very Well'] = percent(data['Language:Other Indo-European:Speak English Well or Very Well'], 
                                                                                data['Language:Speak Other Indo-European Language'])
otherindoeuronotwell = [data['language_speakotherindoeuro_speakenglishnotwell_5to17'],data['language_speakotherindoeuro_speakenglishnotwell_18to64'],
                   data['language_speakotherindoeuro_speakenglishnotwell_65+']]
data['Language:Other Indo-European:Speak English Not Well'] = sum(otherindoeuronotwell)
data['Language:Other Indo-European%:Speak English Not Well'] = percent(data['Language:Other Indo-European:Speak English Not Well'], 
                                                                       data['Language:Speak Other Indo-European Language'])
otherindoeuronotatall = [data['language_speakotherindoeuro_speakenglishnotatall_5to17'],data['language_speakotherindoeuro_speakenglishnotatall_18to64'],
                   data['language_speakotherindoeuro_speakenglishnotatall_65+']]
data['Language:Other Indo-European:Speak English Not At All'] = sum(otherindoeuronotatall)
data['Language:Other Indo-European%:Speak English Not At All'] = percent(data['Language:Other Indo-European:Speak English Not At All'], 
                                                                         data['Language:Speak Other Indo-European Language'])
otherindoeuronotwelloratall = [data['Language:Other Indo-European:Speak English Not Well'], data['Language:Other Indo-European:Speak English Not At All']]
data['Language:Other Indo-European:Speak English Not Well or At All'] = sum(otherindoeuronotwelloratall)
data['Language:Other Indo-European%:Speak English Not Well or At All'] = percent(data['Language:Other Indo-European:Speak English Not Well or At All'], 
                                                                                data['Language:Speak Other Indo-European Language'])

#asianpacificislandlanguage
speakasianpacificisland = [data['language_speakasianpacificisland_5to17'],data['language_speakasianpacificisland_18to64'],data['language_speakasianpacificisland_65+']]
data['Language:Speak Asian or Pacific Island Language'] = sum(speakasianpacificisland)
data['Language%:Speak Asian or Pacific Island Language'] = percent(data['Language:Speak Asian or Pacific Island Language'], data['Language:Population 5 and Older'])
data['Language%:Non-English Speak Asian or Pacific Island Language'] = percent(data['Language:Speak Asian or Pacific Island Language'], 
                                                                               besidesenglish)
asianpacificislandverywell = [data['language_speakasianpacificisland_speakenglishverywell_5to17'],data['language_speakasianpacificisland_speakenglishverywell_18to64'],
                   data['language_speakasianpacificisland_speakenglishverywell_65+']]
data['Language:Asian or Pacific Island:Speak English Very Well'] = sum(asianpacificislandverywell)
data['Language:Asian or Pacific Island%:Speak English Very Well'] = percent(data['Language:Asian or Pacific Island:Speak English Very Well'], 
                                                                            data['Language:Speak Asian or Pacific Island Language'])
asianpacificislandwell = [data['language_speakasianpacificisland_speakenglishwell_5to17'],data['language_speakasianpacificisland_speakenglishwell_18to64'],
                   data['language_speakasianpacificisland_speakenglishwell_65+']]
data['Language:Asian or Pacific Island:Speak English Well'] = sum(asianpacificislandwell)
data['Language:Asian or Pacific Island%:Speak English Well'] = percent(data['Language:Asian or Pacific Island:Speak English Well'], 
                                                                       data['Language:Speak Asian or Pacific Island Language'])
asianpacificislandwellorverywell = [data['Language:Asian or Pacific Island:Speak English Very Well'], data['Language:Asian or Pacific Island:Speak English Well']]
data['Language:Asian or Pacific Island:Speak English Well or Very Well'] = sum(asianpacificislandwellorverywell)
data['Language:Asian or Pacific Island%:Speak English Well or Very Well'] = percent(data['Language:Asian or Pacific Island:Speak English Well or Very Well'], 
                                                                                    data['Language:Speak Asian or Pacific Island Language'])
asianpacificislandnotwell = [data['language_speakasianpacificisland_speakenglishnotwell_5to17'],data['language_speakasianpacificisland_speakenglishnotwell_18to64'],
                   data['language_speakasianpacificisland_speakenglishnotwell_65+']]
data['Language:Asian or Pacific Island:Speak English Not Well'] = sum(asianpacificislandnotwell)
data['Language:Asian or Pacific Island%:Speak English Not Well'] = percent(data['Language:Asian or Pacific Island:Speak English Not Well'], 
                                                                           data['Language:Speak Asian or Pacific Island Language'])
asianpacificislandnotatall = [data['language_speakasianpacificisland_speakenglishnotatall_5to17'],data['language_speakasianpacificisland_speakenglishnotatall_18to64'],
                   data['language_speakasianpacificisland_speakenglishnotatall_65+']]
data['Language:Asian or Pacific Island:Speak English Not At All'] = sum(asianpacificislandnotatall)
data['Language:Asian or Pacific Island%:Speak English Not At All'] = percent(data['Language:Asian or Pacific Island:Speak English Not At All'], 
                                                                             data['Language:Speak Asian or Pacific Island Language'])
asianpacificislandnotwelloratall = [data['Language:Asian or Pacific Island:Speak English Not Well'], data['Language:Asian or Pacific Island:Speak English Not At All']]
data['Language:Asian or Pacific Island:Speak English Not Well or At All'] = sum(asianpacificislandnotwelloratall)
data['Language:Asian or Pacific Island%:Speak English Not Well or At All'] = percent(data['Language:Asian or Pacific Island:Speak English Not Well or At All'], 
                                                                                     data['Language:Speak Asian or Pacific Island Language'])
#other
speakother = [data['language_speakother_5to17'],data['language_speakother_18to64'],data['language_speakother_65+']]
data['Language:Speak Other Language'] = sum(speakother)
data['Language%:Speak Other Language'] = percent(data['Language:Speak Other Language'], data['Language:Population 5 and Older'])
data['Language%:Non-English Speak Other Language'] = percent(data['Language:Speak Other Language'], besidesenglish)
otherverywell = [data['language_speakother_speakenglishverywell_5to17'],data['language_speakother_speakenglishverywell_18to64'],
                   data['language_speakother_speakenglishverywell_65+']]
data['Language:Other:Speak English Very Well'] = sum(otherverywell)
data['Language:Other%:Speak English Very Well'] = percent(data['Language:Other:Speak English Very Well'], data['Language:Speak Other Language'])
otherwell = [data['language_speakother_speakenglishwell_5to17'],data['language_speakother_speakenglishwell_18to64'],
                   data['language_speakother_speakenglishwell_65+']]
data['Language:Other:Speak English Well'] = sum(otherwell)
data['Language:Other%:Speak English Well'] = percent(data['Language:Other:Speak English Well'], data['Language:Speak Other Language'])
otherwellorverywell = [data['Language:Other:Speak English Very Well'], data['Language:Other:Speak English Well']]
data['Language:Other:Speak English Well or Very Well'] = sum(otherwellorverywell)
data['Language:Other%:Speak English Well or Very Well'] = percent(data['Language:Other:Speak English Well or Very Well'], data['Language:Speak Other Language'])
othernotwell = [data['language_speakother_speakenglishnotwell_5to17'],data['language_speakother_speakenglishnotwell_18to64'],
                   data['language_speakother_speakenglishnotwell_65+']]
data['Language:Other:Speak English Not Well'] = sum(othernotwell)
data['Language:Other%:Speak English Not Well'] = percent(data['Language:Other:Speak English Not Well'], data['Language:Speak Other Language'])
othernotatall = [data['language_speakother_speakenglishnotatall_5to17'],data['language_speakother_speakenglishnotatall_18to64'],
                   data['language_speakother_speakenglishnotatall_65+']]
data['Language:Other:Speak English Not At All'] = sum(othernotatall)
data['Language:Other%:Speak English Not At All'] = percent(data['Language:Other:Speak English Not At All'], data['Language:Speak Other Language'])
othernotwelloratall = [data['Language:Other:Speak English Not Well'], data['Language:Other:Speak English Not At All']]
data['Language:Other:Speak English Not Well or At All'] = sum(othernotwelloratall)
data['Language:Other%:Speak English Not Well or At All'] = percent(data['Language:Other:Speak English Not Well or At All'], data['Language:Speak Other Language'])
#All non-English languages
speaknoneng = [data['Language:Speak Spanish'], data['Language:Speak Other Indo-European Language'], 
               data['Language:Speak Asian or Pacific Island Language'], data['Language:Speak Other Language']]
data['Language:Speak Language Besides English'] = sum(speaknoneng)
data['Language%:Speak Language Besides English'] = percent(data['Language:Speak Language Besides English'], data['Language:Population 5 and Older'])
nonengverywell = [data['Language:Spanish:Speak English Very Well'], data['Language:Other Indo-European:Speak English Very Well'],
                  data['Language:Asian or Pacific Island:Speak English Very Well'], data['Language:Other:Speak English Very Well']]
data['Language:Language Besides English:Speak English Very Well'] = sum(nonengverywell)
data['Language%:Language Besides English:Speak English Very Well'] = percent(data['Language:Language Besides English:Speak English Very Well'],
                                                                             data['Language:Speak Language Besides English'])
nonengwell = [data['Language:Spanish:Speak English Well'], data['Language:Other Indo-European:Speak English Well'],
              data['Language:Asian or Pacific Island:Speak English Well'], data['Language:Other:Speak English Well']]
data['Language:Language Besides English:Speak English Well'] = sum(nonengwell)
data['Language%:Language Besides English:Speak English Well'] = percent(data['Language:Language Besides English:Speak English Well'],
                                                                        data['Language:Speak Language Besides English'])
noengwellorverywell = [data['Language:Language Besides English:Speak English Very Well'], data['Language:Language Besides English:Speak English Well']]
data['Language:Language Besides English:Speak English Well or Very Well'] = sum(noengwellorverywell)
data['Language:Language Besides English%:Speak English Well or Very Well'] = percent(data['Language:Language Besides English:Speak English Well or Very Well'], 
                                                                                     data['Language:Speak Language Besides English'])
nonengnotwell = [data['Language:Spanish:Speak English Not Well'], data['Language:Other Indo-European:Speak English Not Well'],
                 data['Language:Asian or Pacific Island:Speak English Not Well'], data['Language:Other:Speak English Not Well']]
data['Language:Language Besides English:Speak English Not Well'] = sum(nonengnotwell)
data['Language%:Language Besides English:Speak English Not Well'] = percent(data['Language:Language Besides English:Speak English Not Well'],
                                                                            data['Language:Speak Language Besides English'])
nonengnotatall = [data['Language:Spanish:Speak English Not At All'], data['Language:Other Indo-European:Speak English Not At All'],
                 data['Language:Asian or Pacific Island:Speak English Not At All'], data['Language:Other:Speak English Not At All']]
data['Language:Language Besides English:Speak English Not At All'] = sum(nonengnotatall)
data['Language%:Language Besides English:Speak English Not At All'] = percent(data['Language:Language Besides English:Speak English Not At All'],
                                                                            data['Language:Speak Language Besides English'])
noengnotwelloratall = [data['Language:Language Besides English:Speak English Not Well'], data['Language:Language Besides English:Speak English Not At All']]
data['Language:Language Besides English:Speak English Not Well or At All'] = sum(noengnotwelloratall)
data['Language:Language Besides English%:Speak English Not Well or At All'] = percent(data['Language:Language Besides English:Speak English Not Well or At All'], 
                                                                                     data['Language:Speak Language Besides English'])
#earnings
#less than $10,000
lessthan10000 = [data['earn_m_1to2499orloss'], data['earn_m_2500to4999'], data['earn_m_5000to7499'], data['earn_m_7500to9999'], 
                 data['earn_f_1to2499orloss'], data['earn_f_2500to4999'], data['earn_f_5000to7499'], data['earn_f_7500to9999']]              
data['Earnings:Less than 10,000'] = sum(lessthan10000)
data['Earnings:% Less than 10,000'] = percent(data['Earnings:Less than 10,000'], data['earn_total_series'])
#less than $25,000
lessthan25 = [data['earn_m_1to2499orloss'], data['earn_m_2500to4999'], data['earn_m_5000to7499'], data['earn_m_7500to9999'], data['earn_m_10000to12499'],
              data['earn_m_12500to14999'], data['earn_m_15000to17499'], data['earn_m_17500to19999'], data['earn_m_20000to22499'], 
              data['earn_f_1to2499orloss'], data['earn_f_2500to4999'], data['earn_f_5000to7499'], data['earn_f_7500to9999'], data['earn_f_10000to12499'],
              data['earn_f_12500to14999'], data['earn_f_15000to17499'], data['earn_f_17500to19999'], data['earn_f_20000to22499']]
data['Earnings:Less than 25,000'] = sum(lessthan25)
data['Earnings:% Less than 25,000'] = percent(data['Earnings:Less than 25,000'], data['earn_total_series'])
#$25,000 to $49,999
twenty5000to49999 = [data['earn_m_22500to24999'], data['earn_m_25000to29999'], data['earn_m_30000to34999'], data['earn_m_35000to39999'], data['earn_m_40000to44999'],
                     data['earn_m_45000to49999'], data['earn_f_22500to24999'], data['earn_f_25000to29999'], data['earn_f_30000to34999'], data['earn_f_35000to39999'],
                     data['earn_f_40000to44999'], data['earn_f_45000to49999']]
data['Earnings:25,000 to 49,999'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999'], data['earn_total_series'])
#$50,000 to $74,000
fifty000to74999 = [data['earn_m_50000to54999'], data['earn_m_55000to64999'], data['earn_m_65000to74999'],
                   data['earn_f_50000to54999'], data['earn_f_55000to64999'], data['earn_f_65000to74999']]
data['Earnings:50,000 to 74,999'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999'], data['earn_total_series'])
#$75,000 to $99,999
seventy5000to99999 = [data['earn_m_75000to99999'], data['earn_f_75000to99999']]
data['Earnings:75,000 to 99,999'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999'], data['earn_total_series'])
#$100,000+
onehundred000up = [data['earn_m_100000ormore'], data['earn_f_100000ormore']]
data['Earnings:100,000 and Over'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over'] = percent(data['Earnings:100,000 and Over'], data['earn_total_series'])
#earnings by race

#white alone
#less than $10,000 White Alone
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_whitealone'], data['earn_m_fulltime_2500to4999_whitealone'], data['earn_m_fulltime_5000to7499_whitealone'],
                 data['earn_m_fulltime_7500to9999_whitealone'], data['earn_f_fulltime_1to2499orloss_whitealone'], data['earn_f_fulltime_2500to4999_whitealone'], 
                 data['earn_f_fulltime_5000to7499_whitealone'], data['earn_f_fulltime_7500to9999_whitealone'], 
                 data['earn_m_other_1to2499orloss_whitealone'], data['earn_m_other_2500to4999_whitealone'], data['earn_m_other_5000to7499_whitealone'],
                 data['earn_m_other_7500to9999_whitealone'], data['earn_f_other_1to2499orloss_whitealone'], data['earn_f_other_2500to4999_whitealone'], 
                 data['earn_f_other_5000to7499_whitealone'], data['earn_f_other_7500to9999_whitealone']]              
data['Earnings:Less than 10,000 White Alone'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 White Alone'] = percent(data['Earnings:Less than 10,000 White Alone'], data['earn_whitealone_total_series'])
#less than $25,000 White Alone
lessthan25 = [data['earn_m_fulltime_1to2499orloss_whitealone'], data['earn_m_fulltime_2500to4999_whitealone'], data['earn_m_fulltime_5000to7499_whitealone'],
              data['earn_m_fulltime_7500to9999_whitealone'], data['earn_m_fulltime_10000to12499_whitealone'],data['earn_m_fulltime_12500to14999_whitealone'], 
              data['earn_m_fulltime_15000to17499_whitealone'], data['earn_m_fulltime_17500to19999_whitealone'], data['earn_m_fulltime_20000to22499_whitealone'], 
              data['earn_m_fulltime_22500to24999_whitealone'], data['earn_f_fulltime_1to2499orloss_whitealone'], data['earn_f_fulltime_2500to4999_whitealone'], 
              data['earn_f_fulltime_5000to7499_whitealone'], data['earn_f_fulltime_7500to9999_whitealone'], data['earn_f_fulltime_10000to12499_whitealone'],
              data['earn_f_fulltime_12500to14999_whitealone'], data['earn_f_fulltime_15000to17499_whitealone'], data['earn_f_fulltime_17500to19999_whitealone'], 
              data['earn_f_fulltime_20000to22499_whitealone'], data['earn_f_fulltime_22500to24999_whitealone'], data['earn_m_other_1to2499orloss_whitealone'], 
              data['earn_m_other_2500to4999_whitealone'], data['earn_m_other_5000to7499_whitealone'], data['earn_m_other_7500to9999_whitealone'], 
              data['earn_m_other_10000to12499_whitealone'],data['earn_m_other_12500to14999_whitealone'], data['earn_m_other_15000to17499_whitealone'], 
              data['earn_m_other_17500to19999_whitealone'], data['earn_m_other_20000to22499_whitealone'], data['earn_m_other_22500to24999_whitealone'], 
              data['earn_f_other_1to2499orloss_whitealone'], data['earn_f_other_2500to4999_whitealone'], data['earn_f_other_5000to7499_whitealone'], 
              data['earn_f_other_7500to9999_whitealone'], data['earn_f_other_10000to12499_whitealone'], data['earn_f_other_12500to14999_whitealone'], 
              data['earn_f_other_15000to17499_whitealone'], data['earn_f_other_17500to19999_whitealone'], data['earn_f_other_20000to22499_whitealone'], 
              data['earn_f_other_22500to24999_whitealone']]
data['Earnings:Less than 25,000 White Alone'] = sum(lessthan25)
data['Earnings:% Less than 25,000 White Alone'] = percent(data['Earnings:Less than 25,000 White Alone'], data['earn_whitealone_total_series'])
#$25,000 to $49,999 White Alone
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_whitealone'], data['earn_m_fulltime_30000to34999_whitealone'], 
                     data['earn_m_fulltime_35000to39999_whitealone'], data['earn_m_fulltime_40000to44999_whitealone'],
                     data['earn_m_fulltime_45000to49999_whitealone'], data['earn_f_fulltime_25000to29999_whitealone'], 
                     data['earn_f_fulltime_30000to34999_whitealone'], data['earn_f_fulltime_35000to39999_whitealone'],
                     data['earn_f_fulltime_40000to44999_whitealone'], data['earn_f_fulltime_45000to49999_whitealone'], 
                     data['earn_m_other_25000to29999_whitealone'], data['earn_m_other_30000to34999_whitealone'], 
                     data['earn_m_other_35000to39999_whitealone'], data['earn_m_other_40000to44999_whitealone'],
                     data['earn_m_other_45000to49999_whitealone'], data['earn_f_other_25000to29999_whitealone'], 
                     data['earn_f_other_30000to34999_whitealone'], data['earn_f_other_35000to39999_whitealone'],
                     data['earn_f_other_40000to44999_whitealone'], data['earn_f_other_45000to49999_whitealone']]
data['Earnings:25,000 to 49,999 White Alone'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 White Alone'] = percent(data['Earnings:25,000 to 49,999 White Alone'], data['earn_whitealone_total_series'])
#$50,000 to $74,000 White Alone
fifty000to74999 = [data['earn_m_fulltime_50000to54999_whitealone'], data['earn_m_fulltime_55000to64999_whitealone'], data['earn_m_fulltime_65000to74999_whitealone'],
                   data['earn_f_fulltime_50000to54999_whitealone'], data['earn_f_fulltime_55000to64999_whitealone'], data['earn_f_fulltime_65000to74999_whitealone'], 
                   data['earn_m_other_50000to54999_whitealone'], data['earn_m_other_55000to64999_whitealone'], data['earn_m_other_65000to74999_whitealone'],
                   data['earn_f_other_50000to54999_whitealone'], data['earn_f_other_55000to64999_whitealone'], data['earn_f_other_65000to74999_whitealone']]
data['Earnings:50,000 to 74,999 White Alone'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 White Alone'] = percent(data['Earnings:50,000 to 74,999 White Alone'], data['earn_whitealone_total_series'])
#$75,000 to $99,999 White Alone
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_whitealone'], data['earn_f_fulltime_75000to99999_whitealone'], 
                      data['earn_m_other_75000to99999_whitealone'], data['earn_f_other_75000to99999_whitealone']]
data['Earnings:75,000 to 99,999 White Alone'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 White Alone'] = percent(data['Earnings:75,000 to 99,999 White Alone'], data['earn_whitealone_total_series'])
#$100,000+ White Alone
onehundred000up = [data['earn_m_fulltime_100000ormore_whitealone'], data['earn_f_fulltime_100000ormore_whitealone'], 
                   data['earn_m_other_100000ormore_whitealone'], data['earn_f_other_100000ormore_whitealone']]
data['Earnings:100,000 and Over White Alone'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over White Alone'] = percent(data['Earnings:100,000 and Over White Alone'], data['earn_whitealone_total_series'])
#Black or African American Alone
#less than $10,000 Black or African American Alone
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_blackafricanamericanalone'], data['earn_m_fulltime_2500to4999_blackafricanamericanalone'], 
                 data['earn_m_fulltime_5000to7499_blackafricanamericanalone'], data['earn_m_fulltime_7500to9999_blackafricanamericanalone'], 
                 data['earn_f_fulltime_1to2499orloss_blackafricanamericanalone'], data['earn_f_fulltime_2500to4999_blackafricanamericanalone'], 
                 data['earn_f_fulltime_5000to7499_blackafricanamericanalone'], data['earn_f_fulltime_7500to9999_blackafricanamericanalone'], 
                 data['earn_m_other_1to2499orloss_blackafricanamericanalone'], data['earn_m_other_2500to4999_blackafricanamericanalone'], 
                 data['earn_m_other_5000to7499_blackafricanamericanalone'], data['earn_m_other_7500to9999_blackafricanamericanalone'], 
                 data['earn_f_other_1to2499orloss_blackafricanamericanalone'], data['earn_f_other_2500to4999_blackafricanamericanalone'], 
                 data['earn_f_other_5000to7499_blackafricanamericanalone'], data['earn_f_other_7500to9999_blackafricanamericanalone']]              
data['Earnings:Less than 10,000 Black or African American Alone'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 Black or African American Alone'] = percent(data['Earnings:Less than 10,000 Black or African American Alone'], 
                                                                              data['earn_blackafricanamericanalone_total_series'])
#less than $25,000 Black or African American Alone
lessthan25 = [data['earn_m_fulltime_1to2499orloss_blackafricanamericanalone'], data['earn_m_fulltime_2500to4999_blackafricanamericanalone'], 
              data['earn_m_fulltime_5000to7499_blackafricanamericanalone'],data['earn_m_fulltime_7500to9999_blackafricanamericanalone'], 
              data['earn_m_fulltime_10000to12499_blackafricanamericanalone'],data['earn_m_fulltime_12500to14999_blackafricanamericanalone'], 
              data['earn_m_fulltime_15000to17499_blackafricanamericanalone'], data['earn_m_fulltime_17500to19999_blackafricanamericanalone'],
              data['earn_m_fulltime_20000to22499_blackafricanamericanalone'], data['earn_m_fulltime_22500to24999_blackafricanamericanalone'], 
              data['earn_f_fulltime_1to2499orloss_blackafricanamericanalone'], data['earn_f_fulltime_2500to4999_blackafricanamericanalone'], 
              data['earn_f_fulltime_5000to7499_blackafricanamericanalone'], data['earn_f_fulltime_7500to9999_blackafricanamericanalone'], 
              data['earn_f_fulltime_10000to12499_blackafricanamericanalone'],data['earn_f_fulltime_12500to14999_blackafricanamericanalone'], 
              data['earn_f_fulltime_15000to17499_blackafricanamericanalone'], data['earn_f_fulltime_17500to19999_blackafricanamericanalone'], 
              data['earn_f_fulltime_20000to22499_blackafricanamericanalone'], data['earn_f_fulltime_22500to24999_blackafricanamericanalone'], 
              data['earn_m_other_1to2499orloss_blackafricanamericanalone'], data['earn_m_other_2500to4999_blackafricanamericanalone'], 
              data['earn_m_other_5000to7499_blackafricanamericanalone'], data['earn_m_other_7500to9999_blackafricanamericanalone'], 
              data['earn_m_other_10000to12499_blackafricanamericanalone'],data['earn_m_other_12500to14999_blackafricanamericanalone'], 
              data['earn_m_other_15000to17499_blackafricanamericanalone'], data['earn_m_other_17500to19999_blackafricanamericanalone'], 
              data['earn_m_other_20000to22499_blackafricanamericanalone'], data['earn_m_other_22500to24999_blackafricanamericanalone'], 
              data['earn_f_other_1to2499orloss_blackafricanamericanalone'], data['earn_f_other_2500to4999_blackafricanamericanalone'], 
              data['earn_f_other_5000to7499_blackafricanamericanalone'], data['earn_f_other_7500to9999_blackafricanamericanalone'], 
              data['earn_f_other_10000to12499_blackafricanamericanalone'], data['earn_f_other_12500to14999_blackafricanamericanalone'], 
              data['earn_f_other_15000to17499_blackafricanamericanalone'], data['earn_f_other_17500to19999_blackafricanamericanalone'], 
              data['earn_f_other_20000to22499_blackafricanamericanalone'], data['earn_f_other_22500to24999_blackafricanamericanalone']]
data['Earnings:Less than 25,000 Black or African American Alone'] = sum(lessthan25)
data['Earnings:% Less than 25,000 Black or African American Alone'] = percent(data['Earnings:Less than 25,000 Black or African American Alone'], 
                                                                              data['earn_blackafricanamericanalone_total_series'])
#$25,000 to $49,999 Black or African American Alone
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_blackafricanamericanalone'], data['earn_m_fulltime_30000to34999_blackafricanamericanalone'], 
                     data['earn_m_fulltime_35000to39999_blackafricanamericanalone'], data['earn_m_fulltime_40000to44999_blackafricanamericanalone'],
                     data['earn_m_fulltime_45000to49999_blackafricanamericanalone'], data['earn_f_fulltime_25000to29999_blackafricanamericanalone'], 
                     data['earn_f_fulltime_30000to34999_blackafricanamericanalone'], data['earn_f_fulltime_35000to39999_blackafricanamericanalone'],
                     data['earn_f_fulltime_40000to44999_blackafricanamericanalone'], data['earn_f_fulltime_45000to49999_blackafricanamericanalone'], 
                     data['earn_m_other_25000to29999_blackafricanamericanalone'], data['earn_m_other_30000to34999_blackafricanamericanalone'], 
                     data['earn_m_other_35000to39999_blackafricanamericanalone'], data['earn_m_other_40000to44999_blackafricanamericanalone'],
                     data['earn_m_other_45000to49999_blackafricanamericanalone'], data['earn_f_other_25000to29999_blackafricanamericanalone'], 
                     data['earn_f_other_30000to34999_blackafricanamericanalone'], data['earn_f_other_35000to39999_blackafricanamericanalone'],
                     data['earn_f_other_40000to44999_blackafricanamericanalone'], data['earn_f_other_45000to49999_blackafricanamericanalone']]
data['Earnings:25,000 to 49,999 Black or African American Alone'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 Black or African American Alone'] = percent(data['Earnings:25,000 to 49,999 Black or African American Alone'], 
                                                                              data['earn_blackafricanamericanalone_total_series'])
#$50,000 to $74,000 Black or African American Alone
fifty000to74999 = [data['earn_m_fulltime_50000to54999_blackafricanamericanalone'], data['earn_m_fulltime_55000to64999_blackafricanamericanalone'], 
                   data['earn_m_fulltime_65000to74999_blackafricanamericanalone'], data['earn_f_fulltime_50000to54999_blackafricanamericanalone'], 
                   data['earn_f_fulltime_55000to64999_blackafricanamericanalone'], data['earn_f_fulltime_65000to74999_blackafricanamericanalone'], 
                   data['earn_m_other_50000to54999_blackafricanamericanalone'], data['earn_m_other_55000to64999_blackafricanamericanalone'], 
                   data['earn_m_other_65000to74999_blackafricanamericanalone'], data['earn_f_other_50000to54999_blackafricanamericanalone'], 
                   data['earn_f_other_55000to64999_blackafricanamericanalone'], data['earn_f_other_65000to74999_blackafricanamericanalone']]
data['Earnings:50,000 to 74,999 Black or African American Alone'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 Black or African American Alone'] = percent(data['Earnings:50,000 to 74,999 Black or African American Alone'], 
                                                                              data['earn_blackafricanamericanalone_total_series'])
#$75,000 to $99,999 Black or African American Alone
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_blackafricanamericanalone'], data['earn_f_fulltime_75000to99999_blackafricanamericanalone'], 
                      data['earn_m_other_75000to99999_blackafricanamericanalone'], data['earn_f_other_75000to99999_blackafricanamericanalone']]
data['Earnings:75,000 to 99,999 Black or African American Alone'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 Black or African American Alone'] = percent(data['Earnings:75,000 to 99,999 Black or African American Alone'], 
                                                                              data['earn_blackafricanamericanalone_total_series'])
#$100,000+ Black or African American Alone
onehundred000up = [data['earn_m_fulltime_100000ormore_blackafricanamericanalone'], data['earn_f_fulltime_100000ormore_blackafricanamericanalone'], 
                   data['earn_m_other_100000ormore_blackafricanamericanalone'], data['earn_f_other_100000ormore_blackafricanamericanalone']]
data['Earnings:100,000 and Over Black or African American Alone'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over Black or African American Alone'] = percent(data['Earnings:100,000 and Over Black or African American Alone'], 
                                                                              data['earn_blackafricanamericanalone_total_series'])
#American Indian Alaska Native Alone
#less than $10,000 American Indian or Alaska Native Alone
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_americanindianalaskanativealone'], data['earn_m_fulltime_2500to4999_americanindianalaskanativealone'], 
                 data['earn_m_fulltime_5000to7499_americanindianalaskanativealone'], data['earn_m_fulltime_7500to9999_americanindianalaskanativealone'], 
                 data['earn_f_fulltime_1to2499orloss_americanindianalaskanativealone'], data['earn_f_fulltime_2500to4999_americanindianalaskanativealone'], 
                 data['earn_f_fulltime_5000to7499_americanindianalaskanativealone'], data['earn_f_fulltime_7500to9999_americanindianalaskanativealone'], 
                 data['earn_m_other_1to2499orloss_americanindianalaskanativealone'], data['earn_m_other_2500to4999_americanindianalaskanativealone'], 
                 data['earn_m_other_5000to7499_americanindianalaskanativealone'], data['earn_m_other_7500to9999_americanindianalaskanativealone'], 
                 data['earn_f_other_1to2499orloss_americanindianalaskanativealone'], data['earn_f_other_2500to4999_americanindianalaskanativealone'], 
                 data['earn_f_other_5000to7499_americanindianalaskanativealone'], data['earn_f_other_7500to9999_americanindianalaskanativealone']]              
data['Earnings:Less than 10,000 American Indian or Alaska Native Alone'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 American Indian or Alaska Native Alone'] = percent(data['Earnings:Less than 10,000 American Indian or Alaska Native Alone'], 
                                                                              data['earn_americanindianalaskanativealone_total_series'])
#less than $25,000 American Indian or Alaska Native Alone
lessthan25 = [data['earn_m_fulltime_1to2499orloss_americanindianalaskanativealone'], data['earn_m_fulltime_2500to4999_americanindianalaskanativealone'], 
              data['earn_m_fulltime_5000to7499_americanindianalaskanativealone'],data['earn_m_fulltime_7500to9999_americanindianalaskanativealone'], 
              data['earn_m_fulltime_10000to12499_americanindianalaskanativealone'],data['earn_m_fulltime_12500to14999_americanindianalaskanativealone'], 
              data['earn_m_fulltime_15000to17499_americanindianalaskanativealone'], data['earn_m_fulltime_17500to19999_americanindianalaskanativealone'],
              data['earn_m_fulltime_20000to22499_americanindianalaskanativealone'], data['earn_m_fulltime_22500to24999_americanindianalaskanativealone'], 
              data['earn_f_fulltime_1to2499orloss_americanindianalaskanativealone'], data['earn_f_fulltime_2500to4999_americanindianalaskanativealone'], 
              data['earn_f_fulltime_5000to7499_americanindianalaskanativealone'], data['earn_f_fulltime_7500to9999_americanindianalaskanativealone'], 
              data['earn_f_fulltime_10000to12499_americanindianalaskanativealone'],data['earn_f_fulltime_12500to14999_americanindianalaskanativealone'], 
              data['earn_f_fulltime_15000to17499_americanindianalaskanativealone'], data['earn_f_fulltime_17500to19999_americanindianalaskanativealone'], 
              data['earn_f_fulltime_20000to22499_americanindianalaskanativealone'], data['earn_f_fulltime_22500to24999_americanindianalaskanativealone'], 
              data['earn_m_other_1to2499orloss_americanindianalaskanativealone'], data['earn_m_other_2500to4999_americanindianalaskanativealone'], 
              data['earn_m_other_5000to7499_americanindianalaskanativealone'], data['earn_m_other_7500to9999_americanindianalaskanativealone'], 
              data['earn_m_other_10000to12499_americanindianalaskanativealone'],data['earn_m_other_12500to14999_americanindianalaskanativealone'], 
              data['earn_m_other_15000to17499_americanindianalaskanativealone'], data['earn_m_other_17500to19999_americanindianalaskanativealone'], 
              data['earn_m_other_20000to22499_americanindianalaskanativealone'], data['earn_m_other_22500to24999_americanindianalaskanativealone'], 
              data['earn_f_other_1to2499orloss_americanindianalaskanativealone'], data['earn_f_other_2500to4999_americanindianalaskanativealone'], 
              data['earn_f_other_5000to7499_americanindianalaskanativealone'], data['earn_f_other_7500to9999_americanindianalaskanativealone'], 
              data['earn_f_other_10000to12499_americanindianalaskanativealone'], data['earn_f_other_12500to14999_americanindianalaskanativealone'], 
              data['earn_f_other_15000to17499_americanindianalaskanativealone'], data['earn_f_other_17500to19999_americanindianalaskanativealone'], 
              data['earn_f_other_20000to22499_americanindianalaskanativealone'], data['earn_f_other_22500to24999_americanindianalaskanativealone']]
data['Earnings:Less than 25,000 American Indian or Alaska Native Alone'] = sum(lessthan25)
data['Earnings:% Less than 25,000 American Indian or Alaska Native Alone'] = percent(data['Earnings:Less than 25,000 American Indian or Alaska Native Alone'], 
                                                                              data['earn_americanindianalaskanativealone_total_series'])
#$25,000 to $49,999 American Indian or Alaska Native Alone
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_americanindianalaskanativealone'], data['earn_m_fulltime_30000to34999_americanindianalaskanativealone'], 
                     data['earn_m_fulltime_35000to39999_americanindianalaskanativealone'], data['earn_m_fulltime_40000to44999_americanindianalaskanativealone'],
                     data['earn_m_fulltime_45000to49999_americanindianalaskanativealone'], data['earn_f_fulltime_25000to29999_americanindianalaskanativealone'], 
                     data['earn_f_fulltime_30000to34999_americanindianalaskanativealone'], data['earn_f_fulltime_35000to39999_americanindianalaskanativealone'],
                     data['earn_f_fulltime_40000to44999_americanindianalaskanativealone'], data['earn_f_fulltime_45000to49999_americanindianalaskanativealone'], 
                     data['earn_m_other_25000to29999_americanindianalaskanativealone'], data['earn_m_other_30000to34999_americanindianalaskanativealone'], 
                     data['earn_m_other_35000to39999_americanindianalaskanativealone'], data['earn_m_other_40000to44999_americanindianalaskanativealone'],
                     data['earn_m_other_45000to49999_americanindianalaskanativealone'], data['earn_f_other_25000to29999_americanindianalaskanativealone'], 
                     data['earn_f_other_30000to34999_americanindianalaskanativealone'], data['earn_f_other_35000to39999_americanindianalaskanativealone'],
                     data['earn_f_other_40000to44999_americanindianalaskanativealone'], data['earn_f_other_45000to49999_americanindianalaskanativealone']]
data['Earnings:25,000 to 49,999 American Indian or Alaska Native Alone'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 American Indian or Alaska Native Alone'] = percent(data['Earnings:25,000 to 49,999 American Indian or Alaska Native Alone'], 
                                                                              data['earn_americanindianalaskanativealone_total_series'])
#$50,000 to $74,000 American Indian or Alaska Native Alone
fifty000to74999 = [data['earn_m_fulltime_50000to54999_americanindianalaskanativealone'], data['earn_m_fulltime_55000to64999_americanindianalaskanativealone'], 
                   data['earn_m_fulltime_65000to74999_americanindianalaskanativealone'], data['earn_f_fulltime_50000to54999_americanindianalaskanativealone'], 
                   data['earn_f_fulltime_55000to64999_americanindianalaskanativealone'], data['earn_f_fulltime_65000to74999_americanindianalaskanativealone'], 
                   data['earn_m_other_50000to54999_americanindianalaskanativealone'], data['earn_m_other_55000to64999_americanindianalaskanativealone'], 
                   data['earn_m_other_65000to74999_americanindianalaskanativealone'], data['earn_f_other_50000to54999_americanindianalaskanativealone'], 
                   data['earn_f_other_55000to64999_americanindianalaskanativealone'], data['earn_f_other_65000to74999_americanindianalaskanativealone']]
data['Earnings:50,000 to 74,999 American Indian or Alaska Native Alone'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 American Indian or Alaska Native Alone'] = percent(data['Earnings:50,000 to 74,999 American Indian or Alaska Native Alone'], 
                                                                              data['earn_americanindianalaskanativealone_total_series'])
#$75,000 to $99,999 American Indian or Alaska Native Alone
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_americanindianalaskanativealone'], data['earn_f_fulltime_75000to99999_americanindianalaskanativealone'], 
                      data['earn_m_other_75000to99999_americanindianalaskanativealone'], data['earn_f_other_75000to99999_americanindianalaskanativealone']]
data['Earnings:75,000 to 99,999 American Indian or Alaska Native Alone'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 American Indian or Alaska Native Alone'] = percent(data['Earnings:75,000 to 99,999 American Indian or Alaska Native Alone'], 
                                                                              data['earn_americanindianalaskanativealone_total_series'])
#$100,000+ American Indian or Alaska Native Alone
onehundred000up = [data['earn_m_fulltime_100000ormore_americanindianalaskanativealone'], data['earn_f_fulltime_100000ormore_americanindianalaskanativealone'], 
                   data['earn_m_other_100000ormore_americanindianalaskanativealone'], data['earn_f_other_100000ormore_americanindianalaskanativealone']]
data['Earnings:100,000 and Over American Indian or Alaska Native Alone'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over American Indian or Alaska Native Alone'] = percent(data['Earnings:100,000 and Over American Indian or Alaska Native Alone'], 
                                                                              data['earn_americanindianalaskanativealone_total_series'])
#Asian Alone
#less than $10,000 Asian Alone
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_asianalone'], data['earn_m_fulltime_2500to4999_asianalone'], 
                 data['earn_m_fulltime_5000to7499_asianalone'], data['earn_m_fulltime_7500to9999_asianalone'], 
                 data['earn_f_fulltime_1to2499orloss_asianalone'], data['earn_f_fulltime_2500to4999_asianalone'], 
                 data['earn_f_fulltime_5000to7499_asianalone'], data['earn_f_fulltime_7500to9999_asianalone'], 
                 data['earn_m_other_1to2499orloss_asianalone'], data['earn_m_other_2500to4999_asianalone'], 
                 data['earn_m_other_5000to7499_asianalone'], data['earn_m_other_7500to9999_asianalone'], 
                 data['earn_f_other_1to2499orloss_asianalone'], data['earn_f_other_2500to4999_asianalone'], 
                 data['earn_f_other_5000to7499_asianalone'], data['earn_f_other_7500to9999_asianalone']]              
data['Earnings:Less than 10,000 Asian Alone'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 Asian Alone'] = percent(data['Earnings:Less than 10,000 Asian Alone'], data['earn_asianalone_total_series'])
#less than $25,000 Asian Alone
lessthan25 = [data['earn_m_fulltime_1to2499orloss_asianalone'], data['earn_m_fulltime_2500to4999_asianalone'], 
              data['earn_m_fulltime_5000to7499_asianalone'],data['earn_m_fulltime_7500to9999_asianalone'], 
              data['earn_m_fulltime_10000to12499_asianalone'],data['earn_m_fulltime_12500to14999_asianalone'], 
              data['earn_m_fulltime_15000to17499_asianalone'], data['earn_m_fulltime_17500to19999_asianalone'],
              data['earn_m_fulltime_20000to22499_asianalone'], data['earn_m_fulltime_22500to24999_asianalone'], 
              data['earn_f_fulltime_1to2499orloss_asianalone'], data['earn_f_fulltime_2500to4999_asianalone'], 
              data['earn_f_fulltime_5000to7499_asianalone'], data['earn_f_fulltime_7500to9999_asianalone'], 
              data['earn_f_fulltime_10000to12499_asianalone'],data['earn_f_fulltime_12500to14999_asianalone'], 
              data['earn_f_fulltime_15000to17499_asianalone'], data['earn_f_fulltime_17500to19999_asianalone'], 
              data['earn_f_fulltime_20000to22499_asianalone'], data['earn_f_fulltime_22500to24999_asianalone'], 
              data['earn_m_other_1to2499orloss_asianalone'], data['earn_m_other_2500to4999_asianalone'], 
              data['earn_m_other_5000to7499_asianalone'], data['earn_m_other_7500to9999_asianalone'], 
              data['earn_m_other_10000to12499_asianalone'],data['earn_m_other_12500to14999_asianalone'], 
              data['earn_m_other_15000to17499_asianalone'], data['earn_m_other_17500to19999_asianalone'], 
              data['earn_m_other_20000to22499_asianalone'], data['earn_m_other_22500to24999_asianalone'], 
              data['earn_f_other_1to2499orloss_asianalone'], data['earn_f_other_2500to4999_asianalone'], 
              data['earn_f_other_5000to7499_asianalone'], data['earn_f_other_7500to9999_asianalone'], 
              data['earn_f_other_10000to12499_asianalone'], data['earn_f_other_12500to14999_asianalone'], 
              data['earn_f_other_15000to17499_asianalone'], data['earn_f_other_17500to19999_asianalone'], 
              data['earn_f_other_20000to22499_asianalone'], data['earn_f_other_22500to24999_asianalone']]
data['Earnings:Less than 25,000 Asian Alone'] = sum(lessthan25)
data['Earnings:% Less than 25,000 Asian Alone'] = percent(data['Earnings:Less than 25,000 Asian Alone'], data['earn_asianalone_total_series'])
#$25,000 to $49,999 Asian Alone
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_asianalone'], data['earn_m_fulltime_30000to34999_asianalone'], 
                     data['earn_m_fulltime_35000to39999_asianalone'], data['earn_m_fulltime_40000to44999_asianalone'],
                     data['earn_m_fulltime_45000to49999_asianalone'], data['earn_f_fulltime_25000to29999_asianalone'], 
                     data['earn_f_fulltime_30000to34999_asianalone'], data['earn_f_fulltime_35000to39999_asianalone'],
                     data['earn_f_fulltime_40000to44999_asianalone'], data['earn_f_fulltime_45000to49999_asianalone'], 
                     data['earn_m_other_25000to29999_asianalone'], data['earn_m_other_30000to34999_asianalone'], 
                     data['earn_m_other_35000to39999_asianalone'], data['earn_m_other_40000to44999_asianalone'],
                     data['earn_m_other_45000to49999_asianalone'], data['earn_f_other_25000to29999_asianalone'], 
                     data['earn_f_other_30000to34999_asianalone'], data['earn_f_other_35000to39999_asianalone'],
                     data['earn_f_other_40000to44999_asianalone'], data['earn_f_other_45000to49999_asianalone']]
data['Earnings:25,000 to 49,999 Asian Alone'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 Asian Alone'] = percent(data['Earnings:25,000 to 49,999 Asian Alone'], data['earn_asianalone_total_series'])
#Asian Alone
#$50,000 to $74,000 Asian Alone
fifty000to74999 = [data['earn_m_fulltime_50000to54999_asianalone'], data['earn_m_fulltime_55000to64999_asianalone'], 
                   data['earn_m_fulltime_65000to74999_asianalone'], data['earn_f_fulltime_50000to54999_asianalone'], 
                   data['earn_f_fulltime_55000to64999_asianalone'], data['earn_f_fulltime_65000to74999_asianalone'], 
                   data['earn_m_other_50000to54999_asianalone'], data['earn_m_other_55000to64999_asianalone'], 
                   data['earn_m_other_65000to74999_asianalone'], data['earn_f_other_50000to54999_asianalone'], 
                   data['earn_f_other_55000to64999_asianalone'], data['earn_f_other_65000to74999_asianalone']]
data['Earnings:50,000 to 74,999 Asian Alone'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 Asian Alone'] = percent(data['Earnings:50,000 to 74,999 Asian Alone'], data['earn_asianalone_total_series'])
#$75,000 to $99,999 Asian Alone
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_asianalone'], data['earn_f_fulltime_75000to99999_asianalone'], 
                      data['earn_m_other_75000to99999_asianalone'], data['earn_f_other_75000to99999_asianalone']]
data['Earnings:75,000 to 99,999 Asian Alone'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 Asian Alone'] = percent(data['Earnings:75,000 to 99,999 Asian Alone'], 
                                                                              data['earn_asianalone_total_series'])
#$100,000+ Asian Alone
onehundred000up = [data['earn_m_fulltime_100000ormore_asianalone'], data['earn_f_fulltime_100000ormore_asianalone'], 
                   data['earn_m_other_100000ormore_asianalone'], data['earn_f_other_100000ormore_asianalone']]
data['Earnings:100,000 and Over Asian Alone'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over Asian Alone'] = percent(data['Earnings:100,000 and Over Asian Alone'], data['earn_asianalone_total_series'])
#Native Hawaiian and Other Pacific Islander Alone
#less than $10,000 Native Hawaiian and Other Pacific Islander Alone
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_nativehawaiianotherpacificislanderalone'], 
                 data['earn_m_fulltime_2500to4999_nativehawaiianotherpacificislanderalone'], 
                 data['earn_m_fulltime_5000to7499_nativehawaiianotherpacificislanderalone'], 
                 data['earn_m_fulltime_7500to9999_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_fulltime_1to2499orloss_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_fulltime_2500to4999_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_fulltime_5000to7499_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_fulltime_7500to9999_nativehawaiianotherpacificislanderalone'], 
                 data['earn_m_other_1to2499orloss_nativehawaiianotherpacificislanderalone'], 
                 data['earn_m_other_2500to4999_nativehawaiianotherpacificislanderalone'], 
                 data['earn_m_other_5000to7499_nativehawaiianotherpacificislanderalone'], 
                 data['earn_m_other_7500to9999_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_other_1to2499orloss_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_other_2500to4999_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_other_5000to7499_nativehawaiianotherpacificislanderalone'], 
                 data['earn_f_other_7500to9999_nativehawaiianotherpacificislanderalone']]              
data['Earnings:Less than 10,000 Native Hawaiian and Other Pacific Islander Alone'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 Native Hawaiian and Other Pacific Islander Alone'] = percent(data['Earnings:Less than 10,000 Native Hawaiian and Other Pacific Islander Alone'],
                                                                                               data['earn_nativehawaiianotherpacificislanderalone_total_series'])
#less than $25,000 Native Hawaiian and Other Pacific Islander Alone
lessthan25 = [data['earn_m_fulltime_1to2499orloss_nativehawaiianotherpacificislanderalone'], data['earn_m_fulltime_2500to4999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_fulltime_5000to7499_nativehawaiianotherpacificislanderalone'],data['earn_m_fulltime_7500to9999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_fulltime_10000to12499_nativehawaiianotherpacificislanderalone'],data['earn_m_fulltime_12500to14999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_fulltime_15000to17499_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_fulltime_17500to19999_nativehawaiianotherpacificislanderalone'],
              data['earn_m_fulltime_20000to22499_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_fulltime_22500to24999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_fulltime_1to2499orloss_nativehawaiianotherpacificislanderalone'], data['earn_f_fulltime_2500to4999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_fulltime_5000to7499_nativehawaiianotherpacificislanderalone'], data['earn_f_fulltime_7500to9999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_fulltime_10000to12499_nativehawaiianotherpacificislanderalone'],data['earn_f_fulltime_12500to14999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_fulltime_15000to17499_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_fulltime_17500to19999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_fulltime_20000to22499_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_fulltime_22500to24999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_other_1to2499orloss_nativehawaiianotherpacificislanderalone'], data['earn_m_other_2500to4999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_other_5000to7499_nativehawaiianotherpacificislanderalone'], data['earn_m_other_7500to9999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_other_10000to12499_nativehawaiianotherpacificislanderalone'],data['earn_m_other_12500to14999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_other_15000to17499_nativehawaiianotherpacificislanderalone'], data['earn_m_other_17500to19999_nativehawaiianotherpacificislanderalone'], 
              data['earn_m_other_20000to22499_nativehawaiianotherpacificislanderalone'], data['earn_m_other_22500to24999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_other_1to2499orloss_nativehawaiianotherpacificislanderalone'], data['earn_f_other_2500to4999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_other_5000to7499_nativehawaiianotherpacificislanderalone'], data['earn_f_other_7500to9999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_other_10000to12499_nativehawaiianotherpacificislanderalone'], data['earn_f_other_12500to14999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_other_15000to17499_nativehawaiianotherpacificislanderalone'], data['earn_f_other_17500to19999_nativehawaiianotherpacificislanderalone'], 
              data['earn_f_other_20000to22499_nativehawaiianotherpacificislanderalone'], data['earn_f_other_22500to24999_nativehawaiianotherpacificislanderalone']]
data['Earnings:Less than 25,000 Native Hawaiian and Other Pacific Islander Alone'] = sum(lessthan25)
data['Earnings:% Less than 25,000 Native Hawaiian and Other Pacific Islander Alone'] = percent(data['Earnings:Less than 25,000 Native Hawaiian and Other Pacific Islander Alone'],
                                                                                               data['earn_nativehawaiianotherpacificislanderalone_total_series'])
#$25,000 to $49,999 Native Hawaiian and Other Pacific Islander Alone
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_m_fulltime_30000to34999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_m_fulltime_35000to39999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_m_fulltime_40000to44999_nativehawaiianotherpacificislanderalone'],
                     data['earn_m_fulltime_45000to49999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_fulltime_25000to29999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_fulltime_30000to34999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_fulltime_35000to39999_nativehawaiianotherpacificislanderalone'],
                     data['earn_f_fulltime_40000to44999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_fulltime_45000to49999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_m_other_25000to29999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_m_other_30000to34999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_m_other_35000to39999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_m_other_40000to44999_nativehawaiianotherpacificislanderalone'],
                     data['earn_m_other_45000to49999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_other_25000to29999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_other_30000to34999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_other_35000to39999_nativehawaiianotherpacificislanderalone'],
                     data['earn_f_other_40000to44999_nativehawaiianotherpacificislanderalone'], 
                     data['earn_f_other_45000to49999_nativehawaiianotherpacificislanderalone']]
data['Earnings:25,000 to 49,999 Native Hawaiian and Other Pacific Islander Alone'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 Native Hawaiian and Other Pacific Islander Alone'] = percent(data['Earnings:25,000 to 49,999 Native Hawaiian and Other Pacific Islander Alone'],
                                                                                               data['earn_nativehawaiianotherpacificislanderalone_total_series'])
#$50,000 to $74,000 Native Hawaiian and Other Pacific Islander Alone
fifty000to74999 = [data['earn_m_fulltime_50000to54999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_m_fulltime_55000to64999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_m_fulltime_65000to74999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_f_fulltime_50000to54999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_f_fulltime_55000to64999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_f_fulltime_65000to74999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_m_other_50000to54999_nativehawaiianotherpacificislanderalone'], data['earn_m_other_55000to64999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_m_other_65000to74999_nativehawaiianotherpacificislanderalone'], data['earn_f_other_50000to54999_nativehawaiianotherpacificislanderalone'], 
                   data['earn_f_other_55000to64999_nativehawaiianotherpacificislanderalone'], data['earn_f_other_65000to74999_nativehawaiianotherpacificislanderalone']]
data['Earnings:50,000 to 74,999 Native Hawaiian and Other Pacific Islander Alone'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 Native Hawaiian and Other Pacific Islander Alone'] = percent(data['Earnings:50,000 to 74,999 Native Hawaiian and Other Pacific Islander Alone'],
                                                                                               data['earn_nativehawaiianotherpacificislanderalone_total_series'])
#$75,000 to $99,999 Native Hawaiian and Other Pacific Islander Alone
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_nativehawaiianotherpacificislanderalone'], 
                      data['earn_f_fulltime_75000to99999_nativehawaiianotherpacificislanderalone'], 
                      data['earn_m_other_75000to99999_nativehawaiianotherpacificislanderalone'], 
                      data['earn_f_other_75000to99999_nativehawaiianotherpacificislanderalone']]
data['Earnings:75,000 to 99,999 Native Hawaiian and Other Pacific Islander Alone'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 Native Hawaiian and Other Pacific Islander Alone'] = percent(data['Earnings:75,000 to 99,999 Native Hawaiian and Other Pacific Islander Alone'], 
                                                          data['earn_nativehawaiianotherpacificislanderalone_total_series'])
#$100,000+ Native Hawaiian and Other Pacific Islander Alone
onehundred000up = [data['earn_m_fulltime_100000ormore_nativehawaiianotherpacificislanderalone'], 
                   data['earn_f_fulltime_100000ormore_nativehawaiianotherpacificislanderalone'], 
                   data['earn_m_other_100000ormore_nativehawaiianotherpacificislanderalone'], 
                   data['earn_f_other_100000ormore_nativehawaiianotherpacificislanderalone']]
data['Earnings:100,000 and Over Native Hawaiian and Other Pacific Islander Alone'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over Native Hawaiian and Other Pacific Islander Alone'] = percent(data['Earnings:100,000 and Over Native Hawaiian and Other Pacific Islander Alone'], 
                                                                                               data['earn_nativehawaiianotherpacificislanderalone_total_series'])
#Some Other Race Alone
#less than $10,000 Some Other Race Alone
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_someotherracealone'], data['earn_m_fulltime_2500to4999_someotherracealone'], 
                 data['earn_m_fulltime_5000to7499_someotherracealone'], data['earn_m_fulltime_7500to9999_someotherracealone'], 
                 data['earn_f_fulltime_1to2499orloss_someotherracealone'], data['earn_f_fulltime_2500to4999_someotherracealone'], 
                 data['earn_f_fulltime_5000to7499_someotherracealone'], data['earn_f_fulltime_7500to9999_someotherracealone'], 
                 data['earn_m_other_1to2499orloss_someotherracealone'], data['earn_m_other_2500to4999_someotherracealone'], 
                 data['earn_m_other_5000to7499_someotherracealone'], data['earn_m_other_7500to9999_someotherracealone'], 
                 data['earn_f_other_1to2499orloss_someotherracealone'], data['earn_f_other_2500to4999_someotherracealone'], 
                 data['earn_f_other_5000to7499_someotherracealone'], data['earn_f_other_7500to9999_someotherracealone']]              
data['Earnings:Less than 10,000 Some Other Race Alone'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 Some Other Race Alone'] = percent(data['Earnings:Less than 10,000 Some Other Race Alone'],
                                                                    data['earn_someotherracealone_total_series'])
#less than $25,000 Some Other Race Alone
lessthan25 = [data['earn_m_fulltime_1to2499orloss_someotherracealone'], data['earn_m_fulltime_2500to4999_someotherracealone'], 
              data['earn_m_fulltime_5000to7499_someotherracealone'],data['earn_m_fulltime_7500to9999_someotherracealone'], 
              data['earn_m_fulltime_10000to12499_someotherracealone'],data['earn_m_fulltime_12500to14999_someotherracealone'], 
              data['earn_m_fulltime_15000to17499_someotherracealone'], data['earn_m_fulltime_17500to19999_someotherracealone'],
              data['earn_m_fulltime_20000to22499_someotherracealone'], data['earn_m_fulltime_22500to24999_someotherracealone'], 
              data['earn_f_fulltime_1to2499orloss_someotherracealone'], data['earn_f_fulltime_2500to4999_someotherracealone'], 
              data['earn_f_fulltime_5000to7499_someotherracealone'], data['earn_f_fulltime_7500to9999_someotherracealone'], 
              data['earn_f_fulltime_10000to12499_someotherracealone'],data['earn_f_fulltime_12500to14999_someotherracealone'], 
              data['earn_f_fulltime_15000to17499_someotherracealone'], data['earn_f_fulltime_17500to19999_someotherracealone'], 
              data['earn_f_fulltime_20000to22499_someotherracealone'], data['earn_f_fulltime_22500to24999_someotherracealone'], 
              data['earn_m_other_1to2499orloss_someotherracealone'], data['earn_m_other_2500to4999_someotherracealone'], 
              data['earn_m_other_5000to7499_someotherracealone'], data['earn_m_other_7500to9999_someotherracealone'], 
              data['earn_m_other_10000to12499_someotherracealone'],data['earn_m_other_12500to14999_someotherracealone'], 
              data['earn_m_other_15000to17499_someotherracealone'], data['earn_m_other_17500to19999_someotherracealone'], 
              data['earn_m_other_20000to22499_someotherracealone'], data['earn_m_other_22500to24999_someotherracealone'], 
              data['earn_f_other_1to2499orloss_someotherracealone'], data['earn_f_other_2500to4999_someotherracealone'], 
              data['earn_f_other_5000to7499_someotherracealone'], data['earn_f_other_7500to9999_someotherracealone'], 
              data['earn_f_other_10000to12499_someotherracealone'], data['earn_f_other_12500to14999_someotherracealone'], 
              data['earn_f_other_15000to17499_someotherracealone'], data['earn_f_other_17500to19999_someotherracealone'], 
              data['earn_f_other_20000to22499_someotherracealone'], data['earn_f_other_22500to24999_someotherracealone']]
data['Earnings:Less than 25,000 Some Other Race Alone'] = sum(lessthan25)
data['Earnings:% Less than 25,000 Some Other Race Alone'] = percent(data['Earnings:Less than 25,000 Some Other Race Alone'],
                                                                    data['earn_someotherracealone_total_series'])
#$25,000 to $49,999 Some Other Race Alone
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_someotherracealone'], data['earn_m_fulltime_30000to34999_someotherracealone'], 
                     data['earn_m_fulltime_35000to39999_someotherracealone'], data['earn_m_fulltime_40000to44999_someotherracealone'],
                     data['earn_m_fulltime_45000to49999_someotherracealone'], data['earn_f_fulltime_25000to29999_someotherracealone'], 
                     data['earn_f_fulltime_30000to34999_someotherracealone'], data['earn_f_fulltime_35000to39999_someotherracealone'],
                     data['earn_f_fulltime_40000to44999_someotherracealone'], data['earn_f_fulltime_45000to49999_someotherracealone'], 
                     data['earn_m_other_25000to29999_someotherracealone'], data['earn_m_other_30000to34999_someotherracealone'], 
                     data['earn_m_other_35000to39999_someotherracealone'], data['earn_m_other_40000to44999_someotherracealone'],
                     data['earn_m_other_45000to49999_someotherracealone'], data['earn_f_other_25000to29999_someotherracealone'], 
                     data['earn_f_other_30000to34999_someotherracealone'], data['earn_f_other_35000to39999_someotherracealone'],
                     data['earn_f_other_40000to44999_someotherracealone'], data['earn_f_other_45000to49999_someotherracealone']]
data['Earnings:25,000 to 49,999 Some Other Race Alone'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 Some Other Race Alone'] = percent(data['Earnings:25,000 to 49,999 Some Other Race Alone'],
                                                                    data['earn_someotherracealone_total_series'])
#$50,000 to $74,000 Some Other Race Alone
fifty000to74999 = [data['earn_m_fulltime_50000to54999_someotherracealone'], data['earn_m_fulltime_55000to64999_someotherracealone'], 
                   data['earn_m_fulltime_65000to74999_someotherracealone'], data['earn_f_fulltime_50000to54999_someotherracealone'], 
                   data['earn_f_fulltime_55000to64999_someotherracealone'], data['earn_f_fulltime_65000to74999_someotherracealone'], 
                   data['earn_m_other_50000to54999_someotherracealone'], data['earn_m_other_55000to64999_someotherracealone'], 
                   data['earn_m_other_65000to74999_someotherracealone'], data['earn_f_other_50000to54999_someotherracealone'], 
                   data['earn_f_other_55000to64999_someotherracealone'], data['earn_f_other_65000to74999_someotherracealone']]
data['Earnings:50,000 to 74,999 Some Other Race Alone'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 Some Other Race Alone'] = percent(data['Earnings:50,000 to 74,999 Some Other Race Alone'],
                                                                    data['earn_someotherracealone_total_series'])
#$75,000 to $99,999 Some Other Race Alone
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_someotherracealone'], 
                      data['earn_f_fulltime_75000to99999_someotherracealone'], 
                      data['earn_m_other_75000to99999_someotherracealone'], 
                      data['earn_f_other_75000to99999_someotherracealone']]
data['Earnings:75,000 to 99,999 Some Other Race Alone'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 Some Other Race Alone'] = percent(data['Earnings:75,000 to 99,999 Some Other Race Alone'], 
                                                                    data['earn_someotherracealone_total_series'])
#$100,000+ Some Other Race Alone
onehundred000up = [data['earn_m_fulltime_100000ormore_someotherracealone'], 
                   data['earn_f_fulltime_100000ormore_someotherracealone'], 
                   data['earn_m_other_100000ormore_someotherracealone'], 
                   data['earn_f_other_100000ormore_someotherracealone']]
data['Earnings:100,000 and Over Some Other Race Alone'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over Some Other Race Alone'] = percent(data['Earnings:100,000 and Over Some Other Race Alone'], 
                                                                    data['earn_someotherracealone_total_series'])
#Two or More Races
#less than $10,000 Two or More Races
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_twoormoreraces'], data['earn_m_fulltime_2500to4999_twoormoreraces'], 
                 data['earn_m_fulltime_5000to7499_twoormoreraces'], data['earn_m_fulltime_7500to9999_twoormoreraces'], 
                 data['earn_f_fulltime_1to2499orloss_twoormoreraces'], data['earn_f_fulltime_2500to4999_twoormoreraces'], 
                 data['earn_f_fulltime_5000to7499_twoormoreraces'], data['earn_f_fulltime_7500to9999_twoormoreraces'], 
                 data['earn_m_other_1to2499orloss_twoormoreraces'], data['earn_m_other_2500to4999_twoormoreraces'], 
                 data['earn_m_other_5000to7499_twoormoreraces'], data['earn_m_other_7500to9999_twoormoreraces'], 
                 data['earn_f_other_1to2499orloss_twoormoreraces'], data['earn_f_other_2500to4999_twoormoreraces'], 
                 data['earn_f_other_5000to7499_twoormoreraces'], data['earn_f_other_7500to9999_twoormoreraces']]              
data['Earnings:Less than 10,000 Two or More Races'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 Two or More Races'] = percent(data['Earnings:Less than 10,000 Two or More Races'], data['earn_twoormoreraces_total_series'])
#less than $25,000 Two or More Races
lessthan25 = [data['earn_m_fulltime_1to2499orloss_twoormoreraces'], data['earn_m_fulltime_2500to4999_twoormoreraces'], 
              data['earn_m_fulltime_5000to7499_twoormoreraces'],data['earn_m_fulltime_7500to9999_twoormoreraces'], 
              data['earn_m_fulltime_10000to12499_twoormoreraces'],data['earn_m_fulltime_12500to14999_twoormoreraces'], 
              data['earn_m_fulltime_15000to17499_twoormoreraces'], data['earn_m_fulltime_17500to19999_twoormoreraces'],
              data['earn_m_fulltime_20000to22499_twoormoreraces'], data['earn_m_fulltime_22500to24999_twoormoreraces'], 
              data['earn_f_fulltime_1to2499orloss_twoormoreraces'], data['earn_f_fulltime_2500to4999_twoormoreraces'], 
              data['earn_f_fulltime_5000to7499_twoormoreraces'], data['earn_f_fulltime_7500to9999_twoormoreraces'], 
              data['earn_f_fulltime_10000to12499_twoormoreraces'],data['earn_f_fulltime_12500to14999_twoormoreraces'], 
              data['earn_f_fulltime_15000to17499_twoormoreraces'], data['earn_f_fulltime_17500to19999_twoormoreraces'], 
              data['earn_f_fulltime_20000to22499_twoormoreraces'], data['earn_f_fulltime_22500to24999_twoormoreraces'], 
              data['earn_m_other_1to2499orloss_twoormoreraces'], data['earn_m_other_2500to4999_twoormoreraces'], 
              data['earn_m_other_5000to7499_twoormoreraces'], data['earn_m_other_7500to9999_twoormoreraces'], 
              data['earn_m_other_10000to12499_twoormoreraces'],data['earn_m_other_12500to14999_twoormoreraces'], 
              data['earn_m_other_15000to17499_twoormoreraces'], data['earn_m_other_17500to19999_twoormoreraces'], 
              data['earn_m_other_20000to22499_twoormoreraces'], data['earn_m_other_22500to24999_twoormoreraces'], 
              data['earn_f_other_1to2499orloss_twoormoreraces'], data['earn_f_other_2500to4999_twoormoreraces'], 
              data['earn_f_other_5000to7499_twoormoreraces'], data['earn_f_other_7500to9999_twoormoreraces'], 
              data['earn_f_other_10000to12499_twoormoreraces'], data['earn_f_other_12500to14999_twoormoreraces'], 
              data['earn_f_other_15000to17499_twoormoreraces'], data['earn_f_other_17500to19999_twoormoreraces'], 
              data['earn_f_other_20000to22499_twoormoreraces'], data['earn_f_other_22500to24999_twoormoreraces']]
data['Earnings:Less than 25,000 Two or More Races'] = sum(lessthan25)
data['Earnings:% Less than 25,000 Two or More Races'] = percent(data['Earnings:Less than 25,000 Two or More Races'],data['earn_twoormoreraces_total_series'])
#$25,000 to $49,999 Two or More Races
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_twoormoreraces'], data['earn_m_fulltime_30000to34999_twoormoreraces'], 
                     data['earn_m_fulltime_35000to39999_twoormoreraces'], data['earn_m_fulltime_40000to44999_twoormoreraces'],
                     data['earn_m_fulltime_45000to49999_twoormoreraces'], data['earn_f_fulltime_25000to29999_twoormoreraces'], 
                     data['earn_f_fulltime_30000to34999_twoormoreraces'], data['earn_f_fulltime_35000to39999_twoormoreraces'],
                     data['earn_f_fulltime_40000to44999_twoormoreraces'], data['earn_f_fulltime_45000to49999_twoormoreraces'], 
                     data['earn_m_other_25000to29999_twoormoreraces'], data['earn_m_other_30000to34999_twoormoreraces'], 
                     data['earn_m_other_35000to39999_twoormoreraces'], data['earn_m_other_40000to44999_twoormoreraces'],
                     data['earn_m_other_45000to49999_twoormoreraces'], data['earn_f_other_25000to29999_twoormoreraces'], 
                     data['earn_f_other_30000to34999_twoormoreraces'], data['earn_f_other_35000to39999_twoormoreraces'],
                     data['earn_f_other_40000to44999_twoormoreraces'], data['earn_f_other_45000to49999_twoormoreraces']]
data['Earnings:25,000 to 49,999 Two or More Races'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 Two or More Races'] = percent(data['Earnings:25,000 to 49,999 Two or More Races'],
                                                                    data['earn_twoormoreraces_total_series'])
#$50,000 to $74,000 Two or More Races
fifty000to74999 = [data['earn_m_fulltime_50000to54999_twoormoreraces'], data['earn_m_fulltime_55000to64999_twoormoreraces'], 
                   data['earn_m_fulltime_65000to74999_twoormoreraces'], data['earn_f_fulltime_50000to54999_twoormoreraces'], 
                   data['earn_f_fulltime_55000to64999_twoormoreraces'], data['earn_f_fulltime_65000to74999_twoormoreraces'], 
                   data['earn_m_other_50000to54999_twoormoreraces'], data['earn_m_other_55000to64999_twoormoreraces'], 
                   data['earn_m_other_65000to74999_twoormoreraces'], data['earn_f_other_50000to54999_twoormoreraces'], 
                   data['earn_f_other_55000to64999_twoormoreraces'], data['earn_f_other_65000to74999_twoormoreraces']]
data['Earnings:50,000 to 74,999 Two or More Races'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 Two or More Races'] = percent(data['Earnings:50,000 to 74,999 Two or More Races'],data['earn_twoormoreraces_total_series'])
#$75,000 to $99,999 Two or More Races
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_twoormoreraces'], data['earn_f_fulltime_75000to99999_twoormoreraces'], 
                      data['earn_m_other_75000to99999_twoormoreraces'], data['earn_f_other_75000to99999_twoormoreraces']]
data['Earnings:75,000 to 99,999 Two or More Races'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 Two or More Races'] = percent(data['Earnings:75,000 to 99,999 Two or More Races'], data['earn_twoormoreraces_total_series'])
#$100,000+ Two or More Races
onehundred000up = [data['earn_m_fulltime_100000ormore_twoormoreraces'], data['earn_f_fulltime_100000ormore_twoormoreraces'], 
                   data['earn_m_other_100000ormore_twoormoreraces'], data['earn_f_other_100000ormore_twoormoreraces']]
data['Earnings:100,000 and Over Two or More Races'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over Two or More Races'] = percent(data['Earnings:100,000 and Over Two or More Races'], data['earn_twoormoreraces_total_series'])
#Non-Minority
#less than $10,000 Non-Minority
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_whitealonenothispanicorlatino'], data['earn_m_fulltime_2500to4999_whitealonenothispanicorlatino'], 
                 data['earn_m_fulltime_5000to7499_whitealonenothispanicorlatino'], data['earn_m_fulltime_7500to9999_whitealonenothispanicorlatino'], 
                 data['earn_f_fulltime_1to2499orloss_whitealonenothispanicorlatino'], data['earn_f_fulltime_2500to4999_whitealonenothispanicorlatino'], 
                 data['earn_f_fulltime_5000to7499_whitealonenothispanicorlatino'], data['earn_f_fulltime_7500to9999_whitealonenothispanicorlatino'], 
                 data['earn_m_other_1to2499orloss_whitealonenothispanicorlatino'], data['earn_m_other_2500to4999_whitealonenothispanicorlatino'], 
                 data['earn_m_other_5000to7499_whitealonenothispanicorlatino'], data['earn_m_other_7500to9999_whitealonenothispanicorlatino'], 
                 data['earn_f_other_1to2499orloss_whitealonenothispanicorlatino'], data['earn_f_other_2500to4999_whitealonenothispanicorlatino'], 
                 data['earn_f_other_5000to7499_whitealonenothispanicorlatino'], data['earn_f_other_7500to9999_whitealonenothispanicorlatino']]              
data['Earnings:Less than 10,000 Non-Minority'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 Non-Minority'] = percent(data['Earnings:Less than 10,000 Non-Minority'], data['earn_whitealonenothispanicorlatino_total_series'])
#less than $25,000 Non-Minority
lessthan25 = [data['earn_m_fulltime_1to2499orloss_whitealonenothispanicorlatino'], data['earn_m_fulltime_2500to4999_whitealonenothispanicorlatino'], 
              data['earn_m_fulltime_5000to7499_whitealonenothispanicorlatino'],data['earn_m_fulltime_7500to9999_whitealonenothispanicorlatino'], 
              data['earn_m_fulltime_10000to12499_whitealonenothispanicorlatino'],data['earn_m_fulltime_12500to14999_whitealonenothispanicorlatino'], 
              data['earn_m_fulltime_15000to17499_whitealonenothispanicorlatino'], data['earn_m_fulltime_17500to19999_whitealonenothispanicorlatino'],
              data['earn_m_fulltime_20000to22499_whitealonenothispanicorlatino'], data['earn_m_fulltime_22500to24999_whitealonenothispanicorlatino'], 
              data['earn_f_fulltime_1to2499orloss_whitealonenothispanicorlatino'], data['earn_f_fulltime_2500to4999_whitealonenothispanicorlatino'], 
              data['earn_f_fulltime_5000to7499_whitealonenothispanicorlatino'], data['earn_f_fulltime_7500to9999_whitealonenothispanicorlatino'], 
              data['earn_f_fulltime_10000to12499_whitealonenothispanicorlatino'],data['earn_f_fulltime_12500to14999_whitealonenothispanicorlatino'], 
              data['earn_f_fulltime_15000to17499_whitealonenothispanicorlatino'], data['earn_f_fulltime_17500to19999_whitealonenothispanicorlatino'], 
              data['earn_f_fulltime_20000to22499_whitealonenothispanicorlatino'], data['earn_f_fulltime_22500to24999_whitealonenothispanicorlatino'], 
              data['earn_m_other_1to2499orloss_whitealonenothispanicorlatino'], data['earn_m_other_2500to4999_whitealonenothispanicorlatino'], 
              data['earn_m_other_5000to7499_whitealonenothispanicorlatino'], data['earn_m_other_7500to9999_whitealonenothispanicorlatino'], 
              data['earn_m_other_10000to12499_whitealonenothispanicorlatino'],data['earn_m_other_12500to14999_whitealonenothispanicorlatino'], 
              data['earn_m_other_15000to17499_whitealonenothispanicorlatino'], data['earn_m_other_17500to19999_whitealonenothispanicorlatino'], 
              data['earn_m_other_20000to22499_whitealonenothispanicorlatino'], data['earn_m_other_22500to24999_whitealonenothispanicorlatino'], 
              data['earn_f_other_1to2499orloss_whitealonenothispanicorlatino'], data['earn_f_other_2500to4999_whitealonenothispanicorlatino'], 
              data['earn_f_other_5000to7499_whitealonenothispanicorlatino'], data['earn_f_other_7500to9999_whitealonenothispanicorlatino'], 
              data['earn_f_other_10000to12499_whitealonenothispanicorlatino'], data['earn_f_other_12500to14999_whitealonenothispanicorlatino'], 
              data['earn_f_other_15000to17499_whitealonenothispanicorlatino'], data['earn_f_other_17500to19999_whitealonenothispanicorlatino'], 
              data['earn_f_other_20000to22499_whitealonenothispanicorlatino'], data['earn_f_other_22500to24999_whitealonenothispanicorlatino']]
data['Earnings:Less than 25,000 Non-Minority'] = sum(lessthan25)
data['Earnings:% Less than 25,000 Non-Minority'] = percent(data['Earnings:Less than 25,000 Non-Minority'],
                                                           data['earn_whitealonenothispanicorlatino_total_series'])
#$25,000 to $49,999 Non-Minority
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_whitealonenothispanicorlatino'], data['earn_m_fulltime_30000to34999_whitealonenothispanicorlatino'], 
                     data['earn_m_fulltime_35000to39999_whitealonenothispanicorlatino'], data['earn_m_fulltime_40000to44999_whitealonenothispanicorlatino'],
                     data['earn_m_fulltime_45000to49999_whitealonenothispanicorlatino'], data['earn_f_fulltime_25000to29999_whitealonenothispanicorlatino'], 
                     data['earn_f_fulltime_30000to34999_whitealonenothispanicorlatino'], data['earn_f_fulltime_35000to39999_whitealonenothispanicorlatino'],
                     data['earn_f_fulltime_40000to44999_whitealonenothispanicorlatino'], data['earn_f_fulltime_45000to49999_whitealonenothispanicorlatino'], 
                     data['earn_m_other_25000to29999_whitealonenothispanicorlatino'], data['earn_m_other_30000to34999_whitealonenothispanicorlatino'], 
                     data['earn_m_other_35000to39999_whitealonenothispanicorlatino'], data['earn_m_other_40000to44999_whitealonenothispanicorlatino'],
                     data['earn_m_other_45000to49999_whitealonenothispanicorlatino'], data['earn_f_other_25000to29999_whitealonenothispanicorlatino'], 
                     data['earn_f_other_30000to34999_whitealonenothispanicorlatino'], data['earn_f_other_35000to39999_whitealonenothispanicorlatino'],
                     data['earn_f_other_40000to44999_whitealonenothispanicorlatino'], data['earn_f_other_45000to49999_whitealonenothispanicorlatino']]
data['Earnings:25,000 to 49,999 Non-Minority'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 Non-Minority'] = percent(data['Earnings:25,000 to 49,999 Non-Minority'],data['earn_whitealonenothispanicorlatino_total_series'])
#$50,000 to $74,000 Non-Minority
fifty000to74999 = [data['earn_m_fulltime_50000to54999_whitealonenothispanicorlatino'], data['earn_m_fulltime_55000to64999_whitealonenothispanicorlatino'], 
                   data['earn_m_fulltime_65000to74999_whitealonenothispanicorlatino'], data['earn_f_fulltime_50000to54999_whitealonenothispanicorlatino'], 
                   data['earn_f_fulltime_55000to64999_whitealonenothispanicorlatino'], data['earn_f_fulltime_65000to74999_whitealonenothispanicorlatino'], 
                   data['earn_m_other_50000to54999_whitealonenothispanicorlatino'], data['earn_m_other_55000to64999_whitealonenothispanicorlatino'], 
                   data['earn_m_other_65000to74999_whitealonenothispanicorlatino'], data['earn_f_other_50000to54999_whitealonenothispanicorlatino'], 
                   data['earn_f_other_55000to64999_whitealonenothispanicorlatino'], data['earn_f_other_65000to74999_whitealonenothispanicorlatino']]
data['Earnings:50,000 to 74,999 Non-Minority'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 Non-Minority'] = percent(data['Earnings:50,000 to 74,999 Non-Minority'],data['earn_whitealonenothispanicorlatino_total_series'])
#$75,000 to $99,999 Non-Minority
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_whitealonenothispanicorlatino'], data['earn_f_fulltime_75000to99999_whitealonenothispanicorlatino'], 
                      data['earn_m_other_75000to99999_whitealonenothispanicorlatino'], data['earn_f_other_75000to99999_whitealonenothispanicorlatino']]
data['Earnings:75,000 to 99,999 Non-Minority'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 Non-Minority'] = percent(data['Earnings:75,000 to 99,999 Non-Minority'], data['earn_whitealonenothispanicorlatino_total_series'])
#$100,000+ Non-Minority
onehundred000up = [data['earn_m_fulltime_100000ormore_whitealonenothispanicorlatino'], data['earn_f_fulltime_100000ormore_whitealonenothispanicorlatino'], 
                   data['earn_m_other_100000ormore_whitealonenothispanicorlatino'], data['earn_f_other_100000ormore_whitealonenothispanicorlatino']]
data['Earnings:100,000 and Over Non-Minority'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over Non-Minority'] = percent(data['Earnings:100,000 and Over Non-Minority'], data['earn_whitealonenothispanicorlatino_total_series'])
#Hispanic or Latino
#less than $10,000 Hispanic or Latino
lessthan10000 = [data['earn_m_fulltime_1to2499orloss_hispanicorlatino'], data['earn_m_fulltime_2500to4999_hispanicorlatino'], 
                 data['earn_m_fulltime_5000to7499_hispanicorlatino'], data['earn_m_fulltime_7500to9999_hispanicorlatino'], 
                 data['earn_f_fulltime_1to2499orloss_hispanicorlatino'], data['earn_f_fulltime_2500to4999_hispanicorlatino'], 
                 data['earn_f_fulltime_5000to7499_hispanicorlatino'], data['earn_f_fulltime_7500to9999_hispanicorlatino'], 
                 data['earn_m_other_1to2499orloss_hispanicorlatino'], data['earn_m_other_2500to4999_hispanicorlatino'], 
                 data['earn_m_other_5000to7499_hispanicorlatino'], data['earn_m_other_7500to9999_hispanicorlatino'], 
                 data['earn_f_other_1to2499orloss_hispanicorlatino'], data['earn_f_other_2500to4999_hispanicorlatino'], 
                 data['earn_f_other_5000to7499_hispanicorlatino'], data['earn_f_other_7500to9999_hispanicorlatino']]              
data['Earnings:Less than 10,000 Hispanic or Latino'] = sum(lessthan10000)
data['Earnings:% Less than 10,000 Hispanic or Latino'] = percent(data['Earnings:Less than 10,000 Hispanic or Latino'], data['earn_hispanicorlatino_total_series'])
#less than $25,000 Hispanic or Latino
lessthan25 = [data['earn_m_fulltime_1to2499orloss_hispanicorlatino'], data['earn_m_fulltime_2500to4999_hispanicorlatino'], 
              data['earn_m_fulltime_5000to7499_hispanicorlatino'],data['earn_m_fulltime_7500to9999_hispanicorlatino'], 
              data['earn_m_fulltime_10000to12499_hispanicorlatino'],data['earn_m_fulltime_12500to14999_hispanicorlatino'], 
              data['earn_m_fulltime_15000to17499_hispanicorlatino'], data['earn_m_fulltime_17500to19999_hispanicorlatino'],
              data['earn_m_fulltime_20000to22499_hispanicorlatino'], data['earn_m_fulltime_22500to24999_hispanicorlatino'], 
              data['earn_f_fulltime_1to2499orloss_hispanicorlatino'], data['earn_f_fulltime_2500to4999_hispanicorlatino'], 
              data['earn_f_fulltime_5000to7499_hispanicorlatino'], data['earn_f_fulltime_7500to9999_hispanicorlatino'], 
              data['earn_f_fulltime_10000to12499_hispanicorlatino'],data['earn_f_fulltime_12500to14999_hispanicorlatino'], 
              data['earn_f_fulltime_15000to17499_hispanicorlatino'], data['earn_f_fulltime_17500to19999_hispanicorlatino'], 
              data['earn_f_fulltime_20000to22499_hispanicorlatino'], data['earn_f_fulltime_22500to24999_hispanicorlatino'], 
              data['earn_m_other_1to2499orloss_hispanicorlatino'], data['earn_m_other_2500to4999_hispanicorlatino'], 
              data['earn_m_other_5000to7499_hispanicorlatino'], data['earn_m_other_7500to9999_hispanicorlatino'], 
              data['earn_m_other_10000to12499_hispanicorlatino'],data['earn_m_other_12500to14999_hispanicorlatino'], 
              data['earn_m_other_15000to17499_hispanicorlatino'], data['earn_m_other_17500to19999_hispanicorlatino'], 
              data['earn_m_other_20000to22499_hispanicorlatino'], data['earn_m_other_22500to24999_hispanicorlatino'], 
              data['earn_f_other_1to2499orloss_hispanicorlatino'], data['earn_f_other_2500to4999_hispanicorlatino'], 
              data['earn_f_other_5000to7499_hispanicorlatino'], data['earn_f_other_7500to9999_hispanicorlatino'], 
              data['earn_f_other_10000to12499_hispanicorlatino'], data['earn_f_other_12500to14999_hispanicorlatino'], 
              data['earn_f_other_15000to17499_hispanicorlatino'], data['earn_f_other_17500to19999_hispanicorlatino'], 
              data['earn_f_other_20000to22499_hispanicorlatino'], data['earn_f_other_22500to24999_hispanicorlatino']]
data['Earnings:Less than 25,000 Hispanic or Latino'] = sum(lessthan25)
data['Earnings:% Less than 25,000 Hispanic or Latino'] = percent(data['Earnings:Less than 25,000 Hispanic or Latino'],data['earn_hispanicorlatino_total_series'])
#$25,000 to $49,999 Hispanic or Latino
twenty5000to49999 = [data['earn_m_fulltime_25000to29999_hispanicorlatino'], data['earn_m_fulltime_30000to34999_hispanicorlatino'], 
                     data['earn_m_fulltime_35000to39999_hispanicorlatino'], data['earn_m_fulltime_40000to44999_hispanicorlatino'],
                     data['earn_m_fulltime_45000to49999_hispanicorlatino'], data['earn_f_fulltime_25000to29999_hispanicorlatino'], 
                     data['earn_f_fulltime_30000to34999_hispanicorlatino'], data['earn_f_fulltime_35000to39999_hispanicorlatino'],
                     data['earn_f_fulltime_40000to44999_hispanicorlatino'], data['earn_f_fulltime_45000to49999_hispanicorlatino'], 
                     data['earn_m_other_25000to29999_hispanicorlatino'], data['earn_m_other_30000to34999_hispanicorlatino'], 
                     data['earn_m_other_35000to39999_hispanicorlatino'], data['earn_m_other_40000to44999_hispanicorlatino'],
                     data['earn_m_other_45000to49999_hispanicorlatino'], data['earn_f_other_25000to29999_hispanicorlatino'], 
                     data['earn_f_other_30000to34999_hispanicorlatino'], data['earn_f_other_35000to39999_hispanicorlatino'],
                     data['earn_f_other_40000to44999_hispanicorlatino'], data['earn_f_other_45000to49999_hispanicorlatino']]
data['Earnings:25,000 to 49,999 Hispanic or Latino'] = sum(twenty5000to49999)
data['Earnings:% 25,000 to 49,999 Hispanic or Latino'] = percent(data['Earnings:25,000 to 49,999 Hispanic or Latino'],data['earn_hispanicorlatino_total_series'])
#$50,000 to $74,000 Hispanic or Latino
fifty000to74999 = [data['earn_m_fulltime_50000to54999_hispanicorlatino'], data['earn_m_fulltime_55000to64999_hispanicorlatino'], 
                   data['earn_m_fulltime_65000to74999_hispanicorlatino'], data['earn_f_fulltime_50000to54999_hispanicorlatino'], 
                   data['earn_f_fulltime_55000to64999_hispanicorlatino'], data['earn_f_fulltime_65000to74999_hispanicorlatino'], 
                   data['earn_m_other_50000to54999_hispanicorlatino'], data['earn_m_other_55000to64999_hispanicorlatino'], 
                   data['earn_m_other_65000to74999_hispanicorlatino'], data['earn_f_other_50000to54999_hispanicorlatino'], 
                   data['earn_f_other_55000to64999_hispanicorlatino'], data['earn_f_other_65000to74999_hispanicorlatino']]
data['Earnings:50,000 to 74,999 Hispanic or Latino'] = sum(fifty000to74999)
data['Earnings:% 50,000 to 74,999 Hispanic or Latino'] = percent(data['Earnings:50,000 to 74,999 Hispanic or Latino'],data['earn_hispanicorlatino_total_series'])
#$75,000 to $99,999 Hispanic or Latino
seventy5000to99999 = [data['earn_m_fulltime_75000to99999_hispanicorlatino'], data['earn_f_fulltime_75000to99999_hispanicorlatino'], 
                      data['earn_m_other_75000to99999_hispanicorlatino'], data['earn_f_other_75000to99999_hispanicorlatino']]
data['Earnings:75,000 to 99,999 Hispanic or Latino'] = sum(seventy5000to99999)
data['Earnings:% 75,000 to 99,999 Hispanic or Latino'] = percent(data['Earnings:75,000 to 99,999 Hispanic or Latino'], data['earn_hispanicorlatino_total_series'])
#$100,000+ Hispanic or Latino
onehundred000up = [data['earn_m_fulltime_100000ormore_hispanicorlatino'], data['earn_f_fulltime_100000ormore_hispanicorlatino'], 
                   data['earn_m_other_100000ormore_hispanicorlatino'], data['earn_f_other_100000ormore_hispanicorlatino']]
data['Earnings:100,000 and Over Hispanic or Latino'] = sum(onehundred000up)
data['Earnings:% 100,000 and Over Hispanic or Latino'] = percent(data['Earnings:100,000 and Over Hispanic or Latino'], data['earn_hispanicorlatino_total_series'])
#earnings by race median
data['Earnings:Median'] = data['earn_medianearnings']
data['Earnings:Median, White Alone'] = data['earn_medianearnings_whitealone']
data['Earnings:Median, Black or African American Alone'] = data['earn_medianearnings_blackafricanamericanalone']
data['Earnings:Median, American Indian Alaska Native Alone'] = data['earn_medianearnings_americanindianalaskanativealone']
data['Earnings:Median, Asian Alone']= data['earn_medianearnings_asianalone']
data['Earnings:Median, Native Hawaiian Pacific Islander Alone'] = data['earn_medianearnings_nativehawaiianpacificislanderalone']
data['Earnings:Median, Some Other Race Alone'] = data['earn_medianearnings_someotherracealone']
data['Earnings:Median, Two or More Races'] = data['earn_medianearnings_twoormoreraces']
data['Earnings:Median, Non-Minority'] = data['earn_medianearnings_whitealonenothispanicorlatino']
data['Earnings:Median, Hispanic or Latino'] = data['earn_medianearnings_hispanicorlatino']
#earnings by race portion of earning groups
#percent of those earning less than 10,000
data['Earnings%:White Alone Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 White Alone'], data['Earnings:Less than 10,000'])
data['Earnings%:Black or African American Alone Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 Black or African American Alone'], 
                                                                                            data['Earnings:Less than 10,000'])
data['Earnings%:American Indian or Alaska Native Alone Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 American Indian or Alaska Native Alone'], 
                                                                                                   data['Earnings:Less than 10,000'])
data['Earnings%:Asian Alone Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 Asian Alone'], data['Earnings:Less than 10,000'])
data['Earnings%:Native Hawaiian and Other Pacific Islander Alone Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 Native Hawaiian and Other Pacific Islander Alone'], 
                                                                        data['Earnings:Less than 10,000'])
data['Earnings%:Some Other Race Alone Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 Some Other Race Alone'],
                                                                                  data['Earnings:Less than 10,000'])
data['Earnings%:Two or More Races Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 Two or More Races'], data['Earnings:Less than 10,000'])
data['Earnings%:Hispanic or Latino Portion of Earners Below 10,000'] = percent(data['Earnings:Less than 10,000 Hispanic or Latino'], data['Earnings:Less than 10,000'])
#less than 25
data['Earnings%:White Alone Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 White Alone'], data['Earnings:Less than 25,000'])
data['Earnings%:Black or African American Alone Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 Black or African American Alone'], 
                                                                                            data['Earnings:Less than 25,000'])
data['Earnings%:American Indian or Alaska Native Alone Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 American Indian or Alaska Native Alone'], 
                                                                                                   data['Earnings:Less than 25,000'])
data['Earnings%:Asian Alone Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 Asian Alone'], data['Earnings:Less than 25,000'])
data['Earnings%:Native Hawaiian and Other Pacific Islander Alone Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 Native Hawaiian and Other Pacific Islander Alone'], 
                                                                        data['Earnings:Less than 25,000'])
data['Earnings%:Some Other Race Alone Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 Some Other Race Alone'],
                                                                                  data['Earnings:Less than 25,000'])
data['Earnings%:Two or More Races Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 Two or More Races'], data['Earnings:Less than 25,000'])
data['Earnings%:Hispanic or Latino Portion of Earners Below 25,000'] = percent(data['Earnings:Less than 25,000 Hispanic or Latino'], data['Earnings:Less than 25,000'])
#25 to 49,999
data['Earnings%:White Alone Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 White Alone'], data['Earnings:25,000 to 49,999'])
data['Earnings%:Black or African American Alone Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 Black or African American Alone'], 
                                                                                            data['Earnings:25,000 to 49,999'])
data['Earnings%:American Indian or Alaska Native Alone Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 American Indian or Alaska Native Alone'], 
                                                                                                   data['Earnings:25,000 to 49,999'])
data['Earnings%:Asian Alone Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 Asian Alone'], data['Earnings:25,000 to 49,999'])
data['Earnings%:Native Hawaiian and Other Pacific Islander Alone Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 Native Hawaiian and Other Pacific Islander Alone'], 
                                                                        data['Earnings:25,000 to 49,999'])
data['Earnings%:Some Other Race Alone Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 Some Other Race Alone'],
                                                                                  data['Earnings:25,000 to 49,999'])
data['Earnings%:Two or More Races Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 Two or More Races'], data['Earnings:25,000 to 49,999'])
data['Earnings%:Hispanic or Latino Portion of Earners 25,000 to 49,999'] = percent(data['Earnings:25,000 to 49,999 Hispanic or Latino'], data['Earnings:25,000 to 49,999'])
#50 to 74,999
data['Earnings%:White Alone Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 White Alone'], data['Earnings:50,000 to 74,999'])
data['Earnings%:Black or African American Alone Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 Black or African American Alone'], 
                                                                                            data['Earnings:50,000 to 74,999'])
data['Earnings%:American Indian or Alaska Native Alone Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 American Indian or Alaska Native Alone'], 
                                                                                                   data['Earnings:50,000 to 74,999'])
data['Earnings%:Asian Alone Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 Asian Alone'], data['Earnings:50,000 to 74,999'])
data['Earnings%:Native Hawaiian and Other Pacific Islander Alone Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 Native Hawaiian and Other Pacific Islander Alone'], 
                                                                        data['Earnings:50,000 to 74,999'])
data['Earnings%:Some Other Race Alone Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 Some Other Race Alone'],
                                                                                  data['Earnings:50,000 to 74,999'])
data['Earnings%:Two or More Races Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 Two or More Races'], data['Earnings:50,000 to 74,999'])
data['Earnings%:Hispanic or Latino Portion of Earners 50,000 to 74,999'] = percent(data['Earnings:50,000 to 74,999 Hispanic or Latino'], data['Earnings:50,000 to 74,999'])
#75 to 99,999
data['Earnings%:White Alone Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 White Alone'], data['Earnings:75,000 to 99,999'])
data['Earnings%:Black or African American Alone Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 Black or African American Alone'], 
                                                                                            data['Earnings:75,000 to 99,999'])
data['Earnings%:American Indian or Alaska Native Alone Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 American Indian or Alaska Native Alone'], 
                                                                                                   data['Earnings:75,000 to 99,999'])
data['Earnings%:Asian Alone Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 Asian Alone'], data['Earnings:75,000 to 99,999'])
data['Earnings%:Native Hawaiian and Other Pacific Islander Alone Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 Native Hawaiian and Other Pacific Islander Alone'], 
                                                                        data['Earnings:75,000 to 99,999'])
data['Earnings%:Some Other Race Alone Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 Some Other Race Alone'],
                                                                                  data['Earnings:75,000 to 99,999'])
data['Earnings%:Two or More Races Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 Two or More Races'], data['Earnings:75,000 to 99,999'])
data['Earnings%:Hispanic or Latino Portion of Earners 75,000 to 99,999'] = percent(data['Earnings:75,000 to 99,999 Hispanic or Latino'], data['Earnings:75,000 to 99,999'])
#7100,000 and Over
data['Earnings%:White Alone Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over White Alone'], data['Earnings:100,000 and Over'])
data['Earnings%:Black or African American Alone Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over Black or African American Alone'], 
                                                                                            data['Earnings:100,000 and Over'])
data['Earnings%:American Indian or Alaska Native Alone Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over American Indian or Alaska Native Alone'], 
                                                                                                   data['Earnings:100,000 and Over'])
data['Earnings%:Asian Alone Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over Asian Alone'], data['Earnings:100,000 and Over'])
data['Earnings%:Native Hawaiian and Other Pacific Islander Alone Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over Native Hawaiian and Other Pacific Islander Alone'], 
                                                                        data['Earnings:100,000 and Over'])
data['Earnings%:Some Other Race Alone Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over Some Other Race Alone'],
                                                                                  data['Earnings:100,000 and Over'])
data['Earnings%:Two or More Races Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over Two or More Races'], data['Earnings:100,000 and Over'])
data['Earnings%:Hispanic or Latino Portion of Earners 100,000 and Over'] = percent(data['Earnings:100,000 and Over Hispanic or Latino'], data['Earnings:100,000 and Over'])

#mortgage status
data['Mortgage:Housing Units with a Mortgage'] = data['mortgage_with']
data['Mortgage%:Housing Units with a Mortgage'] = percent(data['mortgage_with'], data['mortgage_total_series'])
data['Mortgage:Housing Units without a Mortgage'] = data['mortgage_without']
data['Mortgage%:Housing Units without a Mortgage'] = percent(data['mortgage_without'], data['mortgage_total_series'])
#foreign born year of entry
data['Foreign Born:Entered 2010 or Later'] = data['fb_yearentry_2010orlater']
data['Foreign Born%:Entered 2010 or Later'] = percent(data['fb_yearentry_2010orlater'], data['fb_yearentry_total_series'])
data['Foreign Born: Entered 2000 to 2009'] = data['fb_yearentry_2000to2009']
data['Foreign Born%:Entered 2000 to 2009'] = percent(data['fb_yearentry_2000to2009'], data['fb_yearentry_total_series'])
data['Foreign Born: Entered 1990 to 1999'] = data['fb_yearentry_1990to1999']
data['Foreign Born%:Entered 1990 to 1999'] = percent(data['fb_yearentry_1990to1999'], data['fb_yearentry_total_series'])
data['Foreign Born: Entered Before 1990'] = data['fb_yearentry_before1990']
data['Foreign Born%:Entered Before 1990'] = percent(data['fb_yearentry_before1990'], data['fb_yearentry_total_series'])
#Public Assistance/SNAP
data['Public Assistance:Households with Public Assistance/SNAP'] = data['snap_householdswith']
data['Public Assistance%:Households with Public Assistance/SNAP'] = percent(data['snap_householdswith'], data['snap_total_households_series'])
data['Public Assistance:Households without Public Assistance/SNAP'] = data['snap_householdswithout']
data['Public Assistance%:Households without Public Assistance/SNAP'] = percent(data['snap_householdswithout'], data['snap_total_households_series'])
#homeownership by race
#white
data['Homeownership:White Householders'] = data['tenure_total_series_whitealone']
data['Homeownership:White Homeowners'] = data['tenure_owneroccunits_whitealone']
data['Homeownership%:White Householders Homeowner'] = percent(data['tenure_owneroccunits_whitealone'], data['tenure_total_series_whitealone'])
data['Homeownership%:Homeowners White'] = percent(data['tenure_owneroccunits_whitealone'], data['tenure_owneroccunits'])
data['Homeownership:White Renters'] = data['tenure_renteroccunits_whitealone']
data['Homeownership%:White Householders Renters'] = percent(data['tenure_renteroccunits_whitealone'], data['tenure_total_series_whitealone'])
data['Homeownership%:Renters White'] = percent(data['tenure_renteroccunits_whitealone'], data['tenure_renteroccunits'])
#black african american
data['Homeownership:Black or African American Householders'] = data['tenure_total_series_blackafricanamericanalone']
data['Homeownership:Black or African American Homeowners'] = data['tenure_owneroccunits_blackafricanamericanalone']
data['Homeownership%:Black or African American Householders Homeowner'] = percent(data['tenure_owneroccunits_blackafricanamericanalone'], 
                                                                                  data['tenure_total_series_blackafricanamericanalone'])
data['Homeownership%:Homeowners Black or African American'] = percent(data['tenure_owneroccunits_blackafricanamericanalone'], data['tenure_owneroccunits'])
data['Homeownership:Black or African American Renters'] = data['tenure_renteroccunits_blackafricanamericanalone']
data['Homeownership%:Black or African American Householders Renters'] = percent(data['tenure_renteroccunits_blackafricanamericanalone'], 
                                                                                data['tenure_total_series_blackafricanamericanalone'])
data['Homeownership%:Renters Black or African American'] = percent(data['tenure_renteroccunits_blackafricanamericanalone'], data['tenure_renteroccunits'])
#american indian alaska native
data['Homeownership:American Indian Alaska Native Householders'] = data['tenure_total_series_americanindianalaskanativealone']
data['Homeownership:American Indian Alaska Native Homeowners'] = data['tenure_owneroccunits_americanindianalaskanativealone']
data['Homeownership%:American Indian Alaska Native Householders Homeowner'] = percent(data['tenure_owneroccunits_americanindianalaskanativealone'], 
                                                                                      data['tenure_total_series_americanindianalaskanativealone'])
data['Homeownership%:Homeowners American Indian Alaska Native'] = percent(data['tenure_owneroccunits_americanindianalaskanativealone'], data['tenure_owneroccunits'])
data['Homeownership:American Indian Alaska Native Renters'] = data['tenure_renteroccunits_americanindianalaskanativealone']
data['Homeownership%:American Indian Alaska Native Householders Renters'] = percent(data['tenure_renteroccunits_americanindianalaskanativealone'], 
                                                                                    data['tenure_total_series_americanindianalaskanativealone'])
data['Homeownership%:Renters American Indian Alaska Native'] = percent(data['tenure_renteroccunits_americanindianalaskanativealone'], data['tenure_renteroccunits'])
#asian
data['Homeownership:Asian Householders'] = data['tenure_total_series_asianalone']
data['Homeownership:Asian Homeowners'] = data['tenure_owneroccunits_asianalone']
data['Homeownership%:Asian Householders Homeowner'] = percent(data['tenure_owneroccunits_asianalone'], data['tenure_total_series_asianalone'])
data['Homeownership%:Homeowners Asian'] = percent(data['tenure_owneroccunits_asianalone'], data['tenure_owneroccunits'])
data['Homeownership:Asian Renters'] = data['tenure_renteroccunits_asianalone']
data['Homeownership%:Asian Householders Renters'] = percent(data['tenure_renteroccunits_asianalone'], data['tenure_total_series_asianalone'])
data['Homeownership%:Renters Asian'] = percent(data['tenure_renteroccunits_asianalone'], data['tenure_renteroccunits'])
#Native Hawaiian or Other Pacific Islander Alone
data['Homeownership:Native Hawaiian or Other Pacific Islander Alone Householders'] = data['tenure_total_series_nativehawaiianotherpacificislanderalone']
data['Homeownership:Native Hawaiian or Other Pacific Islander Alone Homeowners'] = data['tenure_owneroccunits_nativehawaiianotherpacificislanderalone']
data['Homeownership%:Native Hawaiian or Other Pacific Islander Alone Householders Homeowner'] = percent(data['tenure_owneroccunits_nativehawaiianotherpacificislanderalone'], 
                                                                                                        data['tenure_total_series_nativehawaiianotherpacificislanderalone'])
data['Homeownership%:Homeowners Native Hawaiian or Other Pacific Islander Alone'] = percent(data['tenure_owneroccunits_nativehawaiianotherpacificislanderalone'], 
                                                                                            data['tenure_owneroccunits'])
data['Homeownership:Native Hawaiian or Other Pacific Islander Alone Renters'] = data['tenure_renteroccunits_nativehawaiianotherpacificislanderalone']
data['Homeownership%:Native Hawaiian or Other Pacific Islander Alone Householders Renters'] = percent(data['tenure_renteroccunits_nativehawaiianotherpacificislanderalone'], 
                                                                                                      data['tenure_total_series_nativehawaiianotherpacificislanderalone'])
data['Homeownership%:Renters Native Hawaiian or Other Pacific Islander Alone'] = percent(data['tenure_renteroccunits_nativehawaiianotherpacificislanderalone'], 
                                                                                         data['tenure_renteroccunits'])
#Some Other Race Alone
data['Homeownership:Some Other Race Householders'] = data['tenure_total_series_someotherracealone']
data['Homeownership:Some Other Race Homeowners'] = data['tenure_owneroccunits_someotherracealone']
data['Homeownership%:Some Other Race Householders Homeowner'] = percent(data['tenure_owneroccunits_someotherracealone'], data['tenure_total_series_someotherracealone'])
data['Homeownership%:Homeowners Some Other Race'] = percent(data['tenure_owneroccunits_someotherracealone'], data['tenure_owneroccunits'])
data['Homeownership:Some Other Race Renters'] = data['tenure_renteroccunits_someotherracealone']
data['Homeownership%:Some Other Race Householders Renters'] = percent(data['tenure_renteroccunits_someotherracealone'], data['tenure_total_series_someotherracealone'])
data['Homeownership%:Renters Some Other Race'] = percent(data['tenure_renteroccunits_someotherracealone'], data['tenure_renteroccunits'])
#Two or More Races
data['Homeownership:Two or More Races Householders'] = data['tenure_total_series_twoormoreraces']
data['Homeownership:Two or More Races Homeowners'] = data['tenure_owneroccunits_twoormoreraces']
data['Homeownership%:Two or More Races Householders Homeowner'] = percent(data['tenure_owneroccunits_twoormoreraces'], data['tenure_total_series_twoormoreraces'])
data['Homeownership%:Homeowners Two or More Races'] = percent(data['tenure_owneroccunits_twoormoreraces'], data['tenure_owneroccunits'])
data['Homeownership:Two or More Races Renters'] = data['tenure_renteroccunits_twoormoreraces']
data['Homeownership%:Two or More Races Householders Renters'] = percent(data['tenure_renteroccunits_twoormoreraces'], data['tenure_total_series_twoormoreraces'])
data['Homeownership%:Renters Two or More Races'] = percent(data['tenure_renteroccunits_twoormoreraces'], data['tenure_renteroccunits'])
#Non-Minority
data['Homeownership:Non-Minority Homeowners'] = data['tenure_total_series_whitealonenothispanicorlatino']
data['Homeownership:Non-Minority Homeowners'] = data['tenure_owneroccunits_whitealonenothispanicorlatino']
data['Homeownership%:Non-Minority Householders Homeowner'] = percent(data['tenure_owneroccunits_whitealonenothispanicorlatino'], 
                                                                     data['tenure_total_series_whitealonenothispanicorlatino'])
data['Homeownership%:Homeowners Non-Minority'] = percent(data['tenure_owneroccunits_whitealonenothispanicorlatino'], data['tenure_owneroccunits'])
data['Homeownership:Non-Minority Renters'] = data['tenure_renteroccunits_whitealonenothispanicorlatino']
data['Homeownership%:Non-Minority Householders Renters'] = percent(data['tenure_renteroccunits_whitealonenothispanicorlatino'], 
                                                                   data['tenure_total_series_whitealonenothispanicorlatino'])
data['Homeownership%:Renters Non-Minority'] = percent(data['tenure_renteroccunits_whitealonenothispanicorlatino'], data['tenure_renteroccunits'])
#Hispanic or Latino
data['Homeownership:Hispanic or Latino Householders'] = data['tenure_total_series_hispanicorlatino']
data['Homeownership:Hispanic or Latino Homeowners'] = data['tenure_owneroccunits_hispanicorlatino']
data['Homeownership%:Hispanic or Latino Householders Homeowner'] = percent(data['tenure_owneroccunits_hispanicorlatino'], 
                                                                     data['tenure_total_series_hispanicorlatino'])
data['Homeownership%:Homeowners Hispanic or Latino'] = percent(data['tenure_owneroccunits_hispanicorlatino'], data['tenure_owneroccunits'])
data['Homeownership:Hispanic or Latino Renters'] = data['tenure_renteroccunits_hispanicorlatino']
data['Homeownership%:Hispanic or Latino Householders Renters'] = percent(data['tenure_renteroccunits_hispanicorlatino'], 
                                                                   data['tenure_total_series_hispanicorlatino'])
data['Homeownership%:Renters Hispanic or Latino'] = percent(data['tenure_renteroccunits_hispanicorlatino'], data['tenure_renteroccunits'])
#homeownership by age
#15 to 24
householders15to24 = [data['tenure_ownerocc_15to24'], data['tenure_renterocc_15to24']]
data['Homeownership:Householders Age 15 to 24'] = sum(householders15to24)
data['Homeownership:Homeowners Age 15 to 24'] = data['tenure_ownerocc_15to24']
data['Homeownership%:Homeowners Age 15 to 24'] = percent(data['tenure_ownerocc_15to24'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 15 to 24 Householders Homeowners']  = percent(data['tenure_ownerocc_15to24'], sum(householders15to24))
data['Homeownership:Renters Age 15 to 24'] = data['tenure_renterocc_15to24']
data['Homeownership%:Renters Age 15 to 24'] = percent(data['tenure_renterocc_15to24'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 15 to 24 Householders Renters']  = percent(data['tenure_renterocc_15to24'], sum(householders15to24))
#25 to 34
householders25to34 = [data['tenure_ownerocc_25to34'], data['tenure_renterocc_25to34']]
data['Homeownership:Householders Age 25 to 34'] = sum(householders25to34)
data['Homeownership:Homeowners Age 25 to 34'] = data['tenure_ownerocc_25to34']
data['Homeownership%:Homeowners Age 25 to 34'] = percent(data['tenure_ownerocc_25to34'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 25 to 34 Householders Homeowners']  = percent(data['tenure_ownerocc_25to34'], sum(householders25to34))
data['Homeownership:Renters Age 25 to 34'] = data['tenure_renterocc_25to34']
data['Homeownership%:Renters Age 25 to 34'] = percent(data['tenure_renterocc_25to34'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 25 to 34 Householders Renters']  = percent(data['tenure_renterocc_25to34'], sum(householders25to34))
#35 to 44
householders35to44 = [data['tenure_ownerocc_35to44'], data['tenure_renterocc_35to44']]
data['Homeownership:Householders Age 35 to 44'] = sum(householders35to44)
data['Homeownership:Homeowners Age 35 to 44'] = data['tenure_ownerocc_35to44']
data['Homeownership%:Homeowners Age 35 to 44'] = percent(data['tenure_ownerocc_35to44'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 35 to 44 Householders Homeowners']  = percent(data['tenure_ownerocc_35to44'], sum(householders35to44))
data['Homeownership:Renters Age 35 to 44'] = data['tenure_renterocc_35to44']
data['Homeownership%:Renters Age 35 to 44'] = percent(data['tenure_renterocc_35to44'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 35 to 44 Householders Renters']  = percent(data['tenure_renterocc_35to44'], sum(householders35to44))
#45 to 54
householders45to54 = [data['tenure_ownerocc_45to54'], data['tenure_renterocc_45to54']]
data['Homeownership:Householders Age 45 to 54'] = sum(householders45to54)
data['Homeownership:Homeowners Age 45 to 54'] = data['tenure_ownerocc_45to54']
data['Homeownership%:Homeowners Age 45 to 54'] = percent(data['tenure_ownerocc_45to54'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 35 to 44 Householders Homeowners']  = percent(data['tenure_ownerocc_45to54'], sum(householders45to54))
data['Homeownership:Renters Age 45 to 54'] = data['tenure_renterocc_45to54']
data['Homeownership%:Renters Age 45 to 54'] = percent(data['tenure_renterocc_45to54'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 45 to 54 Householders Renters']  = percent(data['tenure_renterocc_45to54'], sum(householders45to54))
#55 to 59
householders55to59 = [data['tenure_ownerocc_55to59'], data['tenure_renterocc_55to59']]
data['Homeownership:Householders Age 55 to 59'] = sum(householders55to59)
data['Homeownership:Homeowners Age 55 to 59'] = data['tenure_ownerocc_55to59']
data['Homeownership%:Homeowners Age 55 to 59'] = percent(data['tenure_ownerocc_55to59'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 55 to 59 Householders Homeowners']  = percent(data['tenure_ownerocc_55to59'], sum(householders55to59))
data['Homeownership:Renters Age 55 to 59'] = data['tenure_renterocc_55to59']
data['Homeownership%:Renters Age 55 to 59'] = percent(data['tenure_renterocc_55to59'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 55 to 59 Householders Renters']  = percent(data['tenure_renterocc_55to59'], sum(householders55to59))
#60 to 64
thelist = [data['tenure_ownerocc_60to64'], data['tenure_renterocc_60to64']]
totalgroup = sum(thelist)
data['Homeownership:Householders Age 60 to 64'] = totalgroup
data['Homeownership:Homeowners Age 60 to 64'] = data['tenure_ownerocc_60to64']
data['Homeownership%:Homeowners Age 60 to 64'] = percent(data['tenure_ownerocc_60to64'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 60 to 64 Householders Homeowners']  = percent(data['tenure_ownerocc_60to64'], totalgroup)
data['Homeownership:Renters Age 60 to 64'] = data['tenure_renterocc_60to64']
data['Homeownership%:Renters Age 60 to 64'] = percent(data['tenure_renterocc_60to64'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 60 to 64 Householders Renters']  = percent(data['tenure_renterocc_60to64'], totalgroup)
#65 to 74
thelist = [data['tenure_ownerocc_65to74'], data['tenure_renterocc_65to74']]
totalgroup = sum(thelist)
data['Homeownership:Householders Age 65 to 74'] = totalgroup
data['Homeownership:Homeowners Age 65 to 74'] = data['tenure_ownerocc_65to74']
data['Homeownership%:Homeowners Age 65 to 74'] = percent(data['tenure_ownerocc_65to74'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 65 to 74 Householders Homeowners']  = percent(data['tenure_ownerocc_65to74'], totalgroup)
data['Homeownership:Renters Age 65 to 74'] = data['tenure_renterocc_65to74']
data['Homeownership%:Renters Age 65 to 74'] = percent(data['tenure_renterocc_65to74'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 65 to 74 Householders Renters']  = percent(data['tenure_renterocc_65to74'], totalgroup)
#75 to 84
thelist = [data['tenure_ownerocc_75to84'], data['tenure_renterocc_75to84']]
totalgroup = sum(thelist)
data['Homeownership:Householders Age 75 to 84'] = totalgroup
data['Homeownership:Homeowners Age 75 to 84'] = data['tenure_ownerocc_75to84']
data['Homeownership%:Homeowners Age 75 to 84'] = percent(data['tenure_ownerocc_75to84'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 75 to 84 Householders Homeowners']  = percent(data['tenure_ownerocc_75to84'], totalgroup)
data['Homeownership:Renters Age 75 to 84'] = data['tenure_renterocc_75to84']
data['Homeownership%:Renters Age 75 to 84'] = percent(data['tenure_renterocc_75to84'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 75 to 84 Householders Renters']  = percent(data['tenure_renterocc_75to84'], totalgroup)
#85 Over
thelist = [data['tenure_ownerocc_85andover'], data['tenure_renterocc_85andover']]
totalgroup = sum(thelist)
data['Homeownership:Householders Age 85 Over'] = totalgroup
data['Homeownership:Homeowners Age 85 Over'] = data['tenure_ownerocc_85andover']
data['Homeownership%:Homeowners Age 85 Over'] = percent(data['tenure_ownerocc_85andover'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 85 Over Householders Homeowners']  = percent(data['tenure_ownerocc_85andover'], totalgroup)
data['Homeownership:Renters Age 85 Over'] = data['tenure_renterocc_85andover']
data['Homeownership%:Renters Age 85 Over'] = percent(data['tenure_renterocc_85andover'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 85 Over Householders Renters']  = percent(data['tenure_renterocc_85andover'], totalgroup)
#35 to 59
householders35to59 = [data['tenure_ownerocc_35to44'], data['tenure_renterocc_35to44'], 
                      data['tenure_ownerocc_45to54'], data['tenure_renterocc_45to54'],
                      data['tenure_ownerocc_55to59'], data['tenure_renterocc_55to59']]
owners35to59 = [data['tenure_ownerocc_35to44'], data['tenure_ownerocc_45to54'], data['tenure_ownerocc_55to59']]
renters35to59 = [data['tenure_renterocc_35to44'], data['tenure_renterocc_45to54'], data['tenure_renterocc_55to59']]
data['Homeownership:Householders Age 35 to 59'] = sum(householders35to59)
data['Homeownership:Homeowners Age 35 to 59'] = sum(owners35to59)
data['Homeownership%:Homeowners Age 35 to 59'] = percent(data['Homeownership:Homeowners Age 35 to 59'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 35 to 59 Householders Homeowners']  = percent(data['Homeownership:Homeowners Age 35 to 59'], sum(householders35to59))
data['Homeownership:Renters Age 35 to 59'] = sum(renters35to59)
data['Homeownership%:Renters Age 35 to 59'] = percent(data['Homeownership:Renters Age 35 to 59'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 35 to 59 Householders Renters']  = percent(data['Homeownership:Renters Age 35 to 59'], sum(householders35to59))
#55 and Older
householders55over = [data['tenure_ownerocc_55to59'], data['tenure_renterocc_55to59'], data['tenure_ownerocc_60to64'], data['tenure_renterocc_60to64'], 
                      data['tenure_ownerocc_65to74'], data['tenure_renterocc_65to74'], 
                      data['tenure_ownerocc_75to84'],  data['tenure_renterocc_75to84'],
                      data['tenure_ownerocc_85andover'], data['tenure_renterocc_85andover']]
owners55over = [data['tenure_ownerocc_55to59'], data['tenure_ownerocc_60to64'], data['tenure_ownerocc_65to74'], 
                data['tenure_ownerocc_75to84'], data['tenure_ownerocc_85andover']]
renters55over = [data['tenure_renterocc_55to59'], data['tenure_renterocc_60to64'], data['tenure_renterocc_65to74'], 
                 data['tenure_renterocc_75to84'], data['tenure_renterocc_85andover']]
data['Homeownership:Householders Age 55 and Over'] = sum(householders55over)
data['Homeownership:Homeowners Age 55 and Over'] = sum(owners55over)
data['Homeownership%:Homeowners Age 55 and Over'] = percent(data['Homeownership:Homeowners Age 55 and Over'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 55 and Over Householders Homeowners']  = percent(data['Homeownership:Homeowners Age 55 and Over'], sum(householders55over))
data['Homeownership:Renters Age 55 and Over'] = sum(renters55over)
data['Homeownership%:Renters Age 55 and Over'] = percent(data['Homeownership:Renters Age 55 and Over'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 55 and Over Householders Renters']  = percent(data['Homeownership:Renters Age 55 and Over'], sum(householders55over))
#60 and Older
householders60over = [data['tenure_ownerocc_60to64'], data['tenure_renterocc_60to64'], 
                      data['tenure_ownerocc_65to74'], data['tenure_renterocc_65to74'],
                      data['tenure_ownerocc_75to84'], data['tenure_renterocc_75to84'],
                      data['tenure_ownerocc_85andover'], data['tenure_renterocc_85andover']]
owners60over = [data['tenure_ownerocc_60to64'], data['tenure_ownerocc_65to74'], data['tenure_ownerocc_75to84'], data['tenure_ownerocc_85andover']]
renters60over = [data['tenure_renterocc_60to64'], data['tenure_renterocc_65to74'], data['tenure_renterocc_75to84'], data['tenure_renterocc_85andover']]
data['Homeownership:Householders Age 60 and Over'] = sum(householders60over)
data['Homeownership:Homeowners Age 60 and Over'] = sum(owners60over)
data['Homeownership%:Homeowners Age 60 and Over'] = percent(data['Homeownership:Homeowners Age 60 and Over'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 60 and Over Householders Homeowners']  = percent(data['Homeownership:Homeowners Age 60 and Over'], sum(householders60over))
data['Homeownership:Renters Age 60 and Over'] = sum(renters60over)
data['Homeownership%:Renters Age 60 and Over'] = percent(data['Homeownership:Renters Age 60 and Over'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 60 and Over Householders Renters']  = percent(data['Homeownership:Renters Age 60 and Over'], sum(householders60over))
#65 and Older
householders65over = [data['tenure_ownerocc_65to74'], data['tenure_renterocc_65to74'],
                      data['tenure_ownerocc_75to84'], data['tenure_renterocc_75to84'],
                      data['tenure_ownerocc_85andover'], data['tenure_renterocc_85andover']]
owners65over = [data['tenure_ownerocc_65to74'], data['tenure_ownerocc_75to84'], data['tenure_ownerocc_85andover']]
renters65over = [data['tenure_renterocc_65to74'], data['tenure_renterocc_75to84'], data['tenure_renterocc_85andover']]
data['Homeownership:Householders Age 65 and Over'] = sum(householders65over)
data['Homeownership:Homeowners Age 65 and Over'] = sum(owners65over)
data['Homeownership%:Homeowners Age 65 and Over'] = percent(data['Homeownership:Homeowners Age 65 and Over'], data['tenure_age_total_ownerocc'])
data['Homeownership%:Age 65 and Over Householders Homeowners']  = percent(data['Homeownership:Homeowners Age 65 and Over'], sum(householders65over))
data['Homeownership:Renters Age 65 and Over'] = sum(renters65over)
data['Homeownership%:Renters Age 65 and Over'] = percent(data['Homeownership:Renters Age 65 and Over'], data['tenure_age_total_renterocc'])
data['Homeownership%:Age 65 and Over Householders Renters']  = percent(data['Homeownership:Renters Age 65 and Over'], sum(householders65over))
#poverty by age
belowlevel = data['poverty_belowlevel']
#Under 5
allunder5 = [data['poverty_belowlevel_f_u5'], data['poverty_belowlevel_m_u5'], 
             data['poverty_atorabovelevel_f_u5'], data['poverty_atorabovelevel_m_u5']]
data['Poverty:Below Poverty Level Under 5'] = data['poverty_belowlevel_f_u5'] + data['poverty_belowlevel_m_u5']
data['Poverty%:Below Poverty Level Under 5'] = percent(data['Poverty:Below Poverty Level Under 5'], belowlevel)
data['Poverty%:Under 5 Below Poverty Level'] = percent(data['Poverty:Below Poverty Level Under 5'], sum(allunder5))
#under 18
#under 18
povertyunder18y = [data['poverty_belowlevel_f_u5'], data['poverty_belowlevel_m_u5'],
                   data['poverty_belowlevel_m_5'], data['poverty_belowlevel_f_5'],
                   data['poverty_belowlevel_f_6to11'], data['poverty_belowlevel_f_12to14'], data['poverty_belowlevel_f_15'], 
                   data['poverty_belowlevel_m_6to11'], data['poverty_belowlevel_m_12to14'], data['poverty_belowlevel_m_15'], 
                   data['poverty_belowlevel_m_16to17'], data['poverty_belowlevel_f_16to17']]
allundereighteen = [data['poverty_belowlevel_f_u5'], data['poverty_belowlevel_m_u5'],data['poverty_atorabovelevel_f_u5'], data['poverty_atorabovelevel_m_u5'],
                    data['poverty_belowlevel_m_5'], data['poverty_belowlevel_f_5'], data['poverty_atorabovelevel_m_5'], data['poverty_atorabovelevel_f_5'],
                    data['poverty_belowlevel_f_6to11'], data['poverty_belowlevel_f_12to14'], data['poverty_belowlevel_f_15'],
                    data['poverty_belowlevel_m_6to11'], data['poverty_belowlevel_m_12to14'], data['poverty_belowlevel_m_15'], 
                    data['poverty_atorabovelevel_f_6to11'], data['poverty_atorabovelevel_f_12to14'], data['poverty_atorabovelevel_f_15'], 
                    data['poverty_atorabovelevel_m_6to11'], data['poverty_atorabovelevel_m_12to14'], data['poverty_atorabovelevel_m_15'], 
                    data['poverty_atorabovelevel_m_16to17'], data['poverty_atorabovelevel_f_16to17'], 
                    data['poverty_belowlevel_m_16to17'], data['poverty_belowlevel_f_16to17']]
data['Poverty:Below Poverty Level Under 18'] = sum(povertyunder18y)
data['Poverty%:Below Poverty Level Under 18'] = percent(data['Poverty:Below Poverty Level Under 18'], belowlevel)
data['Poverty%:Under 18 Below Poverty Level'] = percent(data['Poverty:Below Poverty Level Under 18'], sum(allundereighteen))
#18 to 64
poverty18to64 = [data['poverty_belowlevel_m_18to24'], data['poverty_belowlevel_f_18to24'], 
                 data['poverty_belowlevel_m_25to34'], data['poverty_belowlevel_f_25to34'],
                 data['poverty_belowlevel_m_35to44'], data['poverty_belowlevel_f_35to44'], 
                 data['poverty_belowlevel_m_45to54'], data['poverty_belowlevel_f_45to54'],
                 data['poverty_belowlevel_m_55to64'], data['poverty_belowlevel_f_55to64']]
all18to64 = [data['poverty_belowlevel_m_18to24'], data['poverty_belowlevel_f_18to24'], 
             data['poverty_belowlevel_m_25to34'], data['poverty_belowlevel_f_25to34'],
             data['poverty_belowlevel_m_35to44'], data['poverty_belowlevel_f_35to44'], 
             data['poverty_belowlevel_m_45to54'], data['poverty_belowlevel_f_45to54'],
             data['poverty_belowlevel_m_55to64'], data['poverty_belowlevel_f_55to64'], 
             data['poverty_atorabovelevel_m_18to24'], data['poverty_atorabovelevel_f_18to24'], 
             data['poverty_atorabovelevel_m_25to34'], data['poverty_atorabovelevel_f_25to34'],
             data['poverty_atorabovelevel_m_35to44'], data['poverty_atorabovelevel_f_35to44'], 
             data['poverty_atorabovelevel_m_45to54'], data['poverty_atorabovelevel_f_45to54'],
             data['poverty_atorabovelevel_m_55to64'], data['poverty_atorabovelevel_f_55to64']]
data['Poverty:Below Poverty Level 18 to 64'] = sum(poverty18to64)
data['Poverty%:Below Poverty Level 18 to 64'] = percent(data['Poverty:Below Poverty Level 18 to 64'], belowlevel)
data['Poverty%:18 to 64 Below Poverty Level'] = percent(data['Poverty:Below Poverty Level 18 to 64'], sum(all18to64))
#65 and over
poverty65over = [data['poverty_belowlevel_m_65to74'], data['poverty_belowlevel_f_65to74'], 
                 data['poverty_belowlevel_m_75andover'], data['poverty_belowlevel_f_75andover']]
all65andover = [data['poverty_belowlevel_m_65to74'], data['poverty_belowlevel_f_65to74'], 
                data['poverty_belowlevel_m_75andover'], data['poverty_belowlevel_f_75andover'], 
                data['poverty_atorabovelevel_m_65to74'], data['poverty_atorabovelevel_f_65to74'], 
                data['poverty_atorabovelevel_m_75andover'], data['poverty_atorabovelevel_f_75andover']]
data['Poverty:Below Poverty Level 65 and Over'] = sum(poverty65over)
data['Poverty%:Below Poverty Level 65 and Over'] = percent(data['Poverty:Below Poverty Level 65 and Over'], belowlevel)
data['Poverty%:65 and Over Below Poverty Level'] = percent(data['Poverty:Below Poverty Level 65 and Over'], sum(all65andover))
#POVERTY BY RACE
belowaggregate = [data['poverty_belowlevel_whitealone'], data['poverty_belowlevel_blackafricanamericanalone'], 
                  data['poverty_belowlevel_americanindianalaskanativealone'], data['poverty_belowlevel_asianalone'], 
                  data['poverty_belowlevel_nativehawaiianotherpacificislanderalone'], 
                  data['poverty_belowlevel_someotherracealone'], data['poverty_belowlevel_twoormoreraces']]
belowlevel = sum(belowaggregate)
#White Alone
data['Poverty:Below Poverty Level White Alone'] = data['poverty_belowlevel_whitealone']
data['Poverty%:Below Poverty Level White Alone'] = percent(data['Poverty:Below Poverty Level White Alone'], belowlevel)
data['Poverty%:White Alone Below Poverty Level'] = percent(data['poverty_belowlevel_whitealone'], data['poverty_total_bysexbyage_whitealone_series'])
#Black or African American Alone
data['Poverty:Below Poverty Level Black or African American Alone'] = data['poverty_belowlevel_blackafricanamericanalone']
data['Poverty%:Below Poverty Level Black or African American Alone'] = percent(data['Poverty:Below Poverty Level Black or African American Alone'], belowlevel)
data['Poverty%:Black or African American Alone Below Poverty Level'] = percent(data['poverty_belowlevel_blackafricanamericanalone'], 
                                                                               data['poverty_total_bysexbyage_blackafricanamericanalone_series'])
#American Indian and Alaska Native Alone
data['Poverty:Below Poverty Level American Indian Alaska Native Alone'] = data['poverty_belowlevel_americanindianalaskanativealone']
data['Poverty%:Below Poverty Level American Indian Alaska Native Alone'] = percent(data['Poverty:Below Poverty Level American Indian Alaska Native Alone'],
                                                                                   belowlevel)
data['Poverty%:American Indian Alaska Native Alone Below Poverty Level'] = percent(data['poverty_belowlevel_americanindianalaskanativealone'], 
                                                                                   data['poverty_total_bysexbyage_americanindianalaskanativealone_series'])
#Asian Alone
data['Poverty:Below Poverty Level Asian Alone'] = data['poverty_belowlevel_asianalone']
data['Poverty%:Below Poverty Level Asian Alone'] = percent(data['Poverty:Below Poverty Level Asian Alone'], belowlevel)
data['Poverty%:Asian Alone Below Poverty Level'] = percent(data['poverty_belowlevel_asianalone'], data['poverty_total_bysexbyage_asianalone_series'])
#Native Hawaiian and Other Pacific Islander Alone
data['Poverty:Below Poverty Level Native Hawaiian Other Pacific Islander Alone'] = data['poverty_belowlevel_nativehawaiianotherpacificislanderalone']
data['Poverty%:Below Poverty Level Native Hawaiian Other Pacific Islander Alone'] = percent(data['Poverty:Below Poverty Level Native Hawaiian Other Pacific Islander Alone'], belowlevel)
data['Poverty%:Native Hawaiian Other Pacific Islander Alone Below Poverty Level'] = percent(data['poverty_belowlevel_nativehawaiianotherpacificislanderalone'],
                                                                                            data['poverty_total_bysexbyage_nativehawaiianotherpacificislanderalone_series'])
#Some Other Race Alone
data['Poverty:Below Poverty Level Some Other Race Alone'] = data['poverty_belowlevel_someotherracealone']
data['Poverty%:Below Poverty Level Some Other Race Alone'] = percent(data['Poverty:Below Poverty Level Some Other Race Alone'], belowlevel)
data['Poverty%:Some Other Race Alone Below Poverty Level'] = percent(data['poverty_belowlevel_someotherracealone'], 
                                                                     data['poverty_total_bysexbyage_someotherracealone_series'])
#Two or More Races
data['Poverty:Below Poverty Level Two or More Races'] = data['poverty_belowlevel_twoormoreraces']
data['Poverty%:Below Poverty Level Two or More Races'] = percent(data['Poverty:Below Poverty Level Two or More Races'], belowlevel)
data['Poverty%:Two or More Races Below Poverty Level'] = percent(data['poverty_belowlevel_twoormoreraces'], data['poverty_total_bysexbyage_twoormoreraces_series'])
#Non-Minority
data['Poverty:Below Poverty Level Non-Minority'] = data['poverty_belowlevel_whitealonenothispanicorlatino']
data['Poverty%:Below Poverty Level Non-Minority'] = percent(data['Poverty:Below Poverty Level Non-Minority'], belowlevel)
data['Poverty%:Non-Minority Below Poverty Level'] = percent(data['poverty_belowlevel_whitealonenothispanicorlatino'], 
                                                            data['poverty_total_bysexbyage_whitealonenothispanicorlatino_series'])
#Hispanic or Latino
data['Poverty:Below Poverty Level Hispanic or Latino'] = data['poverty_belowlevel_hispanicorlatino']
data['Poverty%:Below Poverty Level Hispanic or Latino'] = percent(data['Poverty:Below Poverty Level Hispanic or Latino'], belowlevel)
data['Poverty%:Hispanic or Latino Below Poverty Level'] = percent(data['poverty_belowlevel_hispanicorlatino'], data['poverty_total_bysexbyage_hispanicorlatino_series'])       
#Race and Ethnicity Senior Population
thelist = [data['raceeth_white_m_55to64'], data['raceeth_white_m_65to74'], data['raceeth_white_m_75to84'], data['raceeth_white_m_85+'], 
           data['raceeth_white_f_55to64'], data['raceeth_white_f_65to74'], data['raceeth_white_f_75to84'], data['raceeth_white_f_85+'], 
           data['raceeth_blackafricanamerican_m_55to64'], data['raceeth_blackafricanamerican_m_65to74'], data['raceeth_blackafricanamerican_m_75to84'],
           data['raceeth_blackafricanamerican_m_85+'], data['raceeth_blackafricanamerican_f_55to64'], data['raceeth_blackafricanamerican_f_65to74'], 
           data['raceeth_blackafricanamerican_f_75to84'], data['raceeth_blackafricanamerican_f_85+'], 
           data['raceeth_americanindianalaskanative_m_55to64'], data['raceeth_americanindianalaskanative_m_65to74'], data['raceeth_americanindianalaskanative_m_75to84'], 
           data['raceeth_americanindianalaskanative_m_85+'], data['raceeth_americanindianalaskanative_f_55to64'], 
           data['raceeth_americanindianalaskanative_f_65to74'], data['raceeth_americanindianalaskanative_f_75to84'], 
           data['raceeth_americanindianalaskanative_f_85+'], 
           data['raceeth_asian_m_55to64'], data['raceeth_asian_m_65to74'], data['raceeth_asian_m_75to84'], data['raceeth_asian_m_85+'], 
           data['raceeth_asian_f_55to64'], data['raceeth_asian_f_65to74'], data['raceeth_asian_f_75to84'], data['raceeth_asian_f_85+'], 
           data['raceeth_nativehawaiianpacificislander_m_55to64'], data['raceeth_nativehawaiianpacificislander_m_65to74'], 
           data['raceeth_nativehawaiianpacificislander_m_75to84'], data['raceeth_nativehawaiianpacificislander_m_85+'], 
           data['raceeth_nativehawaiianpacificislander_f_55to64'], data['raceeth_nativehawaiianpacificislander_f_65to74'], 
           data['raceeth_nativehawaiianpacificislander_f_75to84'], data['raceeth_nativehawaiianpacificislander_f_85+'], 
           data['raceeth_someotherrace_m_55to64'], data['raceeth_someotherrace_m_65to74'], data['raceeth_someotherrace_m_75to84'], data['raceeth_someotherrace_m_85+'], 
           data['raceeth_someotherrace_f_55to64'], data['raceeth_someotherrace_f_65to74'], data['raceeth_someotherrace_f_75to84'], data['raceeth_someotherrace_f_85+'], 
           data['raceeth_twoormoreraces_m_55to64'], data['raceeth_twoormoreraces_m_65to74'], data['raceeth_twoormoreraces_m_75to84'], 
           data['raceeth_twoormoreraces_m_85+'],  data['raceeth_twoormoreraces_f_55to64'], data['raceeth_twoormoreraces_f_65to74'], 
           data['raceeth_twoormoreraces_f_75to84'], data['raceeth_twoormoreraces_f_85+']]
all55over = sum(thelist)

thelist = [data['raceeth_white_m_55to64'], data['raceeth_white_m_65to74'], data['raceeth_white_m_75to84'], data['raceeth_white_m_85+'], 
           data['raceeth_white_f_55to64'], data['raceeth_white_f_65to74'], data['raceeth_white_f_75to84'], data['raceeth_white_f_85+']]
data['RaceEth:White Alone 55 and Over'] = sum(thelist)
data['RaceEth%:White Alone that are 55 and Over'] = percent(data['RaceEth:White Alone 55 and Over'], whitealone)
data['RaceEth%:55 and Over that are White Alone'] = percent(data['RaceEth:White Alone 55 and Over'], all55over)
thelist = [data['raceeth_blackafricanamerican_m_55to64'], data['raceeth_blackafricanamerican_m_65to74'], 
           data['raceeth_blackafricanamerican_m_75to84'], data['raceeth_blackafricanamerican_m_85+'], 
           data['raceeth_blackafricanamerican_f_55to64'], data['raceeth_blackafricanamerican_f_65to74'], 
           data['raceeth_blackafricanamerican_f_75to84'], data['raceeth_blackafricanamerican_f_85+']]
data['RaceEth:Black or African American Alone 55 and Over'] = sum(thelist)
data['RaceEth%:Black or African American Alone that are 55 and Over'] = percent(data['RaceEth:Black or African American Alone 55 and Over'], 
                                                                                blackafricanamericanalone)
data['RaceEth%:55 and Over that are Black or African American Alone'] = percent(data['RaceEth:Black or African American Alone 55 and Over'], all55over)
thelist = [data['raceeth_americanindianalaskanative_m_55to64'], data['raceeth_americanindianalaskanative_m_65to74'], 
           data['raceeth_americanindianalaskanative_m_75to84'], data['raceeth_americanindianalaskanative_m_85+'], 
           data['raceeth_americanindianalaskanative_f_55to64'], data['raceeth_americanindianalaskanative_f_65to74'], 
           data['raceeth_americanindianalaskanative_f_75to84'], data['raceeth_americanindianalaskanative_f_85+']]
data['RaceEth:American Indian Alaska Native Alone 55 and Over'] = sum(thelist)
data['RaceEth%:American Indian Alaska Native Alone that are 55 and Over'] = percent(data['RaceEth:American Indian Alaska Native Alone 55 and Over'], 
                                                                                    americanindianalaskanativealone)
data['RaceEth%:55 and Over that are American Indian Alaska Native Alone'] = percent(data['RaceEth:American Indian Alaska Native Alone 55 and Over'], all55over)
thelist = [data['raceeth_asian_m_55to64'], data['raceeth_asian_m_65to74'], data['raceeth_asian_m_75to84'], data['raceeth_asian_m_85+'], 
           data['raceeth_asian_f_55to64'], data['raceeth_asian_f_65to74'], data['raceeth_asian_f_75to84'], data['raceeth_asian_f_85+']]
data['RaceEth:Asian Alone 55 and Over'] = sum(thelist)
data['RaceEth%:Asian Alone that are 55 and Over'] = percent(data['RaceEth:Asian Alone 55 and Over'], asianalone)
data['RaceEth%:55 and Over that are Asian Alone'] = percent(data['RaceEth:Asian Alone 55 and Over'], all55over)
thelist = [data['raceeth_nativehawaiianpacificislander_m_55to64'], data['raceeth_nativehawaiianpacificislander_m_65to74'], 
           data['raceeth_nativehawaiianpacificislander_m_75to84'], data['raceeth_nativehawaiianpacificislander_m_85+'], 
           data['raceeth_nativehawaiianpacificislander_f_55to64'], data['raceeth_nativehawaiianpacificislander_f_65to74'], 
           data['raceeth_nativehawaiianpacificislander_f_75to84'], data['raceeth_nativehawaiianpacificislander_f_85+']]
data['RaceEth:Native Hawaiian Pacific Islander Alone 55 and Over'] = sum(thelist)
data['RaceEth%:Native Hawaiian Pacific Islander Alone that are 55 and Over'] = percent(data['RaceEth:Native Hawaiian Pacific Islander Alone 55 and Over'], 
                                                                                       nativehawaiianotherpacificislanderalone)
data['RaceEth%:55 and Over that are Native Hawaiian Pacific Islander Alone'] = percent(data['RaceEth:Native Hawaiian Pacific Islander Alone 55 and Over'], all55over)
thelist = [data['raceeth_someotherrace_m_55to64'], data['raceeth_someotherrace_m_65to74'], data['raceeth_someotherrace_m_75to84'], data['raceeth_someotherrace_m_85+'], 
           data['raceeth_someotherrace_f_55to64'], data['raceeth_someotherrace_f_65to74'], data['raceeth_someotherrace_f_75to84'], data['raceeth_someotherrace_f_85+']]
data['RaceEth:Some Other Race Alone 55 and Over'] = sum(thelist)
data['RaceEth%:Some Other Race Alone that are 55 and Over'] = percent(data['RaceEth:Some Other Race Alone 55 and Over'], someotherracealone)
data['RaceEth%:55 and Over that are Some Other Race Alone'] = percent(data['RaceEth:Some Other Race Alone 55 and Over'], all55over)
thelist = [data['raceeth_twoormoreraces_m_55to64'], data['raceeth_twoormoreraces_m_65to74'], 
           data['raceeth_twoormoreraces_m_75to84'], data['raceeth_twoormoreraces_m_85+'], 
           data['raceeth_twoormoreraces_f_55to64'], data['raceeth_twoormoreraces_f_65to74'], 
           data['raceeth_twoormoreraces_f_75to84'], data['raceeth_twoormoreraces_f_85+']]
data['RaceEth:Two or More Races 55 and Over'] = sum(thelist)
data['RaceEth%:Two or More Races that are 55 and Over'] = percent(data['RaceEth:Two or More Races 55 and Over'], twoormoreraces)
data['RaceEth%:55 and Over that are Two or More Races'] = percent(data['RaceEth:Two or More Races 55 and Over'], all55over)
thelist = [data['raceeth_whitenothispaniclatino_m_55to64'], data['raceeth_whitenothispaniclatino_m_65to74'], 
           data['raceeth_whitenothispaniclatino_m_75to84'], data['raceeth_whitenothispaniclatino_m_85+'], 
           data['raceeth_whitenothispaniclatino_f_55to64'], data['raceeth_whitenothispaniclatino_f_65to74'], 
           data['raceeth_whitenothispaniclatino_f_75to84'], data['raceeth_whitenothispaniclatino_f_85+']]
data['RaceEth:Non-Minority 55 and Over'] = sum(thelist)
data['RaceEth%:Non-Minority that are 55 and Over'] = percent(data['RaceEth:Non-Minority 55 and Over'], nonminority)
data['RaceEth%:55 and Over that are Non-Minority'] = percent(data['RaceEth:Non-Minority 55 and Over'], all55over)
thelist = [data['raceeth_hispanicorlatino_m_55to64'], data['raceeth_hispanicorlatino_m_65to74'], 
           data['raceeth_hispanicorlatino_m_75to84'], data['raceeth_hispanicorlatino_m_85+'], 
           data['raceeth_hispanicorlatino_f_55to64'], data['raceeth_hispanicorlatino_f_65to74'], 
           data['raceeth_hispanicorlatino_f_75to84'], data['raceeth_hispanicorlatino_f_85+']]
data['RaceEth:Hispanic or Latino 55 and Over'] = sum(thelist)
data['RaceEth%:Hispanic or Latino that are 55 and Over'] = percent(data['RaceEth:Hispanic or Latino 55 and Over'], hispanicorlatino)
data['RaceEth%:55 and Over that are Hispanic or Latino'] = percent(data['RaceEth:Hispanic or Latino 55 and Over'], all55over)

data['RaceEth:Minority 55 and Over'] = all55over - data['RaceEth:Non-Minority 55 and Over']
data['RaceEth%:Minority that are 55 and Over'] = percent(data['RaceEth:Minority 55 and Over'], nonminority)
data['RaceEth%:55 and Over that are Minority'] = percent(data['RaceEth:Minority 55 and Over'], all55over)
#senior living situation
over65 = data['hhtype65over_total_series']
thelist = [data['hhtype65over_inhh_nonfamilyhh_householder_m_livingalone'], data['hhtype65over_inhh_nonfamilyhh_householder_f_livingalone']]
data['LivingSituation:Living Alone 65 Over'] = sum(thelist)
data['LivingSituation%:65 Over that are Living Alone'] = percent(data['LivingSituation:Living Alone 65 Over'], over65)
data['LivingSituation:Living with Spouse 65 Over'] = data['hhtype65over_inhh_familyhh_householder_spouse']
data['LivingSituation%:65 Over that are Living with Spouse'] = percent(data['LivingSituation:Living with Spouse 65 Over'], over65)
#social security
data['SocialSecurity:Households with Social Security Income'] = data['socialsecurityhh_with']
data['SocialSecurity%:Households with Social Security Income'] = percent(data['SocialSecurity:Households with Social Security Income'], 
                                                                         data['socialsecurityhh_total_series'])
#grandparent survey
data['Grandparents:Grandparents that are Primary Caregiver for Grandchild'] = data['grandparent_responsibleforgrandchildu18']
data['Grandparents%:Grandparents that are Primary Caregiver for Grandchild'] = percent(data['Grandparents:Grandparents that are Primary Caregiver for Grandchild'], 
                                                                                      data['grandparent_total_series'])
thelist = [data['grandparent_responsibleforgrandchildu18_hholderorspousenoparentpresent_60over'], 
           data['grandparent_responsibleforgrandchildu18_othergrandparents_60over'], data['grandparent_notresponsibleforgrandchildu18_60over']]
data['Grandparents:Grandparents that are 60 Over'] = sum(thelist)
data['Grandparents%:Grandparents that are 60 Over'] = percent(data['Grandparents:Grandparents that are 60 Over'], data['grandparent_total_series'])
data['Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild'] = data['grandparent_responsibleforgrandchildu18_hholderorspousenoparentpresent_60over']
data['Grandparents%:Grandparents 60 Over that are Primary Caregiver for Grandchild'] = percent(data['Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild'], 
                                                                                              data['Grandparents:Grandparents that are 60 Over'])
#cost burden by age severe not available
thelist = [data['housingcostage_grossrent%hhincome_hholder65over'], data['housingcostage_selectedownercosts%hhincome_hholder65over']]
householdersover65 = sum(thelist)
thelist = [data['housingcostage_grossrent%hhincome_hholder65over_30to34.9'],data['housingcostage_grossrent%hhincome_hholder65over_35ormore'],
           data['housingcostage_selectedownercosts%hhincome_hholder65over_30to34.9'],data['housingcostage_selectedownercosts%hhincome_hholder65over_35ormore']]
data['CBAge:Cost Burdened Households Householder 65 Over'] = sum(thelist)
data['CBAge%:Households Cost Burdened Householder 65 Over'] = percent(data['CBAge:Cost Burdened Households Householder 65 Over'], householdersover65)
renters65over = data['housingcostage_grossrent%hhincome_hholder65over']
thelist = [data['housingcostage_grossrent%hhincome_hholder65over_30to34.9'],data['housingcostage_grossrent%hhincome_hholder65over_35ormore']]
data['CBAge:Cost Burdened Renters Householder 65 Over'] = sum(thelist)
data['CBAge%:Renters Cost Burdened Householder 65 Over'] = percent(data['CBAge:Cost Burdened Renters Householder 65 Over'], renters65over)
homeowners65over = data['housingcostage_selectedownercosts%hhincome_hholder65over']
thelist = [data['housingcostage_selectedownercosts%hhincome_hholder65over_30to34.9'],data['housingcostage_selectedownercosts%hhincome_hholder65over_35ormore']]
data['CBAge:Cost Burdened Homeowners Householder 65 Over'] = sum(thelist)
data['CBAge%:Homeowners Cost Burdened Householder 65 Over'] = percent(data['CBAge:Cost Burdened Homeowners Householder 65 Over'], homeowners65over)
#foreign born by age
thelist = [data['fb_sexbyage_m_55to59'], data['fb_sexbyage_m_60to64'], data['fb_sexbyage_m_65to69'], data['fb_sexbyage_m_70to74'], 
           data['fb_sexbyage_m_75to79'], data['fb_sexbyage_m_80to84'], data['fb_sexbyage_m_85over'], 
           data['fb_sexbyage_f_55to59'], data['fb_sexbyage_f_60to64'], data['fb_sexbyage_f_65to69'], data['fb_sexbyage_f_70to74'], 
           data['fb_sexbyage_f_75to79'], data['fb_sexbyage_f_80to84'], data['fb_sexbyage_f_85over']]
data['ForeignBornAge:Foreign Born Population 55 Over'] = sum(thelist)
data['ForeignBornAge%:Foreign Born Population that is 55 Over'] = percent(data['ForeignBornAge:Foreign Born Population 55 Over'], 
                                                                          data['fb_sexbyage_total_series'])
data['ForeignBornAge%:55 Over that is Foreign Born'] = percent(data['ForeignBornAge:Foreign Born Population 55 Over'], data['Age:55 and Older'])
thelist = [data['fb_sexbyage_m_65to69'], data['fb_sexbyage_m_70to74'], 
           data['fb_sexbyage_m_75to79'], data['fb_sexbyage_m_80to84'], data['fb_sexbyage_m_85over'], 
           data['fb_sexbyage_f_65to69'], data['fb_sexbyage_f_70to74'], 
           data['fb_sexbyage_f_75to79'], data['fb_sexbyage_f_80to84'], data['fb_sexbyage_f_85over']]
data['ForeignBornAge:Foreign Born Population 65 Over'] = sum(thelist)
data['ForeignBornAge%:Foreign Born Population that is 65 Over'] = percent(data['ForeignBornAge:Foreign Born Population 65 Over'], 
                                                                          data['fb_sexbyage_total_series'])
data['ForeignBornAge%:65 Over that is Foreign Born'] = percent(data['ForeignBornAge:Foreign Born Population 65 Over'], data['Age:65 and Older'])
#seniors language spoken at home
thelist = [data['language_speakspanish_65+'], data['language_speakotherindoeuro_65+'], data['language_speakasianpacificisland_65+'], data['language_speakother_65+']]
data['Language65Over:Language Besides English Spoken at Home'] = sum(thelist)
data['Language65Over%:Language Besides English Spoken at Home'] = percent(data['Language65Over:Language Besides English Spoken at Home'],
                                                                            data['language_total_65+'])
thelist = [data['language_speakspanish_speakenglishnotwell_65+'], data['language_speakspanish_speakenglishnotatall_65+'],
           data['language_speakotherindoeuro_speakenglishnotwell_65+'], data['language_speakotherindoeuro_speakenglishnotatall_65+'], 
           data['language_speakasianpacificisland_speakenglishnotwell_65+'], data['language_speakasianpacificisland_speakenglishnotatall_65+'], 
           data['language_speakother_speakenglishnotwell_65+'], data['language_speakother_speakenglishnotatall_65+']]
data['Language65Over:Language Besides English:Speak English Not Well or At All'] = sum(thelist)
data['Language65Over%:Language Besides English:Speak English Not Well or At All'] = percent(data['Language65Over:Language Besides English:Speak English Not Well or At All'], 
                                                                                           data['Language65Over:Language Besides English Spoken at Home'])
#computer and internet subscriptions
thelist = [data['compinternet_hascomputer_u18'], data['compinternet_hascomputer_18to64'], data['compinternet_hascomputer_65over']]
data['Internet:Has a Computer'] = sum(thelist)
data['Internet%:Has a Computer'] = percent(data['Internet:Has a Computer'], data['compinternet_series_total'])
data['Internet:Has a Computer Under 18'] = data['compinternet_hascomputer_u18']
data['Internet%:Under 18 has a Computer'] = percent(data['Internet:Has a Computer Under 18'], data['compinternet_total_u18'])
data['Internet:Has a Computer 18 to 64'] = data['compinternet_hascomputer_18to64']
data['Internet%:18 to 64 has a Computer'] = percent(data['Internet:Has a Computer 18 to 64'], data['compinternet_total_18to64'])
data['Internet:Has a Computer 65 Over'] = data['compinternet_hascomputer_65over']
data['Internet%:65 Over has a Computer'] = percent(data['Internet:Has a Computer 65 Over'], data['compinternet_total_65over'])
thelist = [data['compinternet_hascomputer_dialupinternetonly_u18'], data['compinternet_hascomputer_dialupinternetonly_18to64'], 
           data['compinternet_hascomputer_dialupinternetonly_65over']]
data['Internet:Has a Computer with Dial Up Only'] = sum(thelist)
data['Internet%:Has a Computer with Dial Up Only'] = percent(data['Internet:Has a Computer with Dial Up Only'], data['Internet:Has a Computer'])
data['Internet:Has a Computer with Dial Up Only Under 18'] = data['compinternet_hascomputer_dialupinternetonly_u18']
data['Internet%:Under 18 has a Computer with Dial Up Only'] = percent(data['Internet:Has a Computer with Dial Up Only Under 18'], data['compinternet_total_u18'])
data['Internet:Has a Computer with Dial Up Only 18 to 64'] = data['compinternet_hascomputer_dialupinternetonly_18to64']
data['Internet%:18 to 64 has a Computer with Dial Up Only'] = percent(data['Internet:Has a Computer with Dial Up Only 18 to 64'], data['compinternet_total_18to64'])
data['Internet:Has a Computer with Dial Up Only 65 Over'] = data['compinternet_hascomputer_dialupinternetonly_65over']
data['Internet%:65 Over has a Computer with Dial Up Only'] = percent(data['Internet:Has a Computer with Dial Up Only 65 Over'], data['compinternet_total_65over'])
thelist = [data['compinternet_hascomputer_broadbandinternet_u18'], data['compinternet_hascomputer_broadbandinternet_18to64'], 
           data['compinternet_hascomputer_broadbandinternet_65over']]
data['Internet:Has a Computer with Broadband'] = sum(thelist)
data['Internet%:Has a Computer with Broadband'] = percent(data['Internet:Has a Computer with Broadband'], data['Internet:Has a Computer'])
data['Internet:Has a Computer with Broadband Under 18'] = data['compinternet_hascomputer_broadbandinternet_u18']
data['Internet%:Under 18 has a Computer with Broadband'] = percent(data['Internet:Has a Computer with Broadband Under 18'], data['compinternet_total_u18'])
data['Internet:Has a Computer with Broadband 18 to 64'] = data['compinternet_hascomputer_broadbandinternet_18to64']
data['Internet%:18 to 64 has a Computer with Broadband'] = percent(data['Internet:Has a Computer with Broadband 18 to 64'], data['compinternet_total_18to64'])
data['Internet:Has a Computer with Broadband 65 Over'] = data['compinternet_hascomputer_broadbandinternet_65over']
data['Internet%:65 Over has a Computer with Broadband'] = percent(data['Internet:Has a Computer with Broadband 65 Over'], data['compinternet_total_65over'])
thelist = [data['compinternet_hascomputer_withoutinternet_u18'], data['compinternet_hascomputer_withoutinternet_18to64'], 
           data['compinternet_hascomputer_withoutinternet_65over']]
data['Internet:Has a Computer with No Internet'] = sum(thelist)
data['Internet%:Has a Computer with No Internet'] = percent(data['Internet:Has a Computer with No Internet'], data['Internet:Has a Computer'])
data['Internet:Has a Computer with No Internet Under 18'] = data['compinternet_hascomputer_withoutinternet_u18']
data['Internet%:Under 18 has a Computer with No Internet'] = percent(data['Internet:Has a Computer with No Internet Under 18'], data['compinternet_total_u18'])
data['Internet:Has a Computer with No Internet 18 to 64'] = data['compinternet_hascomputer_withoutinternet_18to64']
data['Internet%:18 to 64 has a Computer with No Internet'] = percent(data['Internet:Has a Computer with No Internet 18 to 64'], data['compinternet_total_18to64'])
data['Internet:Has a Computer with No Internet 65 Over'] = data['compinternet_hascomputer_withoutinternet_65over']
data['Internet%:65 Over has a Computer with No Internet'] = percent(data['Internet:Has a Computer with No Internet 65 Over'], data['compinternet_total_65over'])
thelist = [data['compinternet_nocomputer_u18'], data['compinternet_nocomputer_18to64'], data['compinternet_nocomputer_65over']]
data['Internet:Has No Computer'] = sum(thelist)
data['Internet%:Has No Computer'] = percent(data['Internet:Has No Computer'], data['compinternet_series_total'])
data['Internet:Has No Computer Under 18'] = data['compinternet_nocomputer_u18']
data['Internet%:Under 18 has No Computer'] = percent(data['Internet:Has No Computer Under 18'], data['compinternet_total_u18'])
data['Internet:Has No Computer 18 to 64'] = data['compinternet_nocomputer_18to64']
data['Internet%:18 to 64 has No Computer'] = percent(data['Internet:Has No Computer 18 to 64'], data['compinternet_total_18to64'])
data['Internet:Has No Computer 65 Over'] = data['compinternet_nocomputer_65over']
data['Internet%:65 Over has No Computer'] = percent(data['Internet:Has No Computer 65 Over'], data['compinternet_total_65over'])
#vehicles available by age
thelist = [data['vehicles_tenureage_ownerocc_novehicle_65over'], data['vehicles_tenureage_ownerocc_1ormorevehicles_65over'], 
           data['vehicles_tenureage_renterocc_novehicle_65over'], data['vehicles_tenureage_renterocc_1ormorevehicles_65over']]
sixtyfiveover = sum(thelist)
thelist = [data['vehicles_tenureage_ownerocc_novehicle_65over'], data['vehicles_tenureage_renterocc_novehicle_65over']]
data['Vehicles:No Vehicle Available 65 Over'] = sum(thelist)
data['Vehicles%:65 Over No Vehicle Available'] = percent(data['Vehicles:No Vehicle Available 65 Over'], sixtyfiveover)
thelist = [data['vehicles_tenureage_ownerocc_1ormorevehicles_65over'], data['vehicles_tenureage_renterocc_1ormorevehicles_65over']]
data['Vehicles:1 or More Vehicles Available 65 Over'] = sum(thelist)
data['Vehicles%:65 Over 1 or More Vehicles Available'] = percent(data['Vehicles:1 or More Vehicles Available 65 Over'], sixtyfiveover)
#veteran status for seniors
allveterans = data['veteran_total_veteran']
thelist = [data['veteran_m55to64'], data['veteran_m65to74'], data['veteran_m75+'], 
           data['veteran_f55to64'], data['veteran_f65to74'],data['veteran_f75+']]
fiftyfiveover = sum(thelist)
thelist = [data['veteran_m65to74'], data['veteran_m75+'], data['veteran_f65to74'],data['veteran_f75+']]
sixtyfiveover = sum(thelist)
thelist = [data['veteran_m55to64_veteran'], data['veteran_m65to74_veteran'], data['veteran_m75+_veteran'], 
           data['veteran_f55to64_veteran'], data['veteran_f65to74_veteran'], data['veteran_f75+_veteran']]
data['Veteran:55 Over'] = sum(thelist)
data['Veteran%:55 Over Veterans'] = percent(data['Veteran:55 Over'], fiftyfiveover)
data['Veteran%:Veterans 55 Over'] = percent(data['Veteran:55 Over'], allveterans)
thelist = [data['veteran_m65to74_veteran'], data['veteran_m75+_veteran'], data['veteran_f65to74_veteran'], data['veteran_f75+_veteran']]
data['Veteran:65 Over'] = sum(thelist)
data['Veteran%:65 Over Veterans'] = percent(data['Veteran:65 Over'], sixtyfiveover)
data['Veteran%:Veterans 65 Over'] = percent(data['Veteran:65 Over'], allveterans)
#household income by age
under25 = data['hhincome_byhholderage_total_u25']
data['HHIncome:Under 25 Less than 10,000'] = data['hhincome_byhholderage_lessthan10000_u25']
data['HHIncome%:Under 25 Less than 10,000'] = percent(data['HHIncome:Under 25 Less than 10,000'], under25)
data['HHIncome:Under 25 10 to 14,999'] = data['hhincome_byhholderage_10to14999_u25']
data['HHIncome%:Under 25 10 to 14,999'] = percent(data['HHIncome:Under 25 10 to 14,999'], under25)
data['HHIncome:Under 25 15 to 19,999'] = data['hhincome_byhholderage_15to19999_u25']
data['HHIncome%:Under 25 15 to 19,999'] = percent(data['HHIncome:Under 25 15 to 19,999'], under25)
data['HHIncome:Under 25 20 to 24,999'] = data['hhincome_byhholderage_20to24999_u25']
data['HHIncome%:Under 25 20 to 24,999'] = percent(data['HHIncome:Under 25 20 to 24,999'], under25)
data['HHIncome:Under 25 25 to 29,999'] = data['hhincome_byhholderage_25to29999_u25']
data['HHIncome%:Under 25 25 to 29,999'] = percent(data['HHIncome:Under 25 25 to 29,999'], under25)
data['HHIncome:Under 25 30 to 34,999'] = data['hhincome_byhholderage_30to34999_u25']
data['HHIncome%:Under 25 30 to 34,999'] = percent(data['HHIncome:Under 25 30 to 34,999'], under25)
data['HHIncome:Under 25 35 to 39,999'] = data['hhincome_byhholderage_35to39999_u25']
data['HHIncome%:Under 25 35 to 39,999'] = percent(data['HHIncome:Under 25 35 to 39,999'], under25)
data['HHIncome:Under 25 40 to 44,999'] = data['hhincome_byhholderage_40to44999_u25']
data['HHIncome%:Under 25 40 to 44,999'] = percent(data['HHIncome:Under 25 40 to 44,999'], under25)
data['HHIncome:Under 25 45 to 49,999'] = data['hhincome_byhholderage_45to49999_u25']
data['HHIncome%:Under 25 45 to 49,999'] = percent(data['HHIncome:Under 25 45 to 49,999'], under25)
data['HHIncome:Under 25 50 to 59,999'] = data['hhincome_byhholderage_50to59999_u25']
data['HHIncome%:Under 25 50 to 59,999'] = percent(data['HHIncome:Under 25 50 to 59,999'], under25)
data['HHIncome:Under 25 60 to 74,999'] = data['hhincome_byhholderage_60to74999_u25']
data['HHIncome%:Under 25 60 to 74,999'] = percent(data['HHIncome:Under 25 60 to 74,999'], under25)
data['HHIncome:Under 25 75 to 99,999'] = data['hhincome_byhholderage_75to99999_u25']
data['HHIncome%:Under 25 75 to 99,999'] = percent(data['HHIncome:Under 25 75 to 99,999'], under25)
data['HHIncome:Under 25 100 to 124,999'] = data['hhincome_byhholderage_100to124999_u25']
data['HHIncome%:Under 25 100 to 124,999'] = percent(data['HHIncome:Under 25 100 to 124,999'], under25)
data['HHIncome:Under 25 125 to 149,999'] = data['hhincome_byhholderage_125to149999_u25']
data['HHIncome%:Under 25 125 to 149,999'] = percent(data['HHIncome:Under 25 125 to 149,999'], under25)
data['HHIncome:Under 25 150 to 199,999'] = data['hhincome_byhholderage_150to199999_u25']
data['HHIncome%:Under 25 150 to 199,999'] = percent(data['HHIncome:Under 25 150 to 199,999'], under25)
data['HHIncome:Under 25 200K or More'] = data['hhincome_byhholderage_200ormore_u25']
data['HHIncome%:Under 25 200K or More'] = percent(data['HHIncome:Under 25 200K or More'], under25)
twenty5to44 = data['hhincome_byhholderage_total_25to44']
data['HHIncome:25 to 44 Less than 10,000'] = data['hhincome_byhholderage_lessthan10000_25to44']
data['HHIncome%:25 to 44 Less than 10,000'] = percent(data['HHIncome:25 to 44 Less than 10,000'], twenty5to44)
data['HHIncome:25 to 44 10 to 14,999'] = data['hhincome_byhholderage_10to14999_25to44']
data['HHIncome%:25 to 44 10 to 14,999'] = percent(data['HHIncome:25 to 44 10 to 14,999'], twenty5to44)
data['HHIncome:25 to 44 15 to 19,999'] = data['hhincome_byhholderage_15to19999_25to44']
data['HHIncome%:25 to 44 15 to 19,999'] = percent(data['HHIncome:25 to 44 15 to 19,999'], twenty5to44)
data['HHIncome:25 to 44 20 to 24,999'] = data['hhincome_byhholderage_20to24999_25to44']
data['HHIncome%:25 to 44 20 to 24,999'] = percent(data['HHIncome:25 to 44 20 to 24,999'], twenty5to44)
data['HHIncome:25 to 44 25 to 29,999'] = data['hhincome_byhholderage_25to29999_25to44']
data['HHIncome%:25 to 44 25 to 29,999'] = percent(data['HHIncome:25 to 44 25 to 29,999'], twenty5to44)
data['HHIncome:25 to 44 30 to 34,999'] = data['hhincome_byhholderage_30to34999_25to44']
data['HHIncome%:25 to 44 30 to 34,999'] = percent(data['HHIncome:25 to 44 30 to 34,999'], twenty5to44)
data['HHIncome:25 to 44 35 to 39,999'] = data['hhincome_byhholderage_35to39999_25to44']
data['HHIncome%:25 to 44 35 to 39,999'] = percent(data['HHIncome:25 to 44 35 to 39,999'], twenty5to44)
data['HHIncome:25 to 44 40 to 44,999'] = data['hhincome_byhholderage_40to44999_25to44']
data['HHIncome%:25 to 44 40 to 44,999'] = percent(data['HHIncome:25 to 44 40 to 44,999'], twenty5to44)
data['HHIncome:25 to 44 45 to 49,999'] = data['hhincome_byhholderage_45to49999_25to44']
data['HHIncome%:25 to 44 45 to 49,999'] = percent(data['HHIncome:25 to 44 45 to 49,999'], twenty5to44)
data['HHIncome:25 to 44 50 to 59,999'] = data['hhincome_byhholderage_50to59999_25to44']
data['HHIncome%:25 to 44 50 to 59,999'] = percent(data['HHIncome:25 to 44 50 to 59,999'], twenty5to44)
data['HHIncome:25 to 44 60 to 74,999'] = data['hhincome_byhholderage_60to74999_25to44']
data['HHIncome%:25 to 44 60 to 74,999'] = percent(data['HHIncome:25 to 44 60 to 74,999'], twenty5to44)
data['HHIncome:25 to 44 75 to 99,999'] = data['hhincome_byhholderage_75to99999_25to44']
data['HHIncome%:25 to 44 75 to 99,999'] = percent(data['HHIncome:25 to 44 75 to 99,999'], twenty5to44)
data['HHIncome:25 to 44 100 to 124,999'] = data['hhincome_byhholderage_100to124999_25to44']
data['HHIncome%:25 to 44 100 to 124,999'] = percent(data['HHIncome:25 to 44 100 to 124,999'], twenty5to44)
data['HHIncome:25 to 44 125 to 149,999'] = data['hhincome_byhholderage_125to149999_25to44']
data['HHIncome%:25 to 44 125 to 149,999'] = percent(data['HHIncome:25 to 44 125 to 149,999'], twenty5to44)
data['HHIncome:25 to 44 150 to 199,999'] = data['hhincome_byhholderage_150to199999_25to44']
data['HHIncome%:25 to 44 150 to 199,999'] = percent(data['HHIncome:25 to 44 150 to 199,999'], twenty5to44)
data['HHIncome:25 to 44 200K or More'] = data['hhincome_byhholderage_200ormore_25to44']
data['HHIncome%:25 to 44 200K or More'] = percent(data['HHIncome:25 to 44 200K or More'], twenty5to44)
forty5to64 = data['hhincome_byhholderage_total_45to64']
data['HHIncome:45 to 64 Less than 10,000'] = data['hhincome_byhholderage_lessthan10000_45to64']
data['HHIncome%:45 to 64 Less than 10,000'] = percent(data['HHIncome:45 to 64 Less than 10,000'], forty5to64)
data['HHIncome:45 to 64 10 to 14,999'] = data['hhincome_byhholderage_10to14999_45to64']
data['HHIncome%:45 to 64 10 to 14,999'] = percent(data['HHIncome:45 to 64 10 to 14,999'], forty5to64)
data['HHIncome:45 to 64 15 to 19,999'] = data['hhincome_byhholderage_15to19999_45to64']
data['HHIncome%:45 to 64 15 to 19,999'] = percent(data['HHIncome:45 to 64 15 to 19,999'], forty5to64)
data['HHIncome:45 to 64 20 to 24,999'] = data['hhincome_byhholderage_20to24999_45to64']
data['HHIncome%:45 to 64 20 to 24,999'] = percent(data['HHIncome:45 to 64 20 to 24,999'], forty5to64)
data['HHIncome:45 to 64 25 to 29,999'] = data['hhincome_byhholderage_25to29999_45to64']
data['HHIncome%:45 to 64 25 to 29,999'] = percent(data['HHIncome:45 to 64 25 to 29,999'], forty5to64)
data['HHIncome:45 to 64 30 to 34,999'] = data['hhincome_byhholderage_30to34999_45to64']
data['HHIncome%:45 to 64 30 to 34,999'] = percent(data['HHIncome:45 to 64 30 to 34,999'], forty5to64)
data['HHIncome:45 to 64 35 to 39,999'] = data['hhincome_byhholderage_35to39999_45to64']
data['HHIncome%:45 to 64 35 to 39,999'] = percent(data['HHIncome:45 to 64 35 to 39,999'], forty5to64)
data['HHIncome:45 to 64 40 to 44,999'] = data['hhincome_byhholderage_40to44999_45to64']
data['HHIncome%:45 to 64 40 to 44,999'] = percent(data['HHIncome:45 to 64 40 to 44,999'], forty5to64)
data['HHIncome:45 to 64 45 to 49,999'] = data['hhincome_byhholderage_45to49999_45to64']
data['HHIncome%:45 to 64 45 to 49,999'] = percent(data['HHIncome:45 to 64 45 to 49,999'], forty5to64)
data['HHIncome:45 to 64 50 to 59,999'] = data['hhincome_byhholderage_50to59999_45to64']
data['HHIncome%:45 to 64 50 to 59,999'] = percent(data['HHIncome:45 to 64 50 to 59,999'], forty5to64)
data['HHIncome:45 to 64 60 to 74,999'] = data['hhincome_byhholderage_60to74999_45to64']
data['HHIncome%:45 to 64 60 to 74,999'] = percent(data['HHIncome:45 to 64 60 to 74,999'], forty5to64)
data['HHIncome:45 to 64 75 to 99,999'] = data['hhincome_byhholderage_75to99999_45to64']
data['HHIncome%:45 to 64 75 to 99,999'] = percent(data['HHIncome:45 to 64 75 to 99,999'], forty5to64)
data['HHIncome:45 to 64 100 to 124,999'] = data['hhincome_byhholderage_100to124999_45to64']
data['HHIncome%:45 to 64 100 to 124,999'] = percent(data['HHIncome:45 to 64 100 to 124,999'], forty5to64)
data['HHIncome:45 to 64 125 to 149,999'] = data['hhincome_byhholderage_125to149999_45to64']
data['HHIncome%:45 to 64 125 to 149,999'] = percent(data['HHIncome:45 to 64 125 to 149,999'], forty5to64)
data['HHIncome:45 to 64 150 to 199,999'] = data['hhincome_byhholderage_150to199999_45to64']
data['HHIncome%:45 to 64 150 to 199,999'] = percent(data['HHIncome:45 to 64 150 to 199,999'], forty5to64)
data['HHIncome:45 to 64 200K or More'] = data['hhincome_byhholderage_200ormore_45to64']
data['HHIncome%:45 to 64 200K or More'] = percent(data['HHIncome:45 to 64 200K or More'], forty5to64)
sixty5over = data['hhincome_byhholderage_total_65over']
data['HHIncome:65 Over Less than 10,000'] = data['hhincome_byhholderage_lessthan10000_65over']
data['HHIncome%:65 Over Less than 10,000'] = percent(data['HHIncome:65 Over Less than 10,000'], sixty5over)
data['HHIncome:65 Over 10 to 14,999'] = data['hhincome_byhholderage_10to14999_65over']
data['HHIncome%:65 Over 10 to 14,999'] = percent(data['HHIncome:65 Over 10 to 14,999'], sixty5over)
data['HHIncome:65 Over 15 to 19,999'] = data['hhincome_byhholderage_15to19999_65over']
data['HHIncome%:65 Over 15 to 19,999'] = percent(data['HHIncome:65 Over 15 to 19,999'], sixty5over)
data['HHIncome:65 Over 20 to 24,999'] = data['hhincome_byhholderage_20to24999_65over']
data['HHIncome%:65 Over 20 to 24,999'] = percent(data['HHIncome:65 Over 20 to 24,999'], sixty5over)
data['HHIncome:65 Over 25 to 29,999'] = data['hhincome_byhholderage_25to29999_65over']
data['HHIncome%:65 Over 25 to 29,999'] = percent(data['HHIncome:65 Over 25 to 29,999'], sixty5over)
data['HHIncome:65 Over 30 to 34,999'] = data['hhincome_byhholderage_30to34999_65over']
data['HHIncome%:65 Over 30 to 34,999'] = percent(data['HHIncome:65 Over 30 to 34,999'], sixty5over)
data['HHIncome:65 Over 35 to 39,999'] = data['hhincome_byhholderage_35to39999_65over']
data['HHIncome%:65 Over 35 to 39,999'] = percent(data['HHIncome:65 Over 35 to 39,999'], sixty5over)
data['HHIncome:65 Over 40 to 44,999'] = data['hhincome_byhholderage_40to44999_65over']
data['HHIncome%:65 Over 40 to 44,999'] = percent(data['HHIncome:65 Over 40 to 44,999'], sixty5over)
data['HHIncome:65 Over 45 to 49,999'] = data['hhincome_byhholderage_45to49999_65over']
data['HHIncome%:65 Over 45 to 49,999'] = percent(data['HHIncome:65 Over 45 to 49,999'], sixty5over)
data['HHIncome:65 Over 50 to 59,999'] = data['hhincome_byhholderage_50to59999_65over']
data['HHIncome%:65 Over 50 to 59,999'] = percent(data['HHIncome:65 Over 50 to 59,999'], sixty5over)
data['HHIncome:65 Over 60 to 74,999'] = data['hhincome_byhholderage_60to74999_65over']
data['HHIncome%:65 Over 60 to 74,999'] = percent(data['HHIncome:65 Over 60 to 74,999'], sixty5over)
data['HHIncome:65 Over 75 to 99,999'] = data['hhincome_byhholderage_75to99999_65over']
data['HHIncome%:65 Over 75 to 99,999'] = percent(data['HHIncome:65 Over 75 to 99,999'], sixty5over)
data['HHIncome:65 Over 100 to 124,999'] = data['hhincome_byhholderage_100to124999_65over']
data['HHIncome%:65 Over 100 to 124,999'] = percent(data['HHIncome:65 Over 100 to 124,999'], sixty5over)
data['HHIncome:65 Over 125 to 149,999'] = data['hhincome_byhholderage_125to149999_65over']
data['HHIncome%:65 Over 125 to 149,999'] = percent(data['HHIncome:65 Over 125 to 149,999'], sixty5over)
data['HHIncome:65 Over 150 to 199,999'] = data['hhincome_byhholderage_150to199999_65over']
data['HHIncome%:65 Over 150 to 199,999'] = percent(data['HHIncome:65 Over 150 to 199,999'], sixty5over)
data['HHIncome:65 Over 200K or More'] = data['hhincome_byhholderage_200ormore_65over']
data['HHIncome%:65 Over 200K or More'] = percent(data['HHIncome:65 Over 200K or More'], sixty5over)
#median household income by householder age
data['HHIncome:Median for Householder Under 25'] = data['hhincome_median_hholderu25']
data['HHIncome:Median for Householder Under 25 to 44'] = data['hhincome_median_hholder25to44']
data['HHIncome:Median for Householder Under 45 to 64'] = data['hhincome_median_hholder45to64']
data['HHIncome:Median for Householder Under 65 Over'] = data['hhincome_median_hholder65over']

In [5]:
#WORK AROUNDS FOR CHANGE METRICS
#median home value in 2000 is both owner occ with and without a mortgage combined, for 2010 and 2020 calling owner occupied with a mortgage this and dropping split
data['Median Home Value Owner Occupied'] = data['housingcost_medvalue_ownerocc_wmortgage']
data = data.drop(columns = ['Median Home Value Owner Occupied with Mortgage', 'Median Home Value Owner Occupied without Mortgage'])

In [6]:
cols = coldropper.columns
data = data.drop(columns = cols)
data = data.drop(columns = ['Population'])

In [7]:
data = data.set_index('NAME').transpose()
#rename geographies dict
data = data.rename(columns = {'Allen County, Kentucky': 'Allen County, KY', 'Cheatham County, Tennessee': 'Cheatham County', 
                              'Davidson County, Tennessee': 'Davidson County', 'Dickson County, Tennessee': 'Dickson County', 
                              'Houston County, Tennessee': 'Houston County', 'Humphreys County, Tennessee': 'Humphreys County', 
                              'Maury County, Tennessee': 'Maury County', 'Montgomery County, Tennessee': 'Montgomery County', 
                              'Robertson County, Tennessee': 'Robertson County', 'Rutherford County, Tennessee': 'Rutherford County', 
                              'Simpson County, Kentucky': 'Simpson County, KY', 'Stewart County, Tennessee': 'Stewart County', 
                              'Sumner County, Tennessee': 'Sumner County', 'Trousdale County, Tennessee': 'Trousdale County', 
                              'Williamson County, Tennessee': 'Williamson County', 'Wilson County, Tennessee': 'Wilson County', 
                              'Adams city, Tennessee': 'Adams', 'Ashland City town, Tennessee': 'Ashland City', 'Belle Meade city, Tennessee': 'Belle Meade', 
                              'Berry Hill city, Tennessee': 'Berry Hill', 'Brentwood city, Tennessee': 'Brentwood', 'Burns town, Tennessee': 'Burns', 
                              'Cedar Hill city, Tennessee': 'Cedar Hill', 'Charlotte town, Tennessee': 'Charlotte', 'Clarksville city, Tennessee': 'Clarksville', 
                              'Columbia city, Tennessee': 'Columbia', 'Coopertown town, Tennessee': 'Coopertown', 'Cross Plains city, Tennessee': 'Cross Plains', 
                              'Cumberland City town, Tennessee': 'Cumberland City', 'Dickson city, Tennessee': 'Dickson', 'Dover city, Tennessee': 'Dover', 
                              'Eagleville city, Tennessee': 'Eagleville', 'Erin city, Tennessee': 'Erin', 'Fairview city, Tennessee': 'Fairview', 
                              'Forest Hills city, Tennessee': 'Forest Hills', 'Franklin city, Tennessee': 'Franklin', 'Gallatin city, Tennessee': 'Gallatin', 
                              'Goodlettsville city, Tennessee': 'Goodlettsville', 'Greenbrier town, Tennessee': 'Greenbrier', 
                              'Hendersonville city, Tennessee': 'Hendersonville', 'Kingston Springs town, Tennessee': 'Kingston Springs', 
                              'La Vergne city, Tennessee': 'La Vergne', 'Lafayette city, Tennessee': 'Lafayette', 'Lebanon city, Tennessee': 'Lebanon', 
                              'McEwen city, Tennessee': 'McEwen', 'Millersville city, Tennessee': 'Millersville', 'Mitchellville city, Tennessee': 'Mitchellville', 
                              'Mount Juliet city, Tennessee': 'Mount Juliet', 'Mount Pleasant city, Tennessee': 'Mount Pleasant', 
                              'Murfreesboro city, Tennessee': 'Murfreesboro', 'Nashville-Davidson metropolitan government (balance)': 'Nashville', 
                              'New Johnsonville city, Tennessee': 'New Johnsonville', 'Nolensville town, Tennessee': 'Nolensville', 
                              'Oak Hill city, Tennessee': 'Oak Hill', 'Pegram town, Tennessee': 'Pegram', 'Pleasant View city, Tennessee': 'Pleasant View', 
                              'Portland city, Tennessee': 'Portland', 'Ridgetop city, Tennessee': 'Ridgetop', 'Slayden town, Tennessee': 'Slayden', 
                              'Smyrna town, Tennessee': 'Smyrna', 'Spring Hill city, Tennessee': 'Spring Hill', 'Springfield city, Tennessee': 'Springfield', 
                              'Tennessee Ridge town, Tennessee': 'Tennessee Ridge', "Thompson's Station town, Tennessee": "Thompson's Station", 
                              'Vanleer town, Tennessee': 'Vanleer', 'Watertown city, Tennessee': 'Watertown', 'Waverly city, Tennessee': 'Waverly', 
                              'Westmoreland town, Tennessee': 'Westmoreland', 'White Bluff town, Tennessee': 'White Bluff', 
                              'White House city, Tennessee': 'White House', 'Franklin city, Kentucky': 'Franklin, KY', 
                              'Scottsville city, Kentucky': 'Scottsville, KY', 'United States': 'US'})
data = data.transpose().reset_index(drop = False)

In [8]:
data.to_csv('../../Outputs/2020ACS5YR.csv', index = False)

In [9]:
data.head()

,NAME,GEO_ID,HealthCoverage:Total Series,HealthCoverage:None,HealthCoverage%: None,HealthCoverage: With Healthcare Coverage,HealthCoverage%: With Healthcare Coverage,HealthCoverage: With Public Healthcare Coverage,HealthCoverage%: With Public Healthcare Coverage,HealthCoverage: With Private Healthcare Coverage,HealthCoverage%: With Private Healthcare Coverage,Male Under 5,Female Under 5,Male 5 to 9,Female 5 to 9,Male 5 to 14,Female 5 to 14,Male 15 to 24,Female 15 to 24,Male 10 to 14,Female 10 to 14,Male 15 to 17,Female 15 to 17,Male 18 to 24,Female 18 to 24,Male 25 to 34,Female 25 to 34,Male 35 to 44,Female 35 to 44,Male 45 to 54,Female 45 to 54,Male 55 to 64,Female 55 to 64,Male 65 to 74,Female 65 to 74,Male 75 to 84,Female 75 to 84,Male 85 and Older,Female 85 and Older,Age:Under 5,Age %:Under 5,Age:5 to 9,Age%:5 to 9,Age:10 to 14,Age%:10 to 14,Age:15 to 17,Age%:15 to 17,Age:18 to 24,Age%:18 to 24,Age:25 to 34,Age%:25 to 34,Age:35 to 44,Age%:35 to 44,Age:45 to 54,Age%:45 to 54,Age:55 to 64,Age%:55 to 64,Age:65 to 74,Age%:65 to 74,Age:75 to 84,Age%:75 to 84,Age:85 and Older,Age%:85 and Older,Age:School Age 5 to 18,Age%:School Age 5 to 18,Age:Under 18,Age%:Under 18,Age:18 to 54,Age%:18 to 54,Age:18 to 64,Age%:18 to 64,Age:55 and Older,Age%:55 and Older,Age:65 and Older,Age%:65 and Older,Total Households,Family Households,Family Households: Married Couple Family,"Household Type%: Family, Married Couple Households",Family Households: Not Married Couple Family,"Household Type%: Family, Not Married Couple Households",Family Households: Not Married Couple: Male no Spouse,Family Households: Not Married Couple: Female no Spouse,Nonfamily Households,Nonfamily Households: Householder Alone,"Household Type%: Nonfamily, Householder Alone",Nonfamily Households: Householder not Alone,"Household Type%: Nonfamily, Householder Not Alone",Average Household Size,Median Household Income,HHIncome:Total Households,"HHIncome:Less than 10,000","HHIncome%:Less than 10,000","HHIncome:10 to 14,999","HHIncome%:10 to 14,999","HHIncome:15 to 19,999","HHIncome%:15 to 19,999","HHIncome:20 to 24,999","HHIncome%:20 to 24,999","HHIncome:25 to 29,999","HHIncome%:25 to 29,999","HHIncome:30 to 34,999","HHIncome:%30 to 34,999","HHIncome:35 to 39,999","HHIncome%:35 to 39,999","HHIncome:40 to 44,999","HHIncome%:40 to 44,999","HHIncome:45 to 49,999","HHIncome%:45 to 49,999","HHIncome:50 to 59,999","HHIncome%:50 to 59,999","HHIncome:60 to 74,999","HHIncome%:60 to 74,999","HHIncome:75 to 99,999","HHIncome%:75 to 99,999","HHIncome:100 to 124,999","HHIncome%:100 to 124,999","HHIncome:125 to 149,999","HHIncome%:125 to 149,999","HHIncome:150 to 199,999","HHIncome%:150 to 199,999",HHIncome:200K or More,HHIncome%:200K or More,Ed:Population 25+ Educational Attainment,Ed:Less than High School,Ed%:Less than High School,Ed:High School Graduate or Equivalency,Ed%:High School Graduate or Equivalency,Ed%:High School Graduate or More,Ed:Some College,Ed%:Some College,Ed%:Some College or More,Ed:Associates,Ed%:Associates,Ed:Bachelors,Ed%:Bachelors,Ed%:Bachelors or More,Ed:Masters,Ed%:Masters or More,Ed%:Masters,Ed:Professional School Degree,Ed%:Professional School Degree,Ed%:Professional School Degree or More,Ed:Doctorate Degree,Ed%:Doctorate Degree,Ind:Employed Civilians 16+,"Ind:Agriculture, Forestry, Fishing and Hunting, and Mining","Ind%:Agriculture, Forestry, Fishing and Hunting, and Mining","Ind:Agriculture, Forestry, Fishing and Hunting","Ind%:Agriculture, Forestry, Fishing and Hunting","Ind:Mining, Quarrying, Oil and Gas","Ind%:Mining, Quarrying, Oil and Gas",Ind:Construction,Ind%:Construction,Ind:Manufacturing,Ind%:Manufacturing,Ind:Wholesale Trade,Ind%:Wholesale Trade,Ind:Retail Trade,Ind%:Retail Trade,"Ind:Transportation and Warehousing, Utilities","Ind%:Transportation and Warehousing, Utilities",Ind:Information,Ind%:Information,"Ind:Finance, Insurance, Real Estate and Rental and Leasing","Ind%:Finance, Insurance, Real Estate and Rental and Leasing","Ind:Professional, Scient